# Cluster and HII-regions Multi <a class="tocSkip">

the aim of this notebook is to combine the HII-region and cluster catalogues.
   
This notebook useses multiple galaxies at once.

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from astrotools.packages import *
from astrotools.constants import tab10, single_column, two_column, thesis_width

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
logger = logging.getLogger('pymuse')
handler = logging.StreamHandler(stream=sys.stdout)
handler.setLevel(logging.INFO)
fmt = logging.Formatter("%(asctime)-15s %(message)s",datefmt='%H:%M:%S')
handler.setFormatter(fmt)
logger.addHandler(handler)

# first we need to specify the path to the raw data
basedir = Path('..')
data_ext = Path('a:')/'Archive' #basedir / 'data' / 'raw' 

sample_table = ascii.read(basedir/'..'/'pnlf'/'data'/'interim'/'sample.txt')
sample_table.add_index('name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])
sample_table['power_index'] = 2.3
sample_table['power_index'][sample_table['AO'].mask]=2.8
sample_table['distance'] = Distance(distmod=sample_table['(m-M)'])

sample_table_v1p6 = Table.read(basedir/'data'/'external'/'phangs_sample_table_v1p6.fits')
sample_table_v1p6 = sample_table_v1p6[sample_table_v1p6['survey_muse_status']=='released'].copy()
sample_table_v1p6['name'] = [x.upper() for x in sample_table_v1p6['name']]
sample_table_v1p6['dist_err'] = sample_table_v1p6['dist']*sample_table_v1p6['dist_unc']*np.log(10)
sample_table_v1p6.add_index('name')
# we are interested in props_mstar, dist, dist_err, dist_unc

## Read in data

In [ ]:
# choose which version of the association catalogue to use
version = 'v1p2'
HSTband = 'nuv'
scalepc = 32

### The nebulae catalogue

707 objects are removed as stars or too close to the edge

In [ ]:
# the original catalogue from Francesco
with fits.open(basedir / 'data' / 'interim' / 'Nebulae_Catalogue_v3.fits') as hdul:
    nebulae = Table(hdul[1].data)
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg,frame='icrs')
del nebulae[['EW_HA','EW_HA_ERR']]
nebulae.rename_columns(['cen_x','cen_y','cen_ra','cen_dec','region_area',
                          'EBV','EBV_ERR','SkyCoord'],
                         ['x_neb','y_neb','ra_neb','dec_neb','area_neb',
                          'EBV_balmer','EBV_balmer_err','SkyCoord_neb'])
    
# some additional properties 
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_dig.fits') as hdul:
    dig = Table(hdul[1].data)
dig['dig/hii'] = dig['dig_median'] / dig['hii_median']

with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_FUV_bkg.fits') as hdul:
    fuv = Table(hdul[1].data)
    
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_EW.fits') as hdul:
    eq_width = Table(hdul[1].data)
    
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_density_refit.fits') as hdul:
    density = Table(hdul[1].data)
    
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_refitNII.fits') as hdul:
    refitNII = Table(hdul[1].data)
    
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_in_frame.fits') as hdul:
    in_frame = Table(hdul[1].data)
    
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_CO.fits') as hdul:
    CO = Table(hdul[1].data)
    
nebulae = join(nebulae,fuv,keys=['gal_name','region_ID'])
nebulae = join(nebulae,eq_width,keys=['gal_name','region_ID'])
nebulae = join(nebulae,dig,keys=['gal_name','region_ID'])
nebulae = join(nebulae,density,keys=['gal_name','region_ID'])
nebulae = join(nebulae,refitNII,keys=['gal_name','region_ID'])
nebulae = join(nebulae,in_frame,keys=['gal_name','region_ID'])
nebulae = join(nebulae,CO,keys=['gal_name','region_ID'])

# this will rais a few errors that we just ignore
with np.errstate(divide='ignore',invalid='ignore'):
    nebulae['[SIII]/[SII]'] = np.nan
    SII = nebulae['SII6716_FLUX_CORR']+nebulae['SII6730_FLUX_CORR']
    SII_ERR = np.sqrt(nebulae['SII6716_FLUX_CORR_ERR']**2+nebulae['SII6730_FLUX_CORR_ERR']**2)
    SIII = (1+2.47)*nebulae['SIII9068_FLUX_CORR']
    SIII_ERR = 3.47 * nebulae['SIII9068_FLUX_CORR_ERR']
    
    nebulae['[SIII]/[SII]'][SII>0] = SIII[SII>0]/SII[SII>0]
    nebulae['[SIII]/[SII]_ERR'] = nebulae['[SIII]/[SII]'] * np.sqrt((SII_ERR/SII)**2+(SIII_ERR/SIII)**2)
    
    nebulae['HA/FUV'] = nebulae['HA6562_FLUX']/nebulae['FUV_FLUX']
    nebulae['HA/FUV_err'] = nebulae['HA/FUV']*np.sqrt((nebulae['FUV_FLUX_ERR']/nebulae['FUV_FLUX'])**2+(nebulae['HA6562_FLUX_ERR']/nebulae['HA6562_FLUX'])**2)
    nebulae['HA/FUV_corr'] = nebulae['HA6562_FLUX_CORR']/nebulae['FUV_FLUX_CORR']
    nebulae['HA/FUV_corr_err'] = nebulae['HA/FUV_corr']*np.sqrt((nebulae['FUV_FLUX_CORR_ERR']/nebulae['FUV_FLUX_CORR'])**2+(nebulae['HA6562_FLUX_CORR_ERR']/nebulae['HA6562_FLUX_CORR'])**2)

# calculate luminosity based on distance from sample table
nebulae['distance'] = np.nan
nebulae['distance_err'] = np.nan
for gal_name in np.unique(nebulae['gal_name']):
    nebulae['distance'][nebulae['gal_name']==gal_name] = sample_table_v1p6.loc[gal_name]['dist']
    nebulae['distance_err'][nebulae['gal_name']==gal_name] = sample_table_v1p6.loc[gal_name]['dist_err']
nebulae['HA6562_LUM_CORR'] = (nebulae['HA6562_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2 *4*np.pi*(nebulae['distance']*u.Mpc)**2).to(u.erg/u.s)
nebulae['HA6562_LUM_CORR_ERR'] = nebulae['HA6562_LUM_CORR'] * np.sqrt((nebulae['HA6562_FLUX_CORR_ERR']/nebulae['HA6562_FLUX_CORR'])**2+(nebulae['distance_err']/nebulae['distance'])**2)

# the nebulae catalogue with additional information
folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'
lst = []
for file in folder.glob(f'*{scalepc}pc_nebulae.fits'):
    gal_name = file.stem.split('_')[0]
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    name=gal_name.lower()
    lst.append(tbl)
nebulae_tmp = vstack(lst)

nebulae = join(nebulae,nebulae_tmp,keys=['gal_name','region_ID'],join_type='outer')
nebulae = nebulae[nebulae['flag_star']==0]
nebulae = nebulae[nebulae['flag_edge']==0]
HIIregion_mask = (nebulae['BPT_NII']==0) & (nebulae['BPT_SII']==0) & (nebulae['BPT_OI']==0)

print(f'{len(nebulae)} nebulae in initial catalogue (all galaxies)')
nebulae = nebulae[HIIregion_mask]
print(f'we use {len(lst)} galaxies with {np.sum(~nebulae["overlap_neb"].mask & (nebulae["in_frame"]))} HII regions')
# only use HII regions and only the galaxies with associations
#nebulae = nebulae[HIIregion_mask & ~nebulae["overlap_neb"].mask]
print(f'{np.sum((nebulae["overlap_neb"]>0) & (nebulae["in_frame"]))} HII regions ({100*np.sum((nebulae["overlap_neb"]>0) & (nebulae["in_frame"]))/np.sum(nebulae["in_frame"]):.1f}%) overlap with association')

### The association catalogue

In [ ]:
# those files hold the merged association catalogues
with fits.open(basedir/'data'/'interim'/f'phangshst_associations_{HSTband}_ws{scalepc}pc_{version}.fits') as hdul:
    associations = Table(hdul[1].data)
    
associations['SkyCoord'] = SkyCoord(associations['reg_ra']*u.degree,associations['reg_dec']*u.degree)
associations.rename_columns(['reg_ra','reg_dec','reg_x','reg_y',
                             'reg_dolflux_Age_MinChiSq','reg_dolflux_Mass_MinChiSq','reg_dolflux_Ebv_MinChiSq',
                             'reg_dolflux_Age_MinChiSq_err','reg_dolflux_Mass_MinChiSq_err','reg_dolflux_Ebv_MinChiSq_err',
                             'SkyCoord'],
                            ['ra_asc','dec_asc','x_asc','y_asc','age','mass',
                             'EBV_stars','age_err','mass_err','EBV_stars_err','SkyCoord_asc'])

#with fits.open(basedir/'data'/'interim'/f'association_CO_{HSTband}_ws{scalepc}pc_{version}.fits') as hdul:
#    assoc_CO = Table(hdul[1].data)
#associations = join(associations,assoc_CO,keys=['gal_name','assoc_ID'])

# Halpha measured in the association masks
#with fits.open(basedir/'data'/'interim'/f'phangshst_associations_nuv_ws32pc_v1p1_Halpha.fits') as hdul:
#    assoc_Halpha = Table(hdul[1].data)

with fits.open(basedir/'data'/'interim'/f'phangshst_associations_{HSTband}_ws{scalepc}pc_{version}_in_frame.fits') as hdul:
    in_frame = Table(hdul[1].data)
associations = join(associations,in_frame,keys=['gal_name','assoc_ID'])

# the association catalogue matched with the nebuale catalogue
folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'
lst = []
for file in folder.glob(f'*{scalepc}pc_associations.fits'):
    gal_name = file.stem.split('_')[0]
    #print(f'reading {gal_name}')
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    lst.append(tbl)
assoc_tmp = vstack(lst)
# combine both catalogues
associations = join(associations,assoc_tmp,keys=['gal_name','assoc_ID'])
print(f'{len(lst)} galaxies in sample ({np.sum(associations["in_frame"])} associations)')
print(f'{np.sum((associations["overlap_asc"]>0) & (associations["in_frame"]))} associations ({100*np.sum((associations["overlap_asc"]>0) & (associations["in_frame"]))/np.sum(associations["in_frame"]):.1f} %) overlap with an HII region')

#criteria = np.abs(associations['age']-associations['age_16'])>associations['age_err']
#criteria |= np.abs(associations['age']-associations['age_64'])>associations['age_err']
#associations['uniform_age'] = ~criteria

### The joined catalogue

In [ ]:
# we can also recreate the table from the association an nebulae catalogues
catalogue = join(nebulae[~nebulae['assoc_ID'].mask],associations,keys=['gal_name','assoc_ID','region_ID'])

# add dig and calculate galactic radius
catalogue['galactic_radius'] = np.nan
for gal_name in np.unique(catalogue['gal_name']):
    centre = sample_table.loc[gal_name]['SkyCoord']
    catalogue['galactic_radius'][catalogue['gal_name']==gal_name] = catalogue[catalogue['gal_name']==gal_name]['SkyCoord_neb'].separation(centre).to(u.arcmin)
#catalogue['HA/NUV'] = catalogue['HA6562_FLUX_CORR']/catalogue['NUV_FLUX']/1e20
    
#del catalogue[['SkyCoord_asc','SkyCoord_neb']]
#hdu = fits.BinTableHDU(catalogue,name='matched catalogue')
#hdu.writeto(basedir/'data'/'interim'/f'matched_catalogue_{HSTband}_ws{scalepc}pc_{version}.fits',overwrite=True)
    
print(f'{len(catalogue)} objects in final catalogue')

In [ ]:
hst_sample      = set(np.unique(associations['gal_name']))
astrosat_sample = set(np.unique(nebulae[~np.isnan(nebulae['FUV_FLUX'])]['gal_name']))
muse_sample     = set(sample_table['name'])
complete_sample = hst_sample & astrosat_sample & muse_sample


print(f'nebulae: {len(nebulae[np.isin(nebulae["gal_name"],list(hst_sample))])}')
print(f'associations: {len(associations)} from {len(hst_sample)} galaxies')
print(f'FUV: {len(astrosat_sample)} galaxies')
print(f'match: {len(catalogue)}')
print(f'contained: {len(catalogue[catalogue["overlap"]=="contained"])}')

In [ ]:
N_overlap = np.sum(catalogue['overlap']=='contained')
N_mass = np.sum(catalogue['mass']>1e4)
N_age = np.sum(catalogue['age']<=8)
N_final = np.sum((catalogue['age']<=8) & (catalogue['mass']>1e4) & (catalogue['overlap']=='contained'))
                 
print(f'overlap: {N_overlap}')
print(f'mass: {N_mass}')
print(f'age: {N_age}')
print(f'all: {N_final}')

### others

In [ ]:
phangs_sample_table_v1p6 = Table.read(basedir/'data'/'external'/'phangs_sample_table_v1p6.fits')
phangs_sample_table_v1p6 = phangs_sample_table_v1p6[np.isin(list(phangs_sample_table_v1p6['name']),[x.lower() for x in sample_table['name']])]

In [ ]:
phangs_sample_table_v1p6[['dist','dist_unc','dist_label','dist_ref']]

### Table to showcase the sample

sample table for paper

In [ ]:
latexdict = {'tabletype': 'table',
'col_align':'lrrrrc',
'header_start': '\\toprule',
'header_end': '\\midrule',
'data_end': '\\bottomrule',
'caption': f'Galaxy sample',
'units': {'R.A.':'(J2000)','Dec.':'(J2000)','$i$':'deg','PA':'deg','Distance':'$\si{\mega\parsec}$',
          'r25':'arcmin'},
'preamble': '\\centering',
'tablefoot': f'\\label{{tbl:sample}}'
            }

astrosat_sample = np.unique(nebulae[~np.isnan(nebulae['FUV_FLUX'])]['gal_name'])

tbl = sample_table[['name','(m-M)']]
tbl.sort('name')
tbl.add_column(Distance(distmod=tbl['(m-M)']).to(u.Mpc),index=1,name='Distance')
tbl['Distance'].info.format = '%.2f' 
del tbl['(m-M)']
tbl[r'$N_{\HII}$'] = [np.sum((nebulae['gal_name']==name) & (nebulae['in_frame'])) for name in tbl['name']]
tbl[r'$N_\mathrm{asc}$'] = [np.sum((associations['gal_name']==name) & (associations['in_frame'])) for name in tbl['name']]
tbl[r'$N_\mathrm{match}$'] = [np.sum(catalogue['gal_name']==name) for name in tbl['name']]
tbl.add_column([f'\\galaxyname{{{row["name"][:-4]}}}{{{row["name"][-4:]}}}' for row in tbl],index=0,name='Name')

tbl['\\textit{AstroSat}'] = ['\checkmark' if x else '' for x in np.isin(tbl['name'],astrosat_sample)]
del tbl['name']

tbl.add_row(['',None,np.sum(nebulae['in_frame']),np.sum(associations['in_frame']),len(catalogue),''])
ascii.write(tbl,sys.stdout, Writer = ascii.Latex,latexdict=latexdict,exclude_names=[])

overlap between the two catalogues

In [ ]:
latexdict = {'tabletype': 'table',
'header_start': '\\toprule',
'header_end': '\\midrule',
'data_end': '\\bottomrule',
'caption': f'Overlap statistics',
'preamble': '\\centering',
'col_align':'lrrrr',
'tablefoot': f'\\label{{tbl:overlap_statistics}}'
            }

# we only use nebulae in the 18 galaxies with HST associations
tmp_neb = nebulae[(~nebulae['Nassoc'].mask) & (nebulae['in_frame'])]

Nasc = []
Nneb = []
Nasc_per = []
Nneb_per = []
index = []
for i in range(5):
    index.append(i)
    
    Nneb.append(np.sum((tmp_neb["Nassoc"]==i)))
    Nneb_per.append(f'({100*np.sum((tmp_neb["Nassoc"]==i))/len(tmp_neb):.1f}\%)')
    
    Nasc.append(np.sum((associations["Nnebulae"]==i) & (associations['in_frame'])))
    Nasc_per.append(f'({100*np.sum((associations["Nnebulae"]==i) & associations["in_frame"])/np.sum(associations["in_frame"]):.1f}\%)')

    # add the last row with the sum
Nneb.append(np.sum(tmp_neb["Nassoc"]>i))
Nneb_per.append(f'({100*np.sum((tmp_neb["Nassoc"]>i))/len(tmp_neb):.1f}\%)')
Nasc.append(np.sum(associations["Nnebulae"]>i))
Nasc_per.append(f'({100*np.sum((associations["Nnebulae"]>i) & associations["in_frame"])/np.sum(associations["in_frame"]):.1f}\%)')
index.append(f'>{i+1}')

tbl = Table([index,Nneb,Nneb_per,Nasc,Nasc_per],names=[' ','1','Nebulae','2','Associations'])

print('WARNING: this table also includes non HII region in the overlap of the associations')
ascii.write(tbl,sys.stdout, Writer = ascii.Latex,latexdict=latexdict)

correct `Nnebulae` (we need to exclude non HII regions)

In [ ]:
Nnebulae = []
for gal_name in sample_table['name']:
    with open(basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'/f'{gal_name}_{HSTband}_{scalepc}pc_associations.yml') as f:
        associations_dict = yaml.load(f,Loader=yaml.SafeLoader)
    tmp_neb = nebulae[nebulae['gal_name']==gal_name]
    tmp_asc = associations[associations['gal_name']==gal_name]
    
    for assoc_ID in tmp_asc['assoc_ID']:
        Nnebulae.append(np.sum(np.isin(associations_dict[assoc_ID],tmp_neb['region_ID'])))

In [ ]:
latexdict = {'tabletype': 'table',
'header_start': '\\toprule',
'header_end': '\\midrule',
'data_end': '\\bottomrule',
'caption': f'Overlap statistics',
'preamble': '\\centering',
'col_align':'lrrrr',
'tablefoot': f'\\label{{tbl:overlap_statistics}}'
            }

# we only use nebulae in the 18 galaxies with HST associations
tmp_neb = nebulae[(~nebulae['Nassoc'].mask) & (nebulae['in_frame'])]

Nasc = []
Nneb = []
Nasc_per = []
Nneb_per = []
index = []
for i in range(5):
    index.append(i)
    
    Nneb.append(np.sum((tmp_neb["Nassoc"]==i)))
    Nneb_per.append(f'({100*np.sum((tmp_neb["Nassoc"]==i))/len(tmp_neb):.1f}\%)')
    
    Nasc.append(np.sum((np.array(Nnebulae)==i) & (associations['in_frame'])))
    Nasc_per.append(f'({100*np.sum((np.array(Nnebulae)==i) & associations["in_frame"])/np.sum(associations["in_frame"]):.1f}\%)')

    # add the last row with the sum
Nneb.append(np.sum(tmp_neb["Nassoc"]>i))
Nneb_per.append(f'({100*np.sum((tmp_neb["Nassoc"]>i))/len(tmp_neb):.1f}\%)')
Nasc.append(np.sum(np.array(Nnebulae)>i))
Nasc_per.append(f'({100*np.sum((np.array(Nnebulae)>i) & associations["in_frame"])/np.sum(associations["in_frame"]):.1f}\%)')
index.append(f'>{i+1}')

tbl = Table([index,Nneb,Nneb_per,Nasc,Nasc_per],names=[' ','1','Nebulae','2','Associations'])

ascii.write(tbl,sys.stdout, Writer = ascii.Latex,latexdict=latexdict)

In [ ]:
print(f"massive enough (>1e4 Msun): {np.sum(catalogue['mass']>1e4)}")
print(f"young enough (<8 Myr): {np.sum(catalogue['age']<=8)}")
print(f"contained: {np.sum(catalogue['overlap']=='contained')}")

In [ ]:
sample_table['res_arc'] = [0.72,0.73,0.74,0.63,0.82,0.49,0.65,0.8,0.64,0.72,0.85,0.74,0.77,0.58,0.58,0.64,0.44,0.73,0.79]
sample_table['res_pc'] = sample_table['distance']*sample_table['res_arc']*u.arcsec.to(u.radian)*1e6

the old sample table that contains additional properties

In [ ]:
latexdict = {'tabletype': 'table*',
'header_start': '\\toprule',
'header_end': '\\midrule',
'data_end': '\\bottomrule',
'caption': f'Galaxy sample',
'units': {'R.A.':'(J2000)','Dec.':'(J2000)','$i$':'deg','PA':'deg','Distance':'$\si{\mega\parsec}$',
          'r25':'arcmin'},
'preamble': '\\centering',
'tablefoot': f'\\label{{tbl:sample}}'
            }

tbl = sample_table[['name','Type','R.A.','Dec.','Inclination','posang','r25','mass','SFR','PSF','(m-M)']]

tbl['R.A.'] = [row.replace('h','x').replace('m','y').replace('.','z').replace('s','') for row in tbl['R.A.']]
tbl['R.A.'] = [row.replace('x','$^\mathrm{h}$').replace('y','$^\mathrm{m}$').replace('z','$^\mathrm{s}\kern -3pt.$') for row in tbl['R.A.']]

tbl['Dec.'] = [row.replace('d','x').replace('m','y').replace('.','z').replace('s','') for row in tbl['Dec.']]
tbl['Dec.'] = [row.replace('-','$-$').replace('+','$+$').replace('x','$^\mathrm{d}$').replace('y','$^\mathrm{m}$').replace('z','$^\mathrm{s}\kern -3pt.$') for row in tbl['Dec.']]
tbl.add_column(Distance(distmod=tbl['(m-M)']).to(u.Mpc),index=4,name='Distance')
tbl['Distance'].info.format = '%.2f' 
tbl['Nneb'] = [np.sum((nebulae['gal_name']==name) & (nebulae['in_frame'])) for name in tbl['name']]
tbl['Nasc'] = [np.sum((associations['gal_name']==name) & (associations['in_frame'])) for name in tbl['name']]

#tbl['name'] = [f'\\galaxyname{{{row["name"][:-4]}}}{{{row["name"][-4:]}}}' for row in tbl]
tbl.rename_columns(['Inclination','posang','r25'],['$i$','PA','$r_{25}$'])
#tbl = join(tbl,t,keys='name')

#with open(basedir / 'data' / 'interim' /'sample.tex','w',newline='\n') as f:
#    ascii.write(tbl,f,Writer=ascii.Latex, latexdict=latexdict,overwrite=True,exclude_names=['(m-M)','Sitelle'])
ascii.write(tbl,sys.stdout, Writer = ascii.Latex,latexdict=latexdict,exclude_names=[])

table to showcase the different versions of the association catalogue

In [ ]:
# show all available options and the number of nebulae/associatoins
version = 'v1p2'
bands = []
scale = []
Ngal  = []
Nneb  = []
Nneb_per = []
Nasc  = []
Nasc_per = []
N1to1 = []

for HSTband in ['nuv','v']:
    for scalepc in [16,32,64]:
        folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'
        lst = []
        for file in folder.glob(f'*{scalepc}pc_associations.fits'):
            gal_name = file.stem.split('_')[0]
            tbl = Table(fits.getdata(file,ext=1))
            tbl.add_column(gal_name,name='gal_name',index=0)
            lst.append(tbl)
        assoc_tmp = vstack(lst)
        assoc_in_frame = Table.read(basedir/'data'/'interim'/f'phangshst_associations_{HSTband}_ws{scalepc}pc_{version}_in_frame.fits')
        
        # missing = set(sample_table['name']) - set(np.unique(assoc_tmp['gal_name']))
        #print(f'{HSTband:>3}, {scalepc:>2}pc: {len(lst)} galaxies, {len(assoc_tmp):>5} associations ({np.sum(assoc_tmp["1to1"]):>4} 1to1)')
        
        folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'
        lst = []
        for file in folder.glob(f'*{scalepc}pc_nebulae.fits'):
            gal_name = file.stem.split('_')[0]
            tbl = Table(fits.getdata(file,ext=1))
            tbl.add_column(gal_name,name='gal_name',index=0)
            name=gal_name.lower()
            lst.append(tbl)
        nebulae_tmp = vstack(lst) 
        print(f'{HSTband:>3}, {scalepc:>2}pc: {len(lst)} galaxies, {np.sum(nebulae_tmp["overlap_neb"]>0)} HII regions overlap ({np.sum(nebulae_tmp["1to1"]):>4} 1to1)')

        bands.append(HSTband)
        scale.append(scalepc)
        Ngal.append(len(lst))
        Nneb.append(np.sum(nebulae_tmp["overlap_neb"]>0))
        Nneb_per.append(np.sum(nebulae_tmp["overlap_neb"]>0)/np.sum(nebulae['in_frame']))
        Nasc.append(np.sum(assoc_tmp['overlap_asc']>0))
        Nasc_per.append(np.sum(assoc_tmp['overlap_asc']>0)/np.sum(assoc_in_frame['in_frame']))
        N1to1.append(np.sum(assoc_tmp["1to1"]>0))

In [ ]:
Table([bands,scale,Nneb,Nasc,Nasc_per,N1to1],names=['Band','scale','Nneb','Nasc','asc_per','N1to1'])

maybe later, a table that shows the available data

In [ ]:
astrosat_sample = np.unique(nebulae[~np.isnan(nebulae['FUV_FLUX'])]['gal_name'])
muse_sample     = sample_table['name']
hst_sample      = associations['gal_name']
sitelle_sample  = ['NGC0628','NGC2835','NGC3351','NGC3627','NGC4535']

t = Table({
    'name':muse_sample,
    'MUSE':np.isin(muse_sample,muse_sample),
    'HST':np.isin(muse_sample,hst_sample),
    'Astrosat':np.isin(muse_sample,astrosat_sample),
    'Sitelle':np.isin(muse_sample,sitelle_sample)}
     )

for col in t.columns[1:]:
    t[col] = ['\checkmark' if x else '' for x in t[col] ]
ascii.write(t,sys.stdout, Writer = ascii.Latex)

### catalogue for the paper

In [ ]:
from datetime import date

columns = ['gal_name','region_ID','assoc_ID',
           'ra_neb','dec_neb','ra_asc','dec_asc',
           'age','age_err','mass','mass_err',
           'EBV_stars','EBV_stars_err','overlap_asc','overlap',
           'logq_D91','logq_D91_err','Delta_met_scal','density','density_err',
           'HA6562_LUM_CORR','HA6562_LUM_CORR_ERR','HA/FUV_corr','HA/FUV_corr_err',
           'EW_HA','EW_HA_ERR','continuum_Halpha','continuum_Halpha_error',
           'continuum_Halpha_bkg','continuum_Halpha_bkg_error',
           'EBV_balmer','EBV_balmer_err','overlap_neb','neighbors','GMC_sep']


doc = f'''matched catalogue for Scheuermann et al. (submitted)
S/N cuts are not applied yet.
The code for this project can be found here.
https://github.com/fschmnn/cluster/
last update: {date.today().strftime("%b %d, %Y")}
'''

primary_hdu = fits.PrimaryHDU()
for i,comment in enumerate(doc.split('\n')):
    if i==0:
        primary_hdu.header['COMMENT'] = comment
    else:
        primary_hdu.header[''] = comment
table_hdu   = fits.BinTableHDU(catalogue[columns])
hdul = fits.HDUList([primary_hdu, table_hdu])
hdul.writeto('matched_catalogue.fits',overwrite=True)

### Plot the sample (cutouts)

plot all objects in the merged catalogues

In [ ]:
from cluster.io import read_associations
from matplotlib.backends.backend_pdf import PdfPages
from cluster.plot import single_cutout


criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=10) 
criteria &= (catalogue['overlap_neb']>0.1) 
criteria &= (catalogue['overlap_asc']==1) 
criteria &= (catalogue['neighbors']==0)

tmp = catalogue[criteria]

print(f'{len(tmp)} objects in sample')

size=10*u.arcsec
nrows=5
ncols=4
filename = basedir/'reports'/f'cutouts_{HSTband}_{scalepc}pc'
    
width = 8.27
N = len(tmp) 
Npage = nrows*ncols-1
if N%Npage==0:
    print('sample size % subplots = 0: no subplot for legend')
Npages = int(np.ceil(N/Npage))
gal_name = None

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for i in range(Npages):
        print(f'working on page {i+1} of {Npages}')

        sub_sample = tmp[i*Npage:(i+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            
            # for a new galaxy we need to read in the masks/images
            if row['gal_name'] != gal_name:
                
                gal_name = row['gal_name']
                
                # HST image for the background
                filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
                with fits.open(filename) as hdul:
                    F275 = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                
                # nebulae mask
                filename = data_ext / 'Products' / 'Nebulae_catalogs' / 'Nebulae_catalogue_v2' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
                with fits.open(filename) as hdul:
                    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
                    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
                
                # association mask
                associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',
                                                      target=gal_name.lower(),
                                                      scalepc=scalepc,
                                                      data='mask')

            
            ax = next(axes_iter)
            ax = single_cutout(ax,
                             position = row['SkyCoord_neb'],
                             image = F275,
                             mask1 = nebulae_mask,
                             mask2 = associations_mask,
                             label = f"{row['gal_name']}: {row['region_ID']:.0f}/{row['assoc_ID']:.0f}",
                             size  = 4*u.arcsecond)

        plt.subplots_adjust(wspace=-0.01, hspace=0.05)

        # only the last page has subplots that need to be removed
        h,l = fig.axes[0].get_legend_handles_labels()
        ax = next(axes_iter)
        ax.axis('off')
        ax.legend(h[::len(h)-1],l[::(len(l)-1)],fontsize=7,loc='center',frameon=False)
        t = ax.text(0.07,0.87,'name: region ID/assoc ID', transform=ax.transAxes,color='black',fontsize=8)

        if i == int(np.ceil(N/Npage))-1:

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()

three color composit with CO emission

In [ ]:
from cluster.io import read_associations
from matplotlib.backends.backend_pdf import PdfPages
from reproject import reproject_interp
from skimage.measure import find_contours
from pnlf.plot import create_RGB


criteria = (catalogue['mass']>1e4) 
criteria &= (catalogue['overlap_asc'] == 1)
criteria &= (catalogue['age'] < 10)
#tmp = catalogue[criteria]

print(f'{len(tmp)} objects')

size=5*u.arcsec
nrows=5
ncols=4
filename = basedir/'reports'/f'cutouts_rgb_{HSTband}_{scalepc}pc'

width = 8.27
N = len(tmp) 
Npage = nrows*ncols-1
if N%Npage==0:
    print('sample size % subplots = 0: no subplot for legend')
Npages = int(np.ceil(N/Npage))
gal_name = None

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for j in range(Npages):
        print(f'working on page {j+1} of {Npages}')

        sub_sample = tmp[j*Npage:(j+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            
            # for a new galaxy we need to read in the masks/images
            if row['gal_name'] != gal_name:
                
                gal_name = row['gal_name']
                print(f'reading files for {gal_name}')
                
                # HST image for the background
                filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
                with fits.open(filename) as hdul:
                    F275 = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                
                filename = data_ext/'MUSE'/'DR2.1'/'MUSEDAP'/ f'{gal_name}_MAPS.fits'
                with fits.open(filename) as hdul:
                    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                                    meta=hdul['HA6562_FLUX'].header,
                                    wcs=WCS(hdul['HA6562_FLUX'].header))
    
                # nebulae mask
                filename = data_ext/'Products'/ 'Nebulae catalogue'/'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
                with fits.open(filename) as hdul:
                    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
                    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
                
                # association mask
                associations_mask = read_associations(folder=data_ext/'HST'/'stellar_associations',
                                                      target=gal_name.lower(),
                                                      scalepc=scalepc,
                                                      data='mask')
            
                with fits.open(data_ext/'ALMA'/'v4p0'/f'{gal_name.lower()}_12m+7m+tp_co21_broad_tpeak.fits') as hdul:
                    CO = NDData(data=hdul[0].data,
                                meta=hdul[0].header,
                                wcs=WCS(hdul[0].header))
            
            ax = next(axes_iter)
            
            position = row['SkyCoord_neb']
            
            label = f"{row['gal_name']}: {row['region_ID']:.0f}/{row['assoc_ID']:.0f}"

            cutout_F275 = Cutout2D(F275.data,position,size=size,wcs=F275.wcs)
            norm = simple_norm(cutout_F275.data,stretch='linear',clip=False,percent=99.9)

            cutout_CO, _  = reproject_interp(CO,output_projection=cutout_F275.wcs,shape_out=cutout_F275.shape)    
            cutout_Halpha, _  = reproject_interp(Halpha,output_projection=cutout_F275.wcs,shape_out=cutout_F275.shape)    

            rgb = create_RGB(cutout_CO,cutout_Halpha,cutout_F275.data,
                             percentile=[98,98,99.8],weights=[0.7,0.6,1])
            #ax.imshow(cutout_image.data,origin='lower',norm=norm,cmap=plt.cm.gray_r)
            ax.imshow(rgb,origin='lower')

            # plot the nebulae catalogue
            cutout_mask, _  = reproject_interp(nebulae_mask,output_projection=cutout_F275.wcs,shape_out=cutout_F275.shape,order='nearest-neighbor')    
            region_ID = np.unique(cutout_mask[~np.isnan(cutout_mask)])

            contours = []
            for i in region_ID:
                blank_mask = np.zeros_like(cutout_mask)
                blank_mask[cutout_mask==i] = 1
                contours += find_contours(blank_mask, 0.5)
            for coords in contours:
                ax.plot(coords[:,1],coords[:,0],color='tab:green',lw=0.8,label='HII-region')

            # 32 pc
            cutout_32 = Cutout2D(associations_mask.data,position,size=size,wcs=associations_mask.wcs)
            region_ID = np.unique(cutout_32.data[~np.isnan(cutout_32.data)])
            contours = []
            for i in region_ID:
                blank_mask = np.zeros_like(cutout_32.data)
                blank_mask[cutout_32.data==i] = 1
                contours += find_contours(blank_mask, 0.5)
            for coords in contours:
                ax.plot(coords[:,1],coords[:,0],color='blue',lw=0.8,label='32pc assoc.')
            t = ax.text(0.06,0.87,label, transform=ax.transAxes,color='black',fontsize=8)
            t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
            ax.set_xticks([])
            ax.set_yticks([])
            
        plt.subplots_adjust(wspace=-0.01, hspace=0.05)

        # only the last page has subplots that need to be removed
        h,l = fig.axes[0].get_legend_handles_labels()
        ax = next(axes_iter)
        ax.axis('off')
        ax.legend(h[::len(h)-1],l[::(len(l)-1)],fontsize=7,loc='center',frameon=False)
        t = ax.text(0.07,0.87,'name: region ID/assoc ID', transform=ax.transAxes,color='black',fontsize=8)

        if j == int(np.ceil(N/Npage))-1:

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()

## Search for correlations

### Ha/FUV vs ionization

with log q from Diaz+91

$$
\log u = (-1.684±0.076)\cdot \log([SII]/[SIII])-(2.986 ±0.027)
$$

In [ ]:
from scipy.stats import binned_statistic, spearmanr
from astrotools.plot.corner import corner_binned_percentile
from astrotools.plot.utils import bin_stat
from astrotools.metallicity.ionization import logq_D91, logq_D91_reverse

bins = np.linspace(*np.nanpercentile(catalogue['logq_D91'],[1,99]),10)
xlim = [5.7,7.8]

q1 = 68
q2 = 98

x_name,y1_name,y2_name,y3_name = 'logq_D91','EW_HA','HA/FUV_corr','Delta_met_scal'

fig, (ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/3))

table = nebulae.copy()
table = table[table['logq_D91']>3*table['logq_D91_err']]
#table = table[table['HA/FUV_corr']>3*table['HA/FUV_corr_err']]

print(f'{len(table)} objects in sample')

cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)
sample_table.sort('mass')

rho1,rho2,rho3 = [], [], []
for i,gal_name in enumerate(sample_table['name']):
    #print(gal_name)
    tmp = table[table['gal_name']==gal_name]
    
    if np.any(np.isfinite(tmp['EW_HA'])):
        t = tmp[tmp['EW_HA']>3*tmp['EW_HA_ERR']]
        x,mean,std = bin_stat(t['logq_D91'],t['EW_HA'],bins=bins,statistic='median')
        ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
        rho1.append(spearmanr(tmp['logq_D91'],tmp['EW_HA'],nan_policy='omit')[0])

    if np.any(np.isfinite(tmp['HA/FUV_corr'])):
        t = tmp[(tmp['HA/FUV_corr']>3*tmp['HA/FUV_corr_err'])]
        x,mean,std = bin_stat(t['logq_D91'],t[y2_name],bins=bins,statistic='median')
        ax2.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
        try:
            rho2.append(spearmanr(t['logq_D91'],t['HA/FUV_corr'],nan_policy='omit')[0])
        except:
            pass
    
    if np.any(np.isfinite(tmp['Delta_met_scal'])):
        x,mean,std = bin_stat(tmp['logq_D91'],tmp['Delta_met_scal'],bins=bins,statistic='median')
        ax3.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
        rho3.append(spearmanr(tmp['logq_D91'],tmp['Delta_met_scal'],nan_policy='omit')[0])

# plot contours
for ax,y_name,y_name_err in zip([ax1,ax2,ax3],
                     ['EW_HA','HA/FUV_corr','Delta_met_scal'],
                     ['EW_HA_ERR','HA/FUV_corr_err',None]):
    
    x,y = table[x_name],table[y_name]
    if y_name_err:
        x,y = x[table[y_name]>3*table[y_name_err]], y[table[y_name]>3*table[y_name_err]]
    #ax1.scatter(x,y,s=0.5,color='black')
    print(f'{y_name}: {spearmanr(x,y,nan_policy="omit")[0]:.2f}')

    # just ignore nan values
    x = x[~np.isnan(y) & np.isfinite(y)]
    y = y[~np.isnan(y) & np.isfinite(y)]

    corner_binned_percentile(x,y,ax,nbins=bins,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=bins,n=98,color='gray',alpha=0.3)
    

    x,mean,std = bin_stat(table[x_name],table[y_name],bins=bins,statistic='median')
    ax.errorbar(x,mean,fmt='o-',ms=2.5,color='black')

    
sc = ax1.scatter(19*[1],19*[1],c=sample_table['mass'],cmap=cmap,vmin=9.4,vmax=11)

ax1.set(xlim=xlim, yscale='log',ylim=[3,5e2],
        xlabel=r'$\log q$',ylabel=r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$')
ax2.set(xlim=xlim, yscale='log',ylim=[1,1e2],
        xlabel=r'$\log q$',ylabel=r'H$\alpha$/FUV')
ax3.set(xlim=xlim,ylim=[-0.15,0.15],
        xlabel=r'$\log q$',ylabel=r'$\Delta$(O/H)')

ax1.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax1.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   
ax2.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax2.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   
ax3.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax3.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1))
ax3.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.01))

# get a second x axis on top with [SIII]/[SII]
xmin,xmax=ax1.get_xlim()
ax1_top = ax1.twiny()
ax1_top.set(xlim=[logq_D91_reverse(xmin),logq_D91_reverse(xmax)],xscale='log')
ax1_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
ax1_top.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

ax2_top = ax3.twiny()
ax2_top.set(xlim=[logq_D91_reverse(xmin),logq_D91_reverse(xmax)],xscale='log')
ax2_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
ax2_top.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

ax3_top = ax2.twiny()
ax3_top.set(xlim=[logq_D91_reverse(xmin),logq_D91_reverse(xmax)],xscale='log')
ax3_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
ax3_top.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

for i, rho in enumerate([rho1,rho2,rho3]):
    print(f'plot{i}: rho={np.nanmean(rho):.2f}')

plt.tight_layout()
fig.subplots_adjust(right=0.85,wspace=0.4)
cbar_ax = fig.add_axes([0.87, 0.21, 0.02, 0.6])
fig.colorbar(sc,cax=cbar_ax,label=r'$\log (m/\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.4))

plt.savefig(basedir/'reports'/'nebulae_correlations.pdf',dpi=600)
#plt.savefig(basedir/'reports'/'nebulae_correlations.png',dpi=600)

plt.show()

histogram/binned stat with log bins

In [ ]:
from scipy.stats import binned_statistic
from astrotools.plot.corner import corner_binned_percentile
from astrotools.plot.utils import bin_stat

q1 = 68
q2 = 98

x_name,y1_name,y2_name,y3_name = '[SIII]/[SII]', 'Delta_met_scal','HA/FUV_corr','eq_width'
xlim,ylim1,ylim2,ylim3 = [1e-2,1],[-0.15,0.15],[8e-1,8e1],[3,3e2]

fig, (ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/3.2))

table = nebulae.copy()
table = table[(table['HA/FUV_corr']>3*table['HA/FUV_corr_err']) | np.isnan(table['FUV_FLUX'])]
table = table[table['[SIII]/[SII]']>3*table['[SIII]/[SII]_err']]

cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)
sample_table.sort('mass')

# the density histogram

nbins=20

x,y = table[x_name],table[y1_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.linspace(*ylim1,nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax1.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)

x,y = table[x_name],table[y2_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim2),nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax2.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)

x,y = table[x_name],table[y3_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim3),nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax3.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)


# the bins for the binned mean
#bins = np.logspace(-1.9,-0.2,10)
bins = np.logspace(-2.4,-0.4,12)

for i,gal_name in enumerate(sample_table['name']):
    #print(gal_name)
    
    tmp = table[table['gal_name']==gal_name]

    x,mean,std = bin_stat(tmp[x_name],tmp[y1_name],bins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)

    x,mean,std = bin_stat(tmp[x_name],tmp[y2_name],bins=bins,statistic='median')
    ax2.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)

    x,mean,std = bin_stat(tmp[x_name],tmp[y3_name],bins=bins,statistic='median')
    ax3.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)

# plot contours

for ax,y_name in zip([ax1,ax2,ax3],[y1_name,y2_name,y3_name]):
    
    x,y = table[x_name],table[y_name]
    #ax1.scatter(x,y,s=0.5,color='black')

    # just ignore nan values
    x = x[~np.isnan(y) & np.isfinite(y)]
    y = y[~np.isnan(y) & np.isfinite(y)]
    
    x,mean,std = bin_stat(table[x_name],table[y_name],bins=bins,statistic='median')
    ax.errorbar(x,mean,fmt='o-',ms=2.5,color='black')

    
sc = ax1.scatter(19*[1],19*[1],c=sample_table['mass'],cmap=cmap,vmin=9.4,vmax=11)

ax1.set(xscale='log',xlim=xlim,ylim=ylim1,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\Delta$(O/H)')
ax2.set(xscale='log',xlim=xlim, yscale='log',ylim=ylim2,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'H$\alpha$/FUV')

ax3.set(xscale='log',xlim=xlim, yscale='log',ylim=ylim3,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$')


ax1.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax1.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1))
ax1.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.01))
ax2.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax2.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   
ax3.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax3.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   


plt.tight_layout()
fig.subplots_adjust(right=0.85,wspace=0.4)
cbar_ax = fig.add_axes([0.87, 0.21, 0.02, 0.73])
fig.colorbar(sc,cax=cbar_ax,label=r'$\log (m/\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.4))

#plt.savefig(basedir/'reports'/'nebulae_correlations.pdf',dpi=600)

plt.show()

the same plot but now binned by age

In [ ]:
from scipy.stats import binned_statistic
from astrotools.plot.corner import corner_binned_percentile
from astrotools.plot.utils import bin_stat

q1 = 68
q2 = 98

x_name,y1_name,y2_name,y3_name = '[SIII]/[SII]', 'Delta_met_scal','HA/FUV','eq_width'
xlim,ylim1,ylim2,ylim3 = [1e-2,1],[-0.15,0.15],[8e-1,8e1],[3,3e2]

fig, (ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/3.2))

table = catalogue.copy()
#table = table[(table['FUV_FLUX_CORR']>3*table['FUV_FLUX_CORR_ERR']) | np.isnan(table['FUV_FLUX_CORR'])]
table = table[table['[SIII]/[SII]']>3*table['[SIII]/[SII]_err']]

cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=0,vmax=20)


nbins=20

x,y = table[x_name],table[y1_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.linspace(*ylim1,nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax1.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)

x,y = table[x_name],table[y2_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim2),nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax2.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)

x,y = table[x_name],table[y3_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim3),nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax3.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)


# the bins for the binned mean
#bins = np.logspace(-1.9,-0.2,10)
bins = np.logspace(-2.4,-0.4,12)


tmp = catalogue #[catalogue['mass']>1e4]

for age in [(0,2),(2,5),(5,10),(10,20)]:
    age_mean = np.mean(age)
    tmp = table[(table['age']>age[0]) & (table['age']<age[1])]
    print(f'{age_mean}: {len(tmp)} objects')
    x,mean,std = bin_stat(tmp[x_name],tmp[y1_name],[None,None],nbins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_mean)),label=age_mean)

    x,mean,std = bin_stat(tmp[x_name],tmp[y2_name],[None,None],nbins=bins,statistic='median')
    ax2.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_mean)),label=age_mean)

    x,mean,std = bin_stat(tmp[x_name],tmp[y3_name],[None,None],nbins=bins,statistic='median')
    ax3.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_mean)),label=age_mean)

# plot contours

for ax,y_name in zip([ax1,ax2,ax3],[y1_name,y2_name,y3_name]):
    
    x,y = table[x_name],table[y_name]
    #ax1.scatter(x,y,s=0.5,color='black')

    # just ignore nan values
    x = x[~np.isnan(y) & np.isfinite(y)]
    y = y[~np.isnan(y) & np.isfinite(y)]
    
    x,mean,std = bin_stat(table[x_name],table[y_name],[None,None],nbins=bins,statistic='median')
    #ax.errorbar(x,mean,fmt='o-',ms=2.5,color='black')

    
sc = ax1.scatter(19*[1],19*[1],c=sample_table['mass'],cmap=cmap,vmin=0,vmax=12)

ax1.set(xscale='log',xlim=xlim,ylim=ylim1,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\Delta$(O/H)')
ax2.set(xscale='log',xlim=xlim, yscale='log',ylim=ylim2,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'H$\alpha$/FUV')

ax3.set(xscale='log',xlim=xlim, yscale='log',ylim=ylim3,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$')


ax1.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax1.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1))
ax1.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.01))
ax2.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax2.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   
ax3.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax3.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   


plt.tight_layout()
fig.subplots_adjust(right=0.85,wspace=0.4)
cbar_ax = fig.add_axes([0.87, 0.21, 0.02, 0.73])
fig.colorbar(sc,cax=cbar_ax,label=r'age / Myr',ticks=np.arange(0,20,4))

#plt.savefig(basedir/'reports'/'nebulae_correlations_age_bin.pdf',dpi=600)

plt.show()

a dedicated subplot for each galaxie

In [ ]:
from scipy.stats import binned_statistic, pearsonr, spearmanr

sample = set(astrosat_sample) & set(muse_sample)
filename = basedir/'reports'/'all_objects_HaFUV_over_SII'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 4
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())
    
#vmin,vmax = np.min(catalogue['HA6562_FLUX_CORR']),np.max(catalogue['HA6562_FLUX_CORR'])
vmin,vmax = 1e-16,1e-14
# loop over the galaxies we want to plot
for name in sorted(sample):  
    
    tmp = nebulae[(nebulae['gal_name']==name)]
        
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    tmp = tmp[(tmp['HA/FUV_corr']>3*tmp['HA/FUV_corr_err']) | np.isnan(tmp['FUV_FLUX'])]
    tmp = tmp[tmp['[SIII]/[SII]']>3*tmp['[SIII]/[SII]_err']]

    r,p = spearmanr(tmp['[SIII]/[SII]'],tmp['HA/FUV_corr'])
    print(f'{name}: rho={r:.2f}, {len(tmp)} objects')

    sc = ax.scatter(tmp['[SIII]/[SII]'],tmp['HA/FUV_corr'],
               c=1e-20*tmp['HA6562_FLUX_CORR'],vmin=vmin,vmax=vmax,
               cmap=plt.cm.plasma,
               norm=mpl.colors.LogNorm(),
               s=1,marker='.')
    
    #q = 98
    #bins = np.logspace(*np.log10(np.percentile(tmp['[SIII]/[SII]'],[100-q,q])),10)
    #x,mean,std = bin_stat(tmp['[SIII]/[SII]'],tmp['HA/FUV'],[None,None],nbins=bins)
    #ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label=gal_name)
    
    ax.text(0.05,0.9,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.75,0.15,r'$\rho$'+f'={r:.2f}',transform=ax.transAxes,fontsize=7)
    ax.text(0.62,0.05,f'{len(tmp):.0f} objects', transform=ax.transAxes,fontsize=7)
    
    ax.set(xscale='log',yscale='log',xlim=[1e-2,1],ylim=[2,4e2])
    # https://stackoverflow.com/questions/21920233/matplotlib-log-scale-tick-label-number-formatting/33213196
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    if i==nrows-1:
        ax.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
    if j==0:
        ax.set_ylabel(r'H$\alpha$ / FUV')

        
for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    
    if i==0:
        #ax.remove()
        ax.axis('off')
        cbar = fig.colorbar(sc, ax=ax,
                            label=r'$\mathrm{H}\alpha$ / (erg s$^{-1}$ cm$^{-2}$ Hz$^{-1}$)',
                            orientation='horizontal',
                           )
    else:
        ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')


#plt.savefig(filename.with_suffix('.png'),dpi=600)
#plt.savefig(filename.with_suffix('.pdf'),dpi=600)

plt.show()


In [ ]:
tmp = nebulae[(nebulae['gal_name']=='NGC4254')]
#tmp = tmp[(tmp['HA/FUV_corr']>3*tmp['HA/FUV_corr_err']) | np.isnan(tmp['FUV_FLUX'])]
tmp = tmp[tmp['[SIII]/[SII]']>3*tmp['[SIII]/[SII]_err']]

tmp[['HA/FUV_corr','HA/FUV_corr_err']]

### age histograms

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0.5,10.5,1)

tmp = associations[(associations['mass']>1e4)] #& (associations['age']<=10)]
print(len(tmp))
ages_con = tmp[tmp['overlap']=='contained']['age']
ages_par = tmp[tmp['overlap']=='partial']['age']
ages_iso = tmp[(tmp['overlap']=='isolated') & (tmp['in_frame'])]['age']

print(f'ages: con={np.median(ages_con):.2f}, par={np.median(ages_par):.2f}, iso={np.median(ages_iso):.2f}')

for overlap,ax in zip(['contained','partial','isolated'],[ax1,ax2,ax3]):
    sample = tmp[tmp['overlap']==overlap]
    n = np.sum(sample['age']<=2)
    #  r'$2 \leq \mathrm{age / Myr}$: '+
    text =f'young: {n:>5} ({100*n/len(sample):.0f}\%)\n'
    n = np.sum((sample['age']>2) & (sample['age']<=10))
    # r'$2 < \mathrm{age / Myr} \leq 10$: '+
    text += f'intermediate: {n:>5} ({100*n/len(sample):.0f}\%)\n'
    n = np.sum(sample['age']>10)
    #r'$\mathrm{age / Myr} < 10$: '+
    text += f'old: {n:>5} ({100*n/len(sample):.0f}\%)'
    ax.text(0.95,0.8,text, transform=ax.transAxes,
             color='black',fontsize=8,horizontalalignment='right')
    
n1,_,_=ax1.hist(ages_con,bins=bins,histtype='step',label='contained')
n2,_,_=ax2.hist(ages_par,bins=bins,histtype='step',label='partially')
n3,_,_=ax3.hist(ages_iso,bins=bins,histtype='step',label='isolated')

percentiels = np.nanpercentile(ages_con,[15.85,50,84.15])
minus,plus  = np.ediff1d(percentiels)
ax1.set_title(f'contained (${percentiels[1]:.1f}^{{+{plus:.1f}}}_{{-{minus:.1f}}}$ Myr)')
percentiels = np.nanpercentile(ages_par,[15.85,50,84.15])
minus,plus  = np.ediff1d(percentiels)
ax2.set_title(f'partial (${percentiels[1]:.1f}^{{+{plus:.1f}}}_{{-{minus:.1f}}}$ Myr)')
percentiels = np.nanpercentile(ages_iso,[15.85,50,84.15])
minus,plus  = np.ediff1d(percentiels)
ax3.set_title(f'isolated (${percentiels[1]:.1f}^{{+{plus:.1f}}}_{{-{minus:.1f}}}$ Myr)')

ymax = np.max([n1,n2,n3])
ymax = 1.1*np.round(ymax,-int(np.log10(ymax))+1)
ymax = 1150
for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,ymax],xlim=[0,10],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'age_hist_contained.pdf',dpi=600)
plt.show()

In [ ]:
N = 10000
np.std(np.median(np.random.choice(ages_con,size=(N,len(ages_con)),replace=True),axis=1))

In [ ]:
from astropy.coordinates import match_coordinates_sky

tmp = associations[(associations['mass']>1e4)]
idx,sep,_=match_coordinates_sky(tmp['SkyCoord_asc'],nebulae['SkyCoord_neb'])

ages1 = tmp[(sep<0.4*u.arcsec)]['age']
ages2 = tmp[(sep>0.4*u.arcsec) & (sep<0.8*u.arcsec)]['age']
ages3 = tmp[(sep>0.8*u.arcsec)]['age']

print(f'mean age: 1={np.mean(ages1):.2f}, 2={np.mean(ages2):.2f}, 3={np.mean(ages3):.2f} Myr')

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0,10,1)

ax1.hist(ages1,bins=bins,histtype='step',label='isolated')
ax2.hist(ages2,bins=bins,histtype='step',label='partially')
ax3.hist(ages3,bins=bins,histtype='step',label='contained')
ax1.set_title(r'$s<0.4"$'+f' ({np.mean(ages1):.2f} Myr)')
ax2.set_title(r'$0.4"<s<0.8"$' +f' ({np.mean(ages2):.2f} Myr)')
ax3.set_title(r'$0.8"<s$'+f' ({np.mean(ages3):.2f} Myr)')

for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,1100],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'age_hist_sep.png',dpi=600)

plt.show()

In [ ]:
tmp = catalogue[(catalogue['mass']>=1e4) ]  # & (catalogue['age']<10)]
print(len(tmp))

cont = tmp['continuum_Halpha']-tmp['continuum_Halpha_bkg']
cont_err = np.sqrt(tmp['continuum_Halpha_error']**2+tmp['continuum_Halpha_bkg_error']**2)
tmp['EW_HA'][cont/cont_err<5] = np.nan
tmp['EW_HA_CORR'][cont/cont_err<5] = np.nan

p1,p2=np.nanpercentile(tmp['EW_HA'],[33,66])

ages1 = tmp[tmp['EW_HA']<p1]["age"]
ages2 = tmp[(tmp['EW_HA']>p1) & (tmp['EW_HA']<p2)]["age"]
ages3 = tmp[(tmp['EW_HA']>p2)]["age"]

print(f'median age: 1={np.median(ages1):.2f}, 2={np.median(ages2):.2f}, 3={np.median(ages3):.2f} Myr')

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0.5,10.5,1)

ax1.hist(ages1,bins=bins,histtype='step',label='isolated')
ax2.hist(ages2,bins=bins,histtype='step',label='partially')
ax3.hist(ages3,bins=bins,histtype='step',label='contained')

percentiels = np.nanpercentile(ages1,[15.85,50,84.15])
minus,plus  = np.ediff1d(percentiels)
ax1.set_title(r'1$^\mathrm{st}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$'+f' (${percentiels[1]:.1f}^{{+{plus:.1f}}}_{{-{minus:.1f}}}$ Myr)')
percentiels = np.nanpercentile(ages2,[15.85,50,84.15])
minus,plus  = np.ediff1d(percentiels)
ax2.set_title(r'2$^\mathrm{st}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$'+f' (${percentiels[1]:.1f}^{{+{plus:.1f}}}_{{-{minus:.1f}}}$ Myr)')
percentiels = np.nanpercentile(ages3,[15.85,50,84.15])
minus,plus  = np.ediff1d(percentiels)
ax3.set_title(r'3$^\mathrm{st}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$'+f' (${percentiels[1]:.1f}^{{+{plus:.1f}}}_{{-{minus:.1f}}}$ Myr)')


for sample,ax in zip([ages1,ages2,ages3],[ax1,ax2,ax3]):
    
    n = np.sum(sample<=2)
    #  r'$2 \leq \mathrm{age / Myr}$: '+
    text =f'young: {n:>5} ({100*n/len(sample):.0f}\%)\n'
    n = np.sum((sample>2) & (sample<=10))
    # r'$2 < \mathrm{age / Myr} \leq 10$: '+
    text += f'intermediate: {n:>5} ({100*n/len(sample):.0f}\%)\n'
    n = np.sum(sample>10)
    #r'$\mathrm{age / Myr} < 10$: '+
    text += f'old: {n:>5} ({100*n/len(sample):.0f}\%)'
    ax.text(0.95,0.8,text, transform=ax.transAxes,
             color='black',fontsize=8,horizontalalignment='right')


for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,180],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'age_hist_eq_width.pdf',dpi=600)

plt.show()

In [ ]:
np.sum(~np.isnan(tmp['EW_HA']))

In [ ]:
tmp = catalogue[(catalogue['mass']>1e4) ]  # & (catalogue['age']<10)]
print(len(tmp))

p1,p2=np.nanpercentile(tmp['HA/FUV_corr'],[33,66])

ages1 = tmp[tmp['HA/FUV_corr']<p1]
ages2 = tmp[(tmp['HA/FUV_corr']>p1) & (tmp['HA/FUV_corr']<p2)]
ages3 = tmp[(tmp['HA/FUV_corr']>p2)]

print(f'median age: 1={np.median(ages1["age"]):.2f}, 2={np.median(ages2["age"]):.2f}, 3={np.median(ages3["age"]):.2f} Myr')

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0.5,10.5,1)

ax1.hist(ages1["age"],bins=bins,histtype='step',label='isolated')
ax2.hist(ages2["age"],bins=bins,histtype='step',label='partially')
ax3.hist(ages3["age"],bins=bins,histtype='step',label='contained')
ax1.set_title(r'1$^\mathrm{st}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$'+f' ({np.median(ages1["age"]):.2f} Myr)')
ax2.set_title(r'2$^\mathrm{nd}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$' +f' ({np.median(ages2["age"]):.2f} Myr)')
ax3.set_title(r'3$^\mathrm{rd}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$'+f' ({np.median(ages3["age"]):.2f} Myr)')

for sample,ax in zip([ages1,ages2,ages3],[ax1,ax2,ax3]):
    
    n = np.sum(sample['age']<=2)
    #  r'$2 \leq \mathrm{age / Myr}$: '+
    text =f'young: {n:>5} ({100*n/len(sample):.0f}\%)\n'
    n = np.sum((sample['age']>2) & (sample['age']<=10))
    # r'$2 < \mathrm{age / Myr} \leq 10$: '+
    text += f'intermediate: {n:>5} ({100*n/len(sample):.0f}\%)\n'
    n = np.sum(sample['age']>10)
    #r'$\mathrm{age / Myr} < 10$: '+
    text += f'old: {n:>5} ({100*n/len(sample):.0f}\%)'
    ax.text(0.95,0.8,text, transform=ax.transAxes,
             color='black',fontsize=8,horizontalalignment='right')


for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,180],xlabel='age / Myr')
#plt.savefig(basedir/'reports'/f'age_hist_HaFUV.png',dpi=600)

plt.show()

### Mass-to-Halpha

use age bins instead of galaxy mass

In [ ]:
from astrotools.plot.utils import bin_stat
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import spearmanr, binned_statistic
from astrotools.plot.corner import corner_binned_percentile

fig,(ax1,ax2)=plt.subplots(nrows=2,figsize=(single_column,single_column*(1+1/1.618+0.1)),
                          gridspec_kw={'height_ratios': [1, 1.618]})

# ----------------------- ax1 ------------------------------------------
criteria  = (catalogue['mass']>1e2) 
#criteria &= (catalogue['age']<10)
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()
#tmp = catalogue.copy()

q1 = 68
q2 = 98

xlim =[2e2,1e5]
ylim = [1e4,5e7]
ylim = [1e36,8e39]
vmin,vmax=0,10
nbins = 8
bins = np.logspace(2,5.4,nbins)

cmap = plt.cm.get_cmap('viridis',5)
norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)

age_bins = [0,2,4,6,8,10]
for i in range(len(age_bins)-1):

    sub = tmp[(tmp['age']>=age_bins[i]) & (tmp['age']<age_bins[i+1])]
    x,y = sub['mass'],sub['HA6562_LUM_CORR']
    x,mean,std = bin_stat(x,y,bins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_bins[i]+1)))

rho = spearmanr(tmp['mass'],tmp['HA6562_LUM_CORR'],nan_policy='omit')[0]
label = r'$\rho'+f'={np.nanmean(rho):.2f}$'
t = ax1.text(0.05,0.93,label,transform=ax1.transAxes,ha='left',va='top')
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

x,y = tmp['mass'],tmp['HA6562_LUM_CORR']

x,y = x[~np.isnan(y) & np.isfinite(y)],y[~np.isnan(y) & np.isfinite(y)]
corner_binned_percentile(x,y,ax1,nbins=bins,n=68,color='gray',alpha=0.6)
corner_binned_percentile(x,y,ax1,nbins=bins,n=98,color='gray',alpha=0.3)
#x,mean,std = bin_stat(x,y,[None,None],nbins=bins,statistic='median')
#ax.errorbar(x,mean,fmt='o-',ms=2.5,color='black')    

ax1.set(xscale='log',yscale='log',xlim=xlim,ylim=ylim,
        xlabel=r'mass / M$_\odot$',ylabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$')

# we need a scatter plot instance for the color bar
sc = ax1.scatter(19*[1],19*[1],c=sample_table['distance'],cmap=cmap,vmin=0,vmax=12)

# ----------------------- ax2 ------------------------------------------
criteria  = (catalogue['mass']>0) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()
#tmp = catalogue.copy()
print(f'ax1: {len(tmp)} objects')

xlim = [0,10]
sc=ax2.scatter(tmp['EBV_stars'],tmp['EBV_balmer'],c=tmp['age'],s=3,vmin=vmin,vmax=vmax,cmap=cmap,rasterized=True)
EBV_balmer_err = np.mean(tmp['EBV_balmer_err'])
EBV_stars_err = np.mean(tmp['EBV_stars_err'])
ax2.errorbar(0.58,0.08,xerr=EBV_stars_err,yerr=EBV_balmer_err,fmt='ko',ms=0)

ax2.plot([0,1],[0,2],color='black')
ax2.plot([0,2],[0,2],color='black')
ax2.set(xlim=[0,0.7],ylim=[0,0.7],xlabel=r'$E(B-V)$ stellar',ylabel=r'$E(B-V)$ Balmer')


fig.subplots_adjust(top=0.89)
cax = fig.add_axes([0.125, 0.9, 0.78, 0.02])
cbar = fig.colorbar(sc,label='age / Myr',cax=cax,orientation='horizontal',ticklocation='top')
#cbar.ax.xaxis.set_ticks_position('top')

cbar.set_ticks([0,2,4,6,8,10])
cbar.set_ticklabels([0,2,4,6,8,'10'])


#plt.tight_layout()
plt.savefig(basedir/'reports'/'mass_HA_EBV.pdf',dpi=600)
plt.show()

old stuff

In [ ]:
from scipy.stats import spearmanr
from astrotools.plot.utils import bin_stat
from astrotools.plot.corner import corner_binned_percentile
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig,ax1=plt.subplots(nrows=1,figsize=(single_column,single_column/1.618))

# ----------------------- ax1 ------------------------------------------
criteria  = (catalogue['mass']>1e2) 
#criteria &= (catalogue['age']<=8) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()
print(f'{len(tmp)} objects in sample')
q1 = 68
q2 = 98

xlim =[8e2,1e5]
ylim = [1e4,5e7]
ylim = [1e36,8e39]
vmin,vmax = 0,12
nbins = 8
bins = np.logspace(2.7,5.2,nbins)

cmap = mpl.cm.get_cmap('viridis',6)
norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)
rho = []
age_bins = [0,2,4,6,8,10,12]
for i in range(len(age_bins)-1):

    sub = tmp[(tmp['age']>=age_bins[i]) & (tmp['age']<age_bins[i+1])]
    x,y = sub['mass'],sub['HA6562_LUM_CORR']
    x,mean,std = bin_stat(x,y,bins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_bins[i]+1)))

x,y = tmp['mass'],tmp['HA6562_LUM_CORR']
x,y = x[~np.isnan(y) & np.isfinite(y)],y[~np.isnan(y) & np.isfinite(y)]
corner_binned_percentile(x,y,ax1,nbins=bins,n=68,color='gray',alpha=0.6)
corner_binned_percentile(x,y,ax1,nbins=bins,n=98,color='gray',alpha=0.3)

rho = spearmanr(x,y,nan_policy='omit')[0]
label = r'$\rho'+f'={np.nanmean(rho):.2f}$'
t = ax1.text(0.05,0.93,label,transform=ax1.transAxes,ha='left',va='top')
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

ax1.set(xscale='log',yscale='log',xlim=xlim,ylim=ylim,
        xlabel=r'mass / M$_\odot$',ylabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$')

# we need a scatter plot instance for the color bar
sc = ax1.scatter(19*[1],19*[1],c=19*[1],cmap=cmap,vmin=vmin,vmax=vmax)
#divider = make_axes_locatable(ax1)
#cax = divider.append_axes('top', size="10%", pad=0.3)
#cbar = fig.colorbar(sc,label='age / Myr',cax=cax,orientation='horizontal')
#cbar.ax.xaxis.set_ticks_position('top')

cbar = fig.colorbar(sc,label='age / Myr')

#plt.savefig(basedir/'reports'/'tmp_mass_HA.png',dpi=600)
plt.show()

In [ ]:
from astrotools.plot.utils import bin_stat
from scipy.stats import spearmanr, gaussian_kde, binned_statistic, binned_statistic_2d

criteria  = (catalogue['mass']>1e2) 
#criteria &= (catalogue['age']<=8) #& (catalogue['age']>2)
criteria &= (catalogue['overlap_asc']==1) 

tmp = catalogue[criteria].copy()

fig,ax=plt.subplots(figsize=(single_column,single_column/1.2))

x,y,z = tmp['mass'],tmp['HA6562_LUM_CORR'],tmp['age']


xlim =[1e2,1e5]
ylim = [5e35,8e39]
nbins = 25
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim),nbins)]

hist, x_e, y_e = np.histogram2d(x,y,bins=bins)
stat, x_e, y_e,_ = binned_statistic_2d(x,y,z,bins=bins)
#stat[hist<5] = np.nan
img = ax.pcolormesh(x_e,y_e,stat.T,cmap=plt.cm.viridis,vmin=0,vmax=10)

#sc = ax.scatter(x,y,c=z,vmin=0,vmax=10)
x_stat,mean,std = bin_stat(x,y,bins=bins[0],statistic='median')
ax.errorbar(x_stat,mean,fmt='o-',ms=1.5,color='white')

rho = spearmanr(x,y,nan_policy='omit')[0]
label = r'$\rho'+f'={np.nanmean(rho):.2f}$'
t = ax.text(0.05,0.93,label,transform=ax.transAxes,ha='left',va='top')
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

fig.colorbar(img,label='age / Myr')

ax.set(xscale='log',yscale='log',xlim=xlim,ylim=ylim,
        xlabel=r'mass / M$_\odot$',ylabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$')

#plt.savefig(basedir/'reports'/'mass_HA_age.png',dpi=600)

plt.show()

In [ ]:
from astrotools.plot.utils import bin_stat
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import spearmanr, binned_statistic
from astrotools.plot.corner import corner_binned_percentile

fig,(ax1,ax2)=plt.subplots(nrows=2,figsize=(single_column,single_column*2))

# ----------------------- ax1 ------------------------------------------
criteria  = (catalogue['mass']>1e2) 
criteria &= (catalogue['age']<=8) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()

q1 = 68
q2 = 98

xlim =[8e2,1e5]
ylim = [1e4,5e7]
ylim = [1e36,8e39]
nbins = 8
bins = np.logspace(2.7,5.2,nbins)

cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

rho = []
for k,gal_name in enumerate(np.unique(sample_table['name'])):

    sub = tmp[tmp['gal_name']==gal_name]
    x,y = sub['mass'],sub['HA6562_LUM_CORR']
    if len(x)>5:
        rho.append(spearmanr(x,y,nan_policy='omit')[0]) 
    x,mean,std = bin_stat(x,y,range=xlim,bins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
label = r'$\rho'+f'={np.nanmean(rho):.2f}$'
t = ax1.text(0.05,0.93,label,transform=ax1.transAxes,ha='left',va='top')
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

x,y = tmp['mass'],tmp['HA6562_LUM_CORR']

x,y = x[~np.isnan(y) & np.isfinite(y)],y[~np.isnan(y) & np.isfinite(y)]
corner_binned_percentile(x,y,ax1,nbins=bins,n=68,color='gray',alpha=0.6)
corner_binned_percentile(x,y,ax1,nbins=bins,n=98,color='gray',alpha=0.3)
#x,mean,std = bin_stat(x,y,[None,None],nbins=bins,statistic='median')
#ax.errorbar(x,mean,fmt='o-',ms=2.5,color='black')    

ax1.set(xscale='log',yscale='log',xlim=xlim,ylim=ylim,
        xlabel=r'mass / M$_\odot$',ylabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$')

# we need a scatter plot instance for the color bar
sc = ax1.scatter(19*[1],19*[1],c=sample_table['distance'],cmap=cmap,vmin=9.4,vmax=11)
divider = make_axes_locatable(ax1)
cax = divider.append_axes('top', size="10%", pad=0.3)
cbar = fig.colorbar(sc,label='log M / M$_\odot$',cax=cax,orientation='horizontal',
                    ticks=[9.5,10,10.5,11])
cbar.ax.xaxis.set_ticks_position('top')

# ----------------------- ax2 ------------------------------------------
criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()
print(f'ax1: {len(tmp)} objects')

cmap = plt.cm.get_cmap('viridis',10)


xlim = [0,10]
sc=ax2.scatter(tmp['EBV_stars'],tmp['EBV_balmer'],c=tmp['age'],s=3,vmin=0,vmax=10,cmap=cmap)
EBV_balmer_err = np.mean(tmp['EBV_balmer_err'])
EBV_stars_err = np.mean(tmp['EBV_stars_err'])
ax2.errorbar(0.5,0.1,xerr=EBV_stars_err,yerr=EBV_balmer_err,fmt='ko',ms=0)

ax2.plot([0,1],[0,2],color='black')
ax2.plot([0,2],[0,2],color='black')
ax2.set(xlim=[0,0.7],ylim=[0,0.7],xlabel=r'$E(B-V)$ stars',ylabel=r'$E(B-V)$ Balmer')

divider = make_axes_locatable(ax2)
cax = divider.append_axes('top', size="10%", pad=0.3)
cbar = fig.colorbar(sc,label='age / Myr',cax=cax,orientation='horizontal')
cbar.ax.xaxis.set_ticks_position('top')

cbar.set_ticks([0,2,4,6,8,10])
cbar.set_ticklabels([0,2,4,6,8,'10+'])

plt.tight_layout()
plt.savefig(basedir/'reports'/'mass_HA_EBV.pdf',dpi=600)
plt.show()

### Halpha luminosity function

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

bins = np.logspace(35,40,20)

tmp = nebulae[(nebulae['overlap_neb']==0) & (nebulae['in_frame'])]
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='unmatched',zorder=0)
print(f"isolated: {np.mean(tmp['HA6562_LUM_CORR']):.2g}")

tmp = nebulae[(nebulae['overlap_neb']>0) & (nebulae['in_frame'])]
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='matched',zorder=0)
print(f"overlapping: {np.mean(tmp['HA6562_LUM_CORR']):.2g}")

ax.legend()

ax.set(xscale='log',xlim=[2e35,9e39],xlabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$',
       yscale='linear',ylim=[1,2.2e3],ylabel='N')
plt.savefig(basedir/'reports'/'Halpha_luminosity_function.pdf',dpi=300)
plt.show()


In [ ]:
neb_overlap = np.sum((nebulae['overlap_neb']>0) & (nebulae['in_frame'])) 
neb_in_frame = np.sum(nebulae['in_frame'])
print(f'{neb_overlap} of {neb_in_frame} HII regions overlap ({100*neb_overlap/neb_in_frame:.1f}%)')

asc_overlap = np.sum((associations['overlap_asc']>0) & (associations['in_frame'])) 
asc_in_frame = np.sum(associations['in_frame'])
print(f'{asc_overlap} of {asc_in_frame} associations overlap ({100*asc_overlap/asc_in_frame:.1f}%)')

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

bins = np.linspace(0.5,10.5,10)

tmp = associations[(associations['overlap_asc']==1) & (associations['in_frame'])]
print(np.mean(tmp['age']))
ax.hist(tmp['age'],bins=bins,alpha=0.6,label='contained')

tmp = associations[(associations['overlap_asc']<1) & (associations['overlap_asc']>0) & (associations['in_frame'])]
print(np.mean(tmp['age']))
ax.hist(tmp['age'],bins=bins,alpha=0.6,label='partial')

tmp = associations[(associations['overlap_asc']==0) & (associations['in_frame'])]
print(np.mean(tmp['age']))
ax.hist(tmp['age'],bins=bins,alpha=0.6,label='isolated')

ax.legend()

ax.set(xscale='linear',xlim=[1,10],xlabel=r'age / Myr',
       yscale='linear',ylabel='N')
#plt.savefig(basedir/'reports'/'age_hist.png',dpi=300)
plt.show()


### EBV

we expect 
$$
E(B-V)_{balmer} = 2 \cdot E(B-V)_{stars}
$$


In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astrotools.plot.utils import bin_stat

criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=10) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()
print(f'sample contains {len(tmp)} objects')

cmap = plt.cm.get_cmap('viridis',10)
fig = plt.figure(figsize=(single_column,single_column/1.1))
ax = fig.add_subplot()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size="10%", pad=0.2,)

xlim = [0,10]
#ax.errorbar(tmp['EBV_stars'],tmp['EBV_balmer'],xerr=tmp['EBV_stars_err'],yerr=tmp['EBV_balmer_err'],ms=0,fmt='ok')
sc=ax.scatter(tmp['EBV_stars'],tmp['EBV_balmer'],c=tmp['age'],s=3,vmin=0,vmax=10,cmap=cmap)
#x,mean,std = bin_stat(tmp['EBV_stars'],tmp['EBV_balmer'],[0,0.7],nbins=7,statistic='mean')
#ax.errorbar(x,mean,yerr=std,fmt='-',color='red')
EBV_balmer_err = np.median(tmp['EBV_balmer_err'])
EBV_stars_err = np.median(tmp['EBV_stars_err'])
ax.errorbar(0.5,0.1,xerr=EBV_stars_err,yerr=EBV_balmer_err,fmt='ko',ms=0)

ax.plot([0,1],[0,2],color='black')
ax.plot([0,2],[0,2],color='black')
ax.set(xlim=[0,0.7],ylim=[0,0.7],xlabel=r'$E(B-V)$ stars',ylabel=r'$E(B-V)$ Balmer')
cbar = fig.colorbar(sc,label='age / Myr',cax=cax)
cbar.set_ticks([0,2,4,6,8,10])
cbar.set_ticklabels([0,2,4,6,8,'10+'])

#plt.savefig(basedir/'reports'/f'EBV_Balmer_vs_Stars_{HSTband}_{scalepc}pc.pdf',dpi=600)
#plt.savefig(basedir/'reports'/f'EBV_Balmer_vs_Stars_{HSTband}_{scalepc}pc.png',dpi=600)
plt.show()

In [ ]:
from scipy.stats import spearmanr

age_bins = np.array([0,2,4,6,8,10])

for low,high in zip(age_bins[:-1],age_bins[1:]):
    t = tmp[(tmp['age']>low) & (tmp['age']<high)]
    r,p = spearmanr(t['EBV_stars'],t['EBV_balmer'])
    print(f'{low} to {high} Myr: rho={r:.2f}, {len(t)} objects')


### Temperature and density

In [ ]:
fig,ax=plt.subplots()

ax.scatter(catalogue['age'],catalogue['density'])
x,mean,std = bin_stat(catalogue['age'],catalogue['density'],[0,10],nbins=10,statistic='median')
ax.errorbar(x,mean,yerr=None,fmt='-',color='black')

ax.set(xlim=[0,10])

plt.show()

### GMC comparison

In [ ]:
sep = 4

criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=2)
criteria &= (catalogue['overlap_asc']==1) 

print(f'{np.sum(criteria)} objects are 2 Myr or younger')

tmp1 = catalogue[criteria & (catalogue['GMC_sep']>=sep)]
print(f'{len(tmp1)} not associated with GMC')
print(f'EW(Ha): {np.mean(tmp1["eq_width"]):.1f}')
print(f'Ha/FUV: {np.nanmean(tmp1["HA/FUV_corr"][np.isfinite(tmp1["HA/FUV_corr"])]):.1f}\n')

tmp2 = catalogue[criteria & (catalogue['GMC_sep']<sep)]
print(f'{len(tmp2)} associated with GMC')
print(f'EW(Ha): {np.mean(tmp2["eq_width"]):.1f}')
print(f'Ha/FUV: {np.nanmean(tmp2["HA/FUV_corr"][np.isfinite(tmp2["HA/FUV_corr"])]):.1f}\n')

tmp = catalogue[(catalogue['mass']>1e4) & (catalogue['overlap_asc']==1)]
print(f"age GMC: {np.mean(tmp[tmp['GMC_sep']<sep]['age']):.2f}")
print(f"age without GMC: {np.mean(tmp[tmp['GMC_sep']>=sep]['age']):.2f}")

binned stats, grouped by Ha or GMC mass

In [ ]:
from astrotools.plot.corner import corner_binned_stat
from scipy.stats import spearmanr


fig,(ax1,ax2)=plt.subplots(figsize=(two_column,two_column/2),ncols=2)


tmp = catalogue.copy()
tmp['GMC_sep_pc'] = tmp['GMC_sep'] *u.arcsec.to(u.radian) * tmp['distance']*1e6
tmp = tmp[tmp['GMC_sep_pc']<300]

#sc=ax1.scatter(tmp['age'],tmp['GMC_sep_pc'],c=np.log10(tmp['HA6562_LUM_CORR']),vmin=36,vmax=40)

cmap = mpl.cm.get_cmap('plasma',3)
norm = mpl.colors.Normalize(vmin=36,vmax=39)
Ha_bins = [36,37,38,39]
for i,Ha in enumerate(Ha_bins[:-1]):
    sub = tmp[(tmp['HA6562_LUM_CORR']>10**Ha_bins[i]) & (tmp['HA6562_LUM_CORR']<10**Ha_bins[i+1])]
    x, y = sub['age'], sub['GMC_sep_pc']
    corner_binned_stat(x,y,ax1,bins=np.arange(0.5,11.5,1),color=cmap(norm(Ha+0.5)))
ax1.set(xlim=[0,10],xlabel='age / Myr',ylabel=r'$\delta_\mathrm{GMC}$ / pc')
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax=ax1,orientation='horizontal',location='top',
             ticks=np.arange(36,40,1),label=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$')


cmap = mpl.cm.get_cmap('viridis',3)
norm = mpl.colors.Normalize(vmin=5,vmax=8)
mass_bins = [5,6,7,8]
for i,mass in enumerate(mass_bins[:-1]):
    sub = tmp[(tmp['GMC_mass']>10**mass_bins[i]) & (tmp['GMC_mass']<10**mass_bins[i+1])]
    x, y = sub['age'], sub['GMC_sep_pc']
    corner_binned_stat(x,y,ax2,bins=np.arange(0.5,11.5,1),color=cmap(norm(mass+0.5)))
ax2.set(xlim=[0,10],xlabel='age / Myr',ylabel=r'$\delta_\mathrm{GMC}$ / pc')
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax=ax2,orientation='horizontal',location='top',
             ticks=np.arange(5,9,1),label=r'$M_\mathrm{GMC} / \mathrm{M}_\odot$')


plt.tight_layout()

plt.show()

looks better when using EW as a proxy for the age

In [ ]:
from astrotools.plot.corner import corner_binned_stat, corner_binned_percentile
from scipy.stats import spearmanr, binned_statistic


fig,(ax1,ax2)=plt.subplots(figsize=(single_column,single_column*1.23),nrows=2,sharex=True)

bins = np.logspace(0.5,2.5,6)


tmp = catalogue.copy()
tmp['GMC_sep_pc'] = tmp['GMC_sep'] *u.arcsec.to(u.radian) * tmp['distance']*1e6
tmp = tmp[tmp['GMC_sep_pc']<300]

#sc=ax1.scatter(tmp['age'],tmp['GMC_sep_pc'],c=np.log10(tmp['HA6562_LUM_CORR']),vmin=36,vmax=40)

# the first plot binned by Halpha

cmap = mpl.cm.get_cmap('autumn',3)
norm = mpl.colors.Normalize(vmin=36,vmax=39)

#corner_binned_percentile(tmp['EW_HA'],tmp['GMC_sep_pc'],ax1,nbins=bins,n=68,color='gray',alpha=0.6)
#corner_binned_percentile(tmp['EW_HA'],tmp['GMC_sep_pc'],ax1,nbins=bins,n=98,color='gray',alpha=0.2)

Ha_bins = [36,37,38,39]
#Ha_bins = np.nanpercentile(tmp['HA6562_LUM_CORR'],[0,20,40,60,80,100])
for i,Ha in enumerate(Ha_bins[:-1]):
    sub = tmp[(tmp['HA6562_LUM_CORR']>10**Ha_bins[i]) & (tmp['HA6562_LUM_CORR']<10**Ha_bins[i+1])]
    x, y = sub['EW_HA'], sub['GMC_sep_pc']
    x, y = x[np.isfinite(x) & np.isfinite(y)], y[np.isfinite(x) & np.isfinite(y)]
    mean, edges, _ = binned_statistic(x,y,statistic='median',bins=bins)
    std, edges, _ = binned_statistic(x,y,statistic='std',bins=bins)
    x = (edges[1:]+edges[:-1])/2
    ax1.errorbar(x,mean,yerr=None,fmt='o-',ms=2,color=cmap(norm(Ha+0.5)))
    ax1.fill_between(x,y1=mean-std,y2=mean+std,color=cmap(norm(Ha+0.5)),alpha=0.4)

    
corner_binned_stat(tmp['EW_HA'],tmp['GMC_sep_pc'],ax1,bins=bins,color='black')

ax1.set(xscale='log',xlim=[5,250],ylabel=r'$\delta_\mathrm{GMC}$ / pc',ylim=[10,300])
cbar1 = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax=ax1,orientation='vertical',location='right',
             ticks=np.arange(36,40,1),label=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$')
cbar1.ax.set_yticklabels([f'$10^{{{i}}}$' for i in Ha_bins])

# the second plot binned by GMC mass

cmap = mpl.cm.get_cmap('summer',3)
norm = mpl.colors.Normalize(vmin=5,vmax=8)

#corner_binned_percentile(tmp['EW_HA'],tmp['GMC_sep_pc'],ax2,nbins=bins,n=68,color='gray',alpha=0.6)
#corner_binned_percentile(tmp['EW_HA'],tmp['GMC_sep_pc'],ax2,nbins=bins,n=98,color='gray',alpha=0.2)

                                          
mass_bins = [5,6,7,8]
for i,mass in enumerate(mass_bins[:-1]):
    sub = tmp[(tmp['GMC_mass']>10**mass_bins[i]) & (tmp['GMC_mass']<10**mass_bins[i+1])]
    x, y = sub['EW_HA'], sub['GMC_sep_pc']
    x, y = x[np.isfinite(x) & np.isfinite(y)], y[np.isfinite(x) & np.isfinite(y)]
    mean, edges, _ = binned_statistic(x,y,statistic='median',bins=bins)
    std, edges, _ = binned_statistic(x,y,statistic='std',bins=bins)
    x = (edges[1:]+edges[:-1])/2
    ax2.errorbar(x,mean,yerr=None,fmt='o-',ms=2,color=cmap(norm(mass+0.5)))
    ax2.fill_between(x,y1=mean-std,y2=mean+std,color=cmap(norm(mass+0.5)),alpha=0.4)

corner_binned_stat(tmp['EW_HA'],tmp['GMC_sep_pc'],ax2,bins=bins,color='black')

ax2.set(xscale='log',xlim=[5,250],xlabel=r'EW(Ha) / \AA',ylabel=r'$\delta_\mathrm{GMC}$ / pc',ylim=[10,300])
cbar2 = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax=ax2,orientation='vertical',location='right',
             ticks=np.arange(5,9,1),label=r'$M_\mathrm{GMC}\, /\, \mathrm{M}_\odot$')
cbar2.ax.set_yticklabels([f'$10^{{{i}}}$' for i in mass_bins])

#fig.suptitle('matched HII regions')

plt.tight_layout()
plt.savefig(basedir/'reports'/'GMC_sep_by_EW_binned_matched.png',dpi=400)

plt.show()

In [ ]:

fig,ax=plt.subplots(figsize=(single_column,single_column))


tmp = nebulae.copy()
tmp['GMC_sep_pc'] = tmp['GMC_sep'] *u.arcsec.to(u.radian) * tmp['distance']*1e6
tmp = tmp[tmp['GMC_sep_pc']<300]

ax.scatter(tmp['GMC_mass'],tmp['HA6562_LUM_CORR'])
r,p = spearmanr(tmp['GMC_mass'],tmp['HA6562_LUM_CORR'],nan_policy='omit')
print(f'rho={r:.2f}, p-value={p:.2g}')
ax.set(xlim=[5e4,5e8],xscale='log',ylim=[5e35,5e40],yscale='log')

plt.show()

In [ ]:
from astrotools.plot.corner import corner_binned_stat, corner_binned_percentile
from scipy.stats import spearmanr, binned_statistic


fig,(ax1,ax2)=plt.subplots(figsize=(single_column,single_column*1.23),nrows=2)


tmp = catalogue.copy()
tmp['GMC_sep_pc'] = tmp['GMC_sep'] *u.arcsec.to(u.radian) * tmp['distance']*1e6
tmp = tmp[tmp['GMC_sep_pc']<300]

# 'bar','disc', have very few entries
envs = ['interarm','interbar','spiral arms']


bins = np.logspace(36,40,8)
for i,env in enumerate(envs):
    sub = tmp[tmp['env_neb']==env]
    x, y = sub['HA6562_LUM_CORR'], sub['GMC_sep_pc']
    x, y = x[np.isfinite(x) & np.isfinite(y)], y[np.isfinite(x) & np.isfinite(y)]
    mean, edges, _ = binned_statistic(x,y,statistic='median',bins=bins)
    std, edges, _ = binned_statistic(x,y,statistic='std',bins=bins)
    x = (edges[1:]+edges[:-1])/2
    ax1.errorbar(x,mean,yerr=None,fmt='o-',ms=2,label=env)
    ax1.fill_between(x,y1=mean-std,y2=mean+std,alpha=0.4)
ax1.legend()
corner_binned_stat(tmp['HA6562_LUM_CORR'],tmp['GMC_sep_pc'],ax1,bins=bins,color='black')
ax1.set(xscale='log',xlabel='Halpha luminosity',xlim=[1e36,1e40],ylabel=r'$\delta_\mathrm{GMC}$ / pc',ylim=[10,300])

                            
bins = np.logspace(5,8,8)
for i,env in enumerate(envs):
    sub = tmp[tmp['env_neb']==env]
    x, y = sub['GMC_mass'], sub['GMC_sep_pc']
    x, y = x[np.isfinite(x) & np.isfinite(y)], y[np.isfinite(x) & np.isfinite(y)]
    mean, edges, _ = binned_statistic(x,y,statistic='median',bins=bins)
    std, edges, _ = binned_statistic(x,y,statistic='std',bins=bins)
    x = (edges[1:]+edges[:-1])/2
    ax2.errorbar(x,mean,yerr=None,fmt='o-',ms=2,label=env)
    ax2.fill_between(x,y1=mean-std,y2=mean+std,alpha=0.4)
corner_binned_stat(tmp['GMC_mass'],tmp['GMC_sep_pc'],ax2,bins=bins,color='black')
ax2.set(xscale='log',xlim=[1e5,1e8],xlabel=r'GMC mass',ylabel=r'$\delta_\mathrm{GMC}$ / pc',ylim=[10,300])


plt.tight_layout()
plt.savefig(basedir/'reports'/'GMC_sep_by_LHa_and_GMC_mass.png',dpi=400)

plt.show()

In [ ]:
fig,ax=plt.subplots()

ax.scatter(tmp['age'],tmp['HA6562_LUM_CORR'])

ax.set(xlim=[0,10],ylim=[1e36,1e40],yscale='log')

plt.show()

### Plots for paper

#### define boundaries

In [ ]:
# define limits and labels for corner plots
limits   = {'age':(0.5,8.5),
            'log_age':(5.9,7),
            'age_mw':(0,20),
            'met_scal':(8.3,8.7),
            'Delta_met_scal':(-0.1,0.1),
            'density':(0,150),
            'dig/hii':(0.2,0.9),
            'fesc':(0,1),
            'HA/FUV':(0,200),
            'HA/FUV_corr':(0.8,60),
            'log_HA':(35.7,39.7),
            'HA6562_LUM_CORR':(5e35,5e39),
            'EBV_balmer':(0,1),
            'logq_D91':(5.7,8),
            'EW_HA':(6,600),
            'EW_HA_CORR':(60,6000),
            'temperature':(6000,8e3),
            'log_mass':(4,5.5),
            'GMC_sep':(0,10),
            'GMC_sep_pc':(0,300),
            'RSII':(1.,1.6)}


labels   = {'age':'age / Myr',
            'log_age' : r'$\log (\mathrm{age / Myr})$',
            'age_mw':'age (stellar pops) / Myr',
            'met_scal':'12+logO/H',
            'Delta_met_scal':r'$\Delta$(O/H)',
            'density':r'density / cm$^{-3}$','temperature':'T / K',
            'fesc':r'$f_\mathrm{esc}$',
            'HA/FUV':r'H$\alpha$ / FUV',
            'HA/FUV_corr':r'H$\alpha$ / FUV',
            'HA/FUV_bkg':r'H$\alpha$ / FUV',
            'OIII/HB':r'$[\mathrm{O}\,\textsc{iii}]/\mathrm{H}\beta$',
            'log_HA':r'$\log (L(\mathrm{H}\alpha) / \mathrm{erg s}^{-1})$',
            'HA6562_LUM_CORR':r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$',
            'EBV_balmer':r'$E(B-V)_\mathrm{Balmer}$',
            'sb_HA':r'$SB_{\mathrm{H}\alpha}$',
            'EW_HA' : r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$',
            'EW_HA_CORR' : r'$\mathrm{EW}(\mathrm{H}\alpha)_{\mathrm{corr}}/\mathrm{\AA}$',
            'eq_width_Ha' : r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$',
            'eq_width_Ha_CORR' : r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$',
            'log_eq_width':r'$\log (\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA})$',
            'logq_D91':r'$\log q$',
            'log[SIII]/[SII]':r'$\log([\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}])$',
            'T_N2_REFIT':'temperature / K',
            'log_mass':r'$\log M/\mathrm{M}_\odot$',
            'GMC_sep':r'$\delta_\mathrm{GMC}$ / arcsec',
            'GMC_sep_pc':r'$\delta_\mathrm{GMC}$ / pc',
             'RSII':'RSII'}
scale = {'eq_width':'log','HA/FUV':'log'}

# calculate a few additional columns
with np.errstate(divide='ignore',invalid='ignore'):
    catalogue['log_age'] = 6+np.log10(catalogue['age'])
    catalogue['log_HA'] = np.log10(catalogue['HA6562_LUM_CORR'])
    area_per_pixel = ((0.2*u.arcsec).to(u.rad).value*catalogue['distance']*u.Mpc).to(u.pc)**2
    catalogue['sb_HA_pc']=catalogue['HA6562_FLUX_CORR']/(catalogue['area_neb']*area_per_pixel.value)
    catalogue['sb_HA_arcsec']=catalogue['HA6562_FLUX_CORR']/(catalogue['area_neb']*0.2**2)
    catalogue['log[SIII]/[SII]'] = np.log10(catalogue['[SIII]/[SII]'])
    catalogue['log_mass'] = np.log10(catalogue['mass'])
    catalogue['OIII/HB'] = catalogue['OIII5006_FLUX']/catalogue['HB4861_FLUX_CORR']

In [ ]:
# to determine the right limits

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(two_column,two_column/3))

ax1.hist(nebulae['EW_HA_CORR'],bins=np.logspace(0,4,20))
ax2.hist(nebulae['HA/FUV_corr'],bins=np.logspace(0,2,20))
ax3.hist(nebulae['logq_D91'],bins=np.linspace(5,8,20))

for ax,col in zip([ax1,ax2,ax3],['EW_HA_CORR','HA/FUV_corr','logq_D91']):
    ax.set(xlim=limits[col],xlabel=labels[col])
ax1.set(xscale='log')
ax2.set(xscale='log')
plt.tight_layout()

print(f"EW: {np.log10(np.nanpercentile(nebulae['EW_HA'],[2,98]))}")
print(f"HA/FUV: {np.nanpercentile(nebulae['HA/FUV_corr'],[2,98])}")
print(f"logq: {np.nanpercentile(nebulae['logq_D91'],[2,98])}")

plt.show()

we define the sample here so all three plots use the same data

In [ ]:
criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=8) #& (catalogue['age']>2)
criteria  &= (catalogue['overlap_asc']==1) 

tmp = catalogue[criteria].copy()
tmp = tmp[(tmp['age']>=tmp['age_err'])]

S2N = 5

tmp['logq_D91'][tmp['logq_D91']/tmp['logq_D91_err']<S2N] = np.nan

cont = tmp['continuum_Halpha']-tmp['continuum_Halpha_bkg']
cont_err = np.sqrt(tmp['continuum_Halpha_error']**2+tmp['continuum_Halpha_bkg_error']**2)
tmp['EW_HA'][cont/cont_err<5] = np.nan
tmp['EW_HA_CORR'][cont/cont_err<5] = np.nan
tmp['HA/FUV_corr'][tmp['HA/FUV_corr']/tmp['HA/FUV_corr_err']<S2N] = np.nan
tmp['HA/FUV_corr'][tmp['HA/FUV_corr']<0] = np.nan
# make sure the 3 galaxies without AstroSat are excluded
tmp['HA/FUV_corr'][np.isin(tmp['gal_name'],['NGC1087','NGC4303','NGC5068'])]=np.nan


print(f'sample contains {len(tmp)} objects')

print(f'EW:      {np.sum(~np.isnan(tmp["EW_HA"]))} objects')
print(f'EW_corr: {np.sum(~np.isnan(tmp["EW_HA_CORR"]))} objects')
print(f'HA/FUV:  {np.sum(~np.isnan(tmp["HA/FUV_corr"]))} objects')
print(f'logq:    {np.sum(~np.isnan(tmp["logq_D91"]))} objects')

#### trends with age tracers

In [ ]:
from astrotools.plot.corner import corner_scatter, corner_binned_stat, corner_density_scatter,\
                                corner_density_histogram, corner_gaussian_kde_scatter,\
                                corner_binned_stat2d, corner_binned_stat2d_histogram,\
                                corner_binned_percentile,corner_violin, corner_spearmanr
from scipy.stats import binned_statistic
from astrotools.metallicity.ionization import logq_D91, logq_D91_reverse

    
nbins = 8

columns  = ['age','EW_HA','EW_HA_CORR','HA/FUV_corr','logq_D91']

fig,(ax1,ax2,ax3,ax4) = plt.subplots(nrows=4,figsize=(single_column,single_column*1.618),sharex=True)

for col,ax in zip(columns[1:],[ax1,ax2,ax3,ax4]):
    ax.set(ylim=limits[col])
    x,y = tmp['age'],np.array(tmp[col])
    x,y = x[np.isfinite(x) & np.isfinite(y)], y[np.isfinite(x) & np.isfinite(y)]
    corner_binned_percentile(x,y,ax,nbins=nbins,range=limits['age'],n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=nbins,range=limits['age'],n=98,color='gray',alpha=0.2)

    mean, edges, _ = binned_statistic(x,y,statistic='median',bins=nbins,range=limits['age'])
    xp = (edges[1:]+edges[:-1])/2
    ax.errorbar(xp,mean,fmt='o-',ms=2,color='0.3')
    print(', '.join([f'{100*x:>5.1f}%' for x in mean/mean[0]])+f' ({col})')
    
    corner_density_scatter(x,y,ax,nbins=20,s=0.5,cmap=plt.cm.Reds)
    corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=False,fontsize=7)
    ax.set(ylabel=labels[col])
ax1.set(yscale='log')
ax1.set_yticks([10,100])
ax1.set_yticklabels(['10','100'])

ax2.set(yscale='log')
ax2.set_yticks([100,1000,10000])
ax2.set_yticklabels(['100','1,000',r'$10,000$'])

ax3.set(yscale='log')
ax3.set_yticks([1,10])
ax3.set_yticklabels(['1','10'])

ax4.set(xlabel='age / Myr')
ymin,ymax=limits['logq_D91']
ax4_right = ax4.twinx()
ax4_right.set(ylim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],yscale='log')
ax4_right.set_ylabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
ax4_right.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

plt.tight_layout()

filename = basedir/'reports'/f'age_tracers.pdf'
plt.savefig(filename,dpi=300)

plt.show()

In [ ]:
from scipy.stats import spearmanr

def correlation(x,x_err,y,y_err,name='',sample_size=1000):


    x_sampled = np.random.normal(loc=x,scale=x_err,size=(sample_size,len(x)))
    y_sampled = np.random.normal(loc=y,scale=y_err,size=(sample_size,len(y)))
    
    x_sampled[x_sampled<1]=1
    y_sampled[y_sampled<0]=0

    rho_array,p_array = [],[]
    for xp,yp in zip(x_sampled,y_sampled):
        rho,p = spearmanr(xp,yp,nan_policy='omit')
        rho_array.append(rho)
        p_array.append(p)
    
    print(f'{name}: {np.mean(rho_array):.3f}+-{np.std(rho_array):.3f}')
    

correlation(x=tmp['age'],x_err=tmp['age_err'],
            y=tmp['EW_HA'],y_err=tmp['EW_HA_ERR'],name='EW')
correlation(x=tmp['age'],x_err=tmp['age_err'],
            y=tmp['EW_HA_CORR'],y_err=tmp['logq_D91_err'],name='EW corr')
correlation(x=tmp['age'],x_err=tmp['age_err'],
            y=tmp['HA/FUV_corr'],y_err=tmp['HA/FUV_corr_err'],name='Ha/FUV')
correlation(x=tmp['age'],x_err=tmp['age_err'],
            y=tmp['logq_D91'],y_err=tmp['logq_D91_err'],name='logq')

#### small corner plot grouped by galaxy

In [ ]:
S2N = 5
tmp = nebulae.copy()
#tmp['RSII'] = tmp['SII6716_FLUX_CORR'] / tmp['SII6730_FLUX_CORR']
#tmp['RSII_ERR'] = tmp['RSII'] * np.sqrt( (tmp['SII6716_FLUX_CORR_ERR']/tmp['SII6716_FLUX_CORR'])**2 + (tmp['SII6730_FLUX_CORR_ERR']/tmp['SII6730_FLUX_CORR'])**2 )
tmp['GMC_sep_pc'] = tmp['GMC_sep'] *u.arcsec.to(u.radian) * tmp['distance']*1e6
tmp['GMC_sep_pc'][tmp['GMC_sep_pc']>300] = np.nan

tmp['HA/FUV_corr'][tmp['HA/FUV_corr']/tmp['HA/FUV_corr_err']<S2N] = np.nan
tmp['HA/FUV_corr'][tmp['HA/FUV_corr']<0] = np.nan
tmp['logq_D91'][tmp['logq_D91']/tmp['logq_D91_err']<S2N] = np.nan
# make sure the 3 galaxies without AstroSat are excluded
tmp['HA/FUV_corr'][np.isin(tmp['gal_name'],['NGC1087','NGC4303','NGC5068'])]=np.nan

cont = tmp['continuum_Halpha']-tmp['continuum_Halpha_bkg']
cont_err = np.sqrt(tmp['continuum_Halpha_error']**2+tmp['continuum_Halpha_bkg_error']**2)

# remove objects where we do not detect the continuum
#tmp['EW_HA'][(tmp['EW_HA']<15*tmp['EW_HA_ERR'])] = np.nan
#tmp['EW_HA_CORR'][(tmp['EW_HA_CORR']<15*tmp['EW_HA_CORR_ERR'])] = np.nan
tmp['EW_HA'][cont/cont_err<5] = np.nan
tmp['EW_HA_CORR'][cont/cont_err<5] = np.nan

print(f'sample contains {len(tmp)} objects')
print(f'EW:      {np.sum(~np.isnan(tmp["EW_HA"]))} objects')
print(f'EW_corr: {np.sum(~np.isnan(tmp["EW_HA_CORR"]))} objects')
print(f'HA/FUV:  {np.sum(~np.isnan(tmp["HA/FUV_corr"]))} objects')
print(f'logq:    {np.sum(~np.isnan(tmp["logq_D91"]))} objects')

In [ ]:
import warnings
from scipy.stats import spearmanr
from astrotools.plot import corner
from astrotools.plot.corner import corner_scatter, corner_binned_stat, corner_density_scatter,\
                                corner_density_histogram, corner_gaussian_kde_scatter,\
                                corner_binned_stat2d, corner_binned_stat2d_histogram,\
                                corner_binned_percentile,corner_violin, corner_spearmanr
from astrotools.metallicity.ionization import logq_D91, logq_D91_reverse           
        
cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

def plot_function(x,y,group_by,bins,ax,xlim=None,**kwargs):
    nbins = 6
    finite = np.isfinite(x) & np.isfinite(y)
    x,y,group_by = x[finite], y[finite],group_by[finite]
    
    corner_binned_percentile(x,y,ax,nbins=bins,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=bins,n=98,color='gray',alpha=0.2)
    
    rho = []
    for gal_name in np.unique(group_by):
        x_sub,y_sub=x[group_by==gal_name],y[group_by==gal_name]
        corner_binned_stat(x_sub,y_sub,ax,bins=bins,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        r,p = spearmanr(x_sub,y_sub,nan_policy='omit')
        rho.append(r)
        if np.mean(y_sub)<1:
            print(gal_name)
    r,p = spearmanr(x,y,nan_policy='omit')
    label = r'$\rho'+f'={r:.2f}$'
    t = ax.text(0.08,0.9,label,transform=ax.transAxes,ha='left',va='top',fontsize=7)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
    corner_binned_stat(x,y,ax,bins=bins,color='black')
    #corner_spearmanr(x,y,ax,position=(0.4,0.93),pvalue=True,fontsize=7)
    

columns  = ['EW_HA','HA/FUV_corr','logq_D91']
filename = basedir/'reports'/f'corner_{HSTband}_{scalepc}pc.pdf'

fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(nrows=2,ncols=2,figsize=(single_column,single_column))
ax2.remove()

ax1.set(xlim=limits['EW_HA'],xscale='log',ylim=limits['HA/FUV_corr'],yscale='log',ylabel=labels['HA/FUV_corr'])
ax1.set_xticklabels([])
ax3.set(xlim=limits['EW_HA'],xscale='log',ylim=limits['logq_D91'],xlabel=labels['EW_HA'],ylabel=labels['logq_D91'])
ax4.set(xlim=limits['HA/FUV_corr'],xscale='log',ylim=limits['logq_D91'],xlabel=labels['HA/FUV_corr'])
ax4.set_yticklabels([])

bins = np.logspace(0.7,2.8,7)
plot_function(tmp['EW_HA'],tmp['HA/FUV_corr'],group_by=tmp['gal_name'],bins=bins,ax=ax1,xlim=limits['EW_HA'])
plot_function(tmp['EW_HA'],tmp['logq_D91'],group_by=tmp['gal_name'],bins=bins,ax=ax3,xlim=limits['EW_HA'])
bins = np.logspace(-0.2,1.8,7)
plot_function(tmp['HA/FUV_corr'],tmp['logq_D91'],group_by=tmp['gal_name'],bins=bins,ax=ax4,xlim=limits['HA/FUV_corr'])

ax1.set_yticks([1,10])
ax1.set_yticklabels(['1','10'])
ax3.set_xticks([10,100])
ax3.set_xticklabels(['10','100'])
ax4.set_xticks([1,10])
ax4.set_xticklabels(['1','10'])

ymin,ymax=limits['logq_D91']
ax3_right = ax3.twinx()
ax3_right.set(ylim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],yscale='log')
ax3_right.set_yticklabels([])
ax4_right = ax4.twinx()
ax4_right.set(ylim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],yscale='log')
ax4_right.set_ylabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
ax4_right.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))


# left bottom width height
cbar_ax = fig.add_axes(rect=[0.57,0.65,0.3,0.02])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,orientation='horizontal',ticklocation='top',
             label=r'$\log (M_\star\,/\,\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.8))
#plt.savefig(basedir/'reports'/'nebulae_age_tracers_corner.pdf',dpi=600)
plt.show()

same plot but with corrected EW

In [ ]:
import warnings
from scipy.stats import spearmanr
from astrotools.plot import corner
from astrotools.plot.corner import corner_scatter, corner_binned_stat, corner_density_scatter,\
                                corner_density_histogram, corner_gaussian_kde_scatter,\
                                corner_binned_stat2d, corner_binned_stat2d_histogram,\
                                corner_binned_percentile,corner_violin, corner_spearmanr
from astrotools.metallicity.ionization import logq_D91, logq_D91_reverse                         
        
cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

def plot_function(x,y,group_by,bins,ax,xlim=None,**kwargs):
    nbins = 6
    finite = np.isfinite(x) & np.isfinite(y)
    x,y,group_by = x[finite], y[finite],group_by[finite]
    
    corner_binned_percentile(x,y,ax,nbins=bins,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=bins,n=98,color='gray',alpha=0.2)
    
    rho = []
    for gal_name in np.unique(group_by):
        x_sub,y_sub=x[group_by==gal_name],y[group_by==gal_name]
        corner_binned_stat(x_sub,y_sub,ax,bins=bins,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        r,p = spearmanr(x_sub,y_sub,nan_policy='omit')
        rho.append(r)
        if np.mean(y_sub)<1:
            pass
    r,p = spearmanr(x,y,nan_policy='omit')
    label = r'$\rho'+f'={r:.2f}$'
    t = ax.text(0.08,0.9,label,transform=ax.transAxes,ha='left',va='top',fontsize=7)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
    corner_binned_stat(x,y,ax,bins=bins,color='black')
    #corner_spearmanr(x,y,ax,position=(0.4,0.93),pvalue=True,fontsize=7)
    

columns  = ['EW_HA_CORR','HA/FUV_corr','logq_D91']

fig, ((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(nrows=3,ncols=3,figsize=(1.1*single_column,1.1*single_column),
                                                                sharex='col',sharey='row')
ax2.remove()
ax3.remove()
ax6.remove()

ax1.set(xlim=limits['EW_HA'],xscale='log',ylim=limits['EW_HA_CORR'],yscale='log',ylabel=labels['EW_HA_CORR'])
ax1.set_xticklabels([])
ax4.set(xlim=limits['EW_HA'],xscale='log',ylim=limits['HA/FUV_corr'],yscale='log',ylabel=labels['HA/FUV_corr'])
ax4.set_xticklabels([])
ax7.set(xlim=limits['EW_HA'],xscale='log',ylim=limits['logq_D91'],xlabel=labels['EW_HA'],ylabel=labels['logq_D91'])

ax5.set(xlim=limits['EW_HA_CORR'],xscale='log',ylim=limits['HA/FUV_corr'])
ax8.set(xlim=limits['EW_HA_CORR'],xscale='log',ylim=limits['logq_D91'],xlabel=labels['EW_HA_CORR'])
ax9.set(xlim=limits['HA/FUV_corr'],xscale='log',ylim=limits['logq_D91'],xlabel=labels['HA/FUV_corr'])


bins = np.logspace(0.7,2.8,7)
plot_function(tmp['EW_HA'],tmp['EW_HA_CORR'],group_by=tmp['gal_name'],bins=bins,ax=ax1,xlim=limits['EW_HA'])
plot_function(tmp['EW_HA'],tmp['HA/FUV_corr'],group_by=tmp['gal_name'],bins=bins,ax=ax4,xlim=limits['EW_HA'])
plot_function(tmp['EW_HA'],tmp['logq_D91'],group_by=tmp['gal_name'],bins=bins,ax=ax7,xlim=limits['EW_HA'])

bins = np.logspace(1.7,3.7,7)
plot_function(tmp['EW_HA_CORR'],tmp['HA/FUV_corr'],group_by=tmp['gal_name'],bins=bins,ax=ax5,xlim=limits['EW_HA_CORR'])
plot_function(tmp['EW_HA_CORR'],tmp['logq_D91'],group_by=tmp['gal_name'],bins=bins,ax=ax8,xlim=limits['EW_HA_CORR'])

bins = np.logspace(0.,1.8,7)
plot_function(tmp['HA/FUV_corr'],tmp['logq_D91'],group_by=tmp['gal_name'],bins=bins,ax=ax9,xlim=limits['HA/FUV_corr'])

ax4.set_yticks([1,10])
ax4.set_yticklabels(['1','10'])
ax9.set_xticks([1,10])
ax9.set_xticklabels(['1','10'])
ax7.set_xticks([10,100])
ax7.set_xticklabels(['10','100'])


ymin,ymax=limits['logq_D91']
ax7_right = ax7.twinx()
ax7_right.set(ylim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],yscale='log')
ax7_right.set_yticklabels([])
ax8_right = ax8.twinx()
ax8_right.set(ylim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],yscale='log')
ax8_right.set_yticklabels([])
ax9_right = ax9.twinx()
ax9_right.set(ylim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],yscale='log')
ax9_right.set_ylabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
ax9_right.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

plt.subplots_adjust(wspace=0.08, hspace=0.08)

# left bottom width height
cbar_ax = fig.add_axes(rect=[0.5,0.72,0.35,0.02])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,orientation='horizontal',ticklocation='top',
             label=r'$\log (M_\star\,/\,\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.8))
plt.savefig(basedir/'reports'/'nebulae_age_tracers_corner.pdf',dpi=600)

plt.show()

#### physical trends with proposed age tracers

In [ ]:
from astrotools.plot.corner import corner_binned_stat, corner_binned_percentile
from scipy.stats import spearmanr,binned_statistic
from astrotools.metallicity.ionization import logq_D91, logq_D91_reverse

cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

def plot_function(x,y,group_by,ax,bins,**kwargs):
    nbins = 6
    finite = np.isfinite(x) & np.isfinite(y)
    x,y,group_by = x[finite], y[finite],group_by[finite]
    
    corner_binned_percentile(x,y,ax,nbins=bins,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=bins,n=98,color='gray',alpha=0.2)
    
    rho = []
    for gal_name in np.unique(group_by):
        x_sub,y_sub=x[group_by==gal_name],y[group_by==gal_name]
        corner_binned_stat(x_sub,y_sub,ax,bins=bins,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        r,p = spearmanr(x_sub,y_sub,nan_policy='omit')
        #print(f'{gal_name}: {r:.2f} ({p:.3g})')
        rho.append(r)
        
        #mean, edges, _ = binned_statistic(x_sub,y_sub,statistic='median',bins=bins)
        #std, edges, _ = binned_statistic(x_sub,y_sub,statistic='std',bins=bins)
        #x_mid = (edges[1:]+edges[:-1])/2
        #ax.errorbar(x_mid,mean,yerr=None,fmt='o-',ms=2,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        #ax.fill_between(x_mid,y1=mean-std,y2=mean+std,alpha=0.4,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        
    r,p = spearmanr(x,y,nan_policy='omit')
    label = r'$\rho'+f'={r:.2f}$'
    t = ax.text(0.08,0.9,label,transform=ax.transAxes,ha='left',va='top',fontsize=7)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
    corner_binned_stat(x,y,ax,bins=bins,color='black')


columns = ['Delta_met_scal','HA6562_LUM_CORR','density','EBV_balmer','GMC_sep_pc']

fig,axes=plt.subplots(ncols=3,nrows=len(columns),figsize=(two_column,1.1*two_column/3/1.618*len(columns)),
                      sharex='col',sharey='row')

for i,col in enumerate(columns):
    ax1,ax2,ax3 = axes[i]
    
    bins = np.logspace(0.7,2.8,7)
    plot_function(tmp['EW_HA'],tmp[col],tmp['gal_name'],ax1,bins=bins)
    bins = np.logspace(-0.2,1.8,7)
    plot_function(tmp['HA/FUV_corr'],tmp[col],tmp['gal_name'],ax2,bins=bins)
    bins = np.linspace(5.6,8.1,7)
    plot_function(tmp['logq_D91'],tmp[col],tmp['gal_name'],ax3,bins=bins)

    ax1.set(ylabel=labels[col],ylim=limits[col])
    if col == 'HA6562_LUM_CORR':
        for ax in axes[i]:
            ax.set(yscale='log',ylim=limits[col])
    
    ymin,ymax=limits['logq_D91']
    ax3_top = ax3.twiny()
    ax3_top.set(xlim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],xscale='log')
    ax3_top.set_xticklabels([])
    if i==0:
        ax3_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
        #ax3_top.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(x)))
        ax3_top.set_xticks([0.1,1])
        ax3_top.set_xticklabels(['0.1','1'])

ax1.set(xlabel=labels['EW_HA'],xscale='log',xlim=limits['EW_HA'])    
ax2.set_xticks([10,100])
ax2.set_xticklabels(['10','100'])
ax2.set(xlabel=labels['HA/FUV_corr'],xscale='log',xlim=limits['HA/FUV_corr'])   
ax2.set_xticks([1,10])
ax2.set_xticklabels(['1','10'])
ax3.set(xlabel=labels['logq_D91'],xlim=limits['logq_D91'])    

plt.tight_layout()
plt.subplots_adjust(wspace=0.07, hspace=0.1,top=0.88)
cbar_ax = fig.add_axes([0.18, 0.9, 0.4, 0.02])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,orientation='horizontal',ticklocation='top',
             label=r'$\log (M_\star\,/\,\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.4))

# add arrow to indicate age 
for ax in axes[0]:
    ax.arrow(0.9,0.15,-0.2,0,transform=ax.transAxes,
              head_width=0.08,head_length=0.04,color='black')
    ax.text(0.8,0.18,'age',transform=ax.transAxes,ha='center',va='bottom',fontsize=7)

plt.savefig(basedir/'reports'/'trends_with_age_tracers.pdf',dpi=600)

plt.show()

with corrected EW

In [ ]:
from astrotools.plot.corner import corner_binned_stat, corner_binned_percentile
from scipy.stats import spearmanr,binned_statistic

cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

def plot_function(x,y,group_by,ax,bins,**kwargs):
    nbins = 6
    finite = np.isfinite(x) & np.isfinite(y)
    x,y,group_by = x[finite], y[finite],group_by[finite]
    
    corner_binned_percentile(x,y,ax,nbins=bins,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=bins,n=98,color='gray',alpha=0.2)
    
    rho = []
    for gal_name in np.unique(group_by):
        x_sub,y_sub=x[group_by==gal_name],y[group_by==gal_name]
        corner_binned_stat(x_sub,y_sub,ax,bins=bins,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        r,p = spearmanr(x_sub,y_sub,nan_policy='omit')
        #print(f'{gal_name}: {r:.2f} ({p:.3g})')
        rho.append(r)
        
        #mean, edges, _ = binned_statistic(x_sub,y_sub,statistic='median',bins=bins)
        #std, edges, _ = binned_statistic(x_sub,y_sub,statistic='std',bins=bins)
        #x_mid = (edges[1:]+edges[:-1])/2
        #ax.errorbar(x_mid,mean,yerr=None,fmt='o-',ms=2,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        #ax.fill_between(x_mid,y1=mean-std,y2=mean+std,alpha=0.4,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        
    r,p = spearmanr(x,y,nan_policy='omit')
    label = r'$\rho'+f'={r:.2f}$ {p:.3g}'
    t = ax.text(0.08,0.9,label,transform=ax.transAxes,ha='left',va='top',fontsize=7)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
    corner_binned_stat(x,y,ax,bins=bins,color='black')

columns = ['Delta_met_scal','HA6562_LUM_CORR','density','EBV_balmer','GMC_sep_pc']

fig,axes=plt.subplots(ncols=4,nrows=len(columns),figsize=(two_column,two_column/4*len(columns)),
                      sharex='col',sharey='row')

for i,col in enumerate(columns):
    ax1,ax2,ax3,ax4 = axes[i]
    
    bins = np.logspace(0.7,2.8,7)
    plot_function(tmp['EW_HA'],tmp[col],tmp['gal_name'],ax1,bins=bins)
    bins = np.logspace(1.7,3.7,7)
    plot_function(tmp['EW_HA_CORR'],tmp[col],tmp['gal_name'],ax2,bins=bins)
    bins = np.logspace(0.,1.8,7)
    plot_function(tmp['HA/FUV_corr'],tmp[col],tmp['gal_name'],ax3,bins=bins)
    bins = np.linspace(5.6,8.1,7)
    plot_function(tmp['logq_D91'],tmp[col],tmp['gal_name'],ax4,bins=bins)

    ax1.set(ylabel=labels[col],ylim=limits[col])
    if col == 'HA6562_LUM_CORR':
        for ax in axes[i]:
            ax.set(yscale='log',ylim=limits[col])
    
    ymin,ymax=limits['logq_D91']
    ax4_top = ax4.twiny()
    ax4_top.set(xlim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],xscale='log')
    ax4_top.set_xticklabels([])
    if i==0:
        ax4_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
        ax4_top.set_xticks([0.1,1])

ax1.set(xlabel=labels['EW_HA'],xscale='log',xlim=limits['EW_HA'])    
#ax2.set_xticks([100,100,1000])
ax2.set(xlabel=labels['EW_HA_CORR'],xscale='log',xlim=limits['EW_HA_CORR'])    
ax2.set_xticks([100,100,1000])
ax2.set_xticklabels(['10','100','1000'])
ax3.set(xlabel=labels['HA/FUV_corr'],xscale='log',xlim=limits['HA/FUV_corr'])   
ax3.set_xticks([1,10])
ax3.set_xticklabels(['1','10'])
ax4.set(xlabel=labels['logq_D91'],xlim=limits['logq_D91'])    


plt.subplots_adjust(wspace=0.05, hspace=0.07,top=0.88)
cbar_ax = fig.add_axes([0.2, 0.89, 0.4, 0.02])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,orientation='horizontal',ticklocation='top',
             label=r'$\log (M_\star\,/\,\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.4))

# add arrow to indicate age 
for ax in axes[0]:
    ax.arrow(0.9,0.15,-0.2,0,transform=ax.transAxes,
              head_width=0.08,head_length=0.04,color='black')
    ax.text(0.8,0.18,'age',transform=ax.transAxes,ha='center',va='bottom',fontsize=7)

plt.savefig(basedir/'reports'/'trends_with_age_tracers.pdf',dpi=600)

plt.show()

In [ ]:
from astrotools.plot.corner import corner_binned_stat, corner_binned_percentile
from scipy.stats import spearmanr,binned_statistic
from astrotools.metallicity.ionization import logq_D91, logq_D91_reverse

cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=35,vmax=40)

def plot_function(x,y,group_by,ax,bins,**kwargs):
    nbins = 6
    finite = np.isfinite(x) & np.isfinite(y)
    x,y,group_by = x[finite], y[finite],group_by[finite]
    
    corner_scatter(x,y,ax=ax,s=0.4,alpha=0.5,cmap=cmap,c=group_by,vmin=35,vmax=40)
    
    r,p = spearmanr(x,y,nan_policy='omit')
    label = r'$\rho'+f'={r:.2f}$'
    t = ax.text(0.08,0.9,label,transform=ax.transAxes,ha='left',va='top',fontsize=7)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
    corner_binned_stat(x,y,ax,bins=bins,color='black')


columns = ['Delta_met_scal','HA6562_LUM_CORR','density','EBV_balmer','GMC_sep_pc']

fig,axes=plt.subplots(ncols=4,nrows=len(columns),figsize=(two_column,1.1*two_column/3/1.618*len(columns)),
                      sharex='col',sharey='row')

color = 'HA6562_LUM_CORR'
for i,col in enumerate(columns):
    ax1,ax2,ax3,ax4 = axes[i]
    
    bins = np.logspace(0.7,2.8,7)
    plot_function(tmp['EW_HA'],tmp[col],np.log10(tmp[color]),ax1,bins=bins)
    bins = np.logspace(1.7,3.7,7)
    plot_function(tmp['EW_HA_CORR'],tmp[col],np.log10(tmp[color]),ax2,bins=bins)
    bins = np.logspace(0.,1.8,7)
    plot_function(tmp['HA/FUV_corr'],tmp[col],np.log10(tmp[color]),ax3,bins=bins)
    bins = np.linspace(5.6,8.1,7)
    plot_function(tmp['logq_D91'],tmp[col],np.log10(tmp[color]),ax4,bins=bins)

    ax1.set(ylabel=labels[col],ylim=limits[col])
    if col == 'HA6562_LUM_CORR':
        for ax in axes[i]:
            ax.set(yscale='log',ylim=limits[col])
    
    ymin,ymax=limits['logq_D91']
    ax4_top = ax4.twiny()
    ax4_top.set(xlim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],xscale='log')
    ax4_top.set_xticklabels([])
    if i==0:
        ax4_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
        #ax3_top.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(x)))
        ax4_top.set_xticks([0.1,1])
        ax4_top.set_xticklabels(['0.1','1'])

ax1.set(xlabel=labels['EW_HA'],xscale='log',xlim=limits['EW_HA'])    
ax2.set(xlabel=labels['EW_HA_CORR'],xscale='log',xlim=limits['EW_HA_CORR'])    
ax3.set_xticks([10,100])
ax3.set_xticklabels(['10','100'])
ax3.set(xlabel=labels['HA/FUV_corr'],xscale='log',xlim=limits['HA/FUV_corr'])   
ax3.set_xticks([1,10])
ax3.set_xticklabels(['1','10'])
ax4.set(xlabel=labels['logq_D91'],xlim=limits['logq_D91'])    

plt.tight_layout()
plt.subplots_adjust(wspace=0.07, hspace=0.1,top=0.88)
cbar_ax = fig.add_axes([0.18, 0.9, 0.4, 0.02])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,orientation='horizontal',ticklocation='top',
             label=r'Halpha Luminosity')

# add arrow to indicate age 
for ax in axes[0]:
    ax.arrow(0.9,0.15,-0.2,0,transform=ax.transAxes,
              head_width=0.08,head_length=0.04,color='black')
    ax.text(0.8,0.18,'age',transform=ax.transAxes,ha='center',va='bottom',fontsize=7)

#plt.savefig(basedir/'reports'/'trends_with_age_tracers.pdf',dpi=600)

plt.show()

### corner plot

In [ ]:
import warnings
from astrotools.plot import corner
from astrotools.plot.corner import corner_scatter, corner_binned_stat, corner_density_scatter,\
                                corner_density_histogram, corner_gaussian_kde_scatter,\
                                corner_binned_stat2d, corner_binned_stat2d_histogram,\
                                corner_binned_percentile,corner_violin, corner_spearmanr
                             
                             
def plot_function(x,y,ax,xlim=None,**kwargs):
    nbins = 7
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=98,color='gray',alpha=0.2)
    corner_binned_stat(x,y,ax,bins=nbins,color='0.3')
    #corner_density_histogram(x,y,ax,nbins=10,cmap=plt.cm.gray_r)
    #corner_scatter(x,y,ax,s=0.5)
    #corner_gaussian_kde_scatter(x,y,ax,s=0.5,cmap=plt.cm.Reds)
    corner_density_scatter(x,y,ax,nbins=20,s=0.5,cmap=plt.cm.Reds)
    corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=False,fontsize=7)
    

tmp = nebulae.copy()
print(f'sample contains {len(tmp)} objects')
tmp['log_HA'] = np.log10(tmp['HA6562_LUM_CORR'])

columns  = ['EW_HA','HA/FUV_corr','log_HA','Delta_met_scal','T_N2_REFIT','density','EBV_balmer','GMC_sep']
filename = basedir/'reports'/f'corner_{HSTband}_{scalepc}pc.pdf'

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner(tmp,columns,function=plot_function,
           limits=limits,labels=labels,
           filename=basedir/'reports'/f'corner_large.pdf',
           figsize=2*two_column,aspect_ratio=1)
plt.show()

In [ ]:
import warnings
from astrotools.plot import corner
from astrotools.plot.corner import corner_scatter, corner_binned_stat, corner_density_scatter,\
                                corner_density_histogram, corner_gaussian_kde_scatter,\
                                corner_binned_stat2d, corner_binned_stat2d_histogram,\
                                corner_binned_percentile,corner_violin, corner_spearmanr
                             
def plot_function(x,y,ax,xlim=None,**kwargs):
    nbins = 7
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=98,color='gray',alpha=0.2)
    corner_binned_stat(x,y,ax,bins=nbins,color='0.3')
    #corner_density_histogram(x,y,ax,nbins=10,cmap=plt.cm.gray_r)
    #corner_scatter(x,y,ax,s=0.5)
    #corner_gaussian_kde_scatter(x,y,ax,s=0.5,cmap=plt.cm.Reds)
    corner_density_scatter(x,y,ax,nbins=20,s=0.5,cmap=plt.cm.Reds)
    corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=True,fontsize=7)
    

criteria  = (catalogue['mass']>1e4) 
#criteria &= (catalogue['HA/FUV']>3*catalogue['HA/FUV_err']) | np.isnan(catalogue['FUV_FLUX'])
#criteria &= catalogue['[SIII]/[SII]']>3*catalogue['[SIII]/[SII]_err']
criteria &= (catalogue['age']<=8) #& (catalogue['age']>2)
# young objects should be associated with a GMC
#criteria &= ((catalogue['GMC_sep']<4) | (catalogue['age']>2))
#criteria &= catalogue['U_dolmag_vega']-catalogue['B_dolmag_vega']<-1.
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria  &= (catalogue['overlap_asc']==1) 
#criteria &= (catalogue['neighbors']==0)

tmp = catalogue[criteria].copy()
tmp['HA/FUV_corr'][tmp['FUV_FLUX_CORR']/tmp['FUV_FLUX_CORR_ERR']<3] = np.nan
tmp['EW_HA'][tmp['EW_HA']/tmp['EW_HA_ERR']<3] = np.nan
tmp['logq_D91'][tmp['logq_D91']/tmp['logq_D91_err']<3] = np.nan
tmp = tmp.filled()

print(f'sample contains {len(tmp)} objects')


columns  = ['age','eq_width_Ha','HA/FUV_corr','logq_D91']

filename = None #basedir/'reports'/f'corner_{HSTband}_{scalepc}pc.pdf'

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner(tmp,columns,function=plot_function,
           limits=limits,labels=labels,
           filename=filename,histogram=False,
           figsize=two_column,aspect_ratio=1)
plt.show()

In [ ]:
def plot_function(x,y,ax,xlim=None,**kwargs):
    nbins = 7
    x,y = x[np.isfinite(x) & np.isfinite(y)], y[np.isfinite(x) & np.isfinite(y)]
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=98,color='gray',alpha=0.2)
    corner_binned_stat(x,y,ax,bins=nbins,color='0.3')
    corner_density_scatter(x,y,ax,nbins=20,s=0.5,cmap=plt.cm.Reds)
    corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=False,fontsize=7)
    
tmp = nebulae.copy()  
tmp['log_HA'] = np.log10(tmp['HA6562_LUM_CORR'])

columns = ['Delta_met_scal','log_HA','T_N2_REFIT','density','EBV_balmer','GMC_sep']

fig,axes=plt.subplots(ncols=3,nrows=len(columns),figsize=(two_column,two_column/3/1.618*len(columns)),
                      sharex='col',sharey='row')


for i,col in enumerate(columns):
    ax1,ax2,ax3 = axes[i]
    
    plot_function(tmp['EW_HA'],tmp[col],ax1,xlim=limits['EW_HA'])
    plot_function(tmp['HA/FUV_corr'],tmp[col],ax2,xlim=limits['HA/FUV_corr'])
    plot_function(tmp['logq_D91'],tmp[col],ax3,xlim=limits['logq_D91'])
    
    ax1.set(ylabel=labels[col],ylim=limits[col])
    
ax1.set(xlabel=labels['EW_HA'],xlim=limits['EW_HA'])    
ax2.set(xlabel=labels['HA/FUV_corr'],xlim=limits['HA/FUV_corr'])    
ax3.set(xlabel=labels['logq_D91'],xlim=limits['logq_D91'])    

plt.tight_layout()

plt.show()

or binned by galaxy

In [ ]:
limits['log_HA'] = (35.,40)
limits['GMC_sep'] = (0.,20)
limits['T_N2_REFIT'] = (6000,12000)

tmp = nebulae.copy()  
tmp['log_HA'] = np.log10(tmp['HA6562_LUM_CORR'])

columns = ['Delta_met_scal','log_HA','T_N2_REFIT','density','EBV_balmer','GMC_sep']

fig,axes=plt.subplots(ncols=3,nrows=len(columns),figsize=(two_column,two_column/3/1.618*len(columns)),
                      sharex='col',sharey='row')

nbins = 7
cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

for i,col in enumerate(columns):
    ax1,ax2,ax3 = axes[i]
    
    for xcol,ax in zip(['EW_HA','HA/FUV_corr','logq_D91'],[ax1,ax2,ax3]):
        
        x,y = tmp[xcol],tmp[col]
        x,y = x[np.isfinite(x) & np.isfinite(y)], y[np.isfinite(x) & np.isfinite(y)]
        corner_binned_percentile(x,y,ax,nbins=nbins,range=limits[xcol],n=68,color='gray',alpha=0.6)
        corner_binned_percentile(x,y,ax,nbins=nbins,range=limits[xcol],n=98,color='gray',alpha=0.2)
        corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=False,fontsize=7)
        
        for gal_name in np.unique(tmp['gal_name']):
            x,y = tmp[tmp['gal_name']==gal_name][xcol],tmp[tmp['gal_name']==gal_name][col]
            x,y = x[np.isfinite(x) & np.isfinite(y)], y[np.isfinite(x) & np.isfinite(y)]
            corner_binned_stat(x,y,ax,bins=nbins,color=cmap(norm(sample_table.loc[gal_name]['mass'])))

        corner_binned_stat(x,y,ax,bins=nbins,color='0.3')
        
    
    ax1.set(ylabel=labels[col],ylim=limits[col])
    
ax1.set(xlabel=labels['EW_HA'],xlim=limits['EW_HA'])    
ax2.set(xlabel=labels['HA/FUV_corr'],xlim=limits['HA/FUV_corr'])    
ax3.set(xlabel=labels['logq_D91'],xlim=limits['logq_D91'])    

plt.tight_layout()

plt.show()

histograms binned by age

In [ ]:
criteria  = (catalogue['mass']>=1e4) 
criteria &= (catalogue['overlap_asc']==1) 
#criteria &= ((catalogue['GMC_sep']<2) | (catalogue['age']>2))

tmp = catalogue[criteria].copy()

fig,(ax1,ax2,ax3,ax4) = plt.subplots(ncols=4,figsize=(1.5*two_column,1.5*two_column/3))

style = {'alpha':0.5}
nbins = 10
age_bins = [0,2,8]
for i in range(len(age_bins)-1):
    sub = tmp[(tmp['age']>age_bins[i]) & (tmp['age']<=age_bins[i+1])]
    label = f'{age_bins[i]}\> age \>= {age_bins[i+1]} Myr: '
    
    ax1.hist(sub['eq_width'],bins=np.linspace(*limits['eq_width'],nbins),**style,
             label=label+f"{np.nanmean(sub['eq_width']):.1f}")
    ax2.hist(sub['HA/FUV_corr'],bins=np.linspace(*limits['HA/FUV_corr'],nbins),**style,
            label=label+f"{np.nanmedian(sub['HA/FUV_corr']):.1f}")
    ax3.hist(sub['Delta_met_scal'],bins=np.linspace(*limits['Delta_met_scal'],nbins),**style,
            label=label+f"{np.nanmean(sub['Delta_met_scal']):.2f}")
    ax4.hist(sub['logq_D91'],bins=np.linspace(*limits['logq_D91'],nbins),**style,
             label=label+f"{np.nanmean(sub['logq_D91']):.1f}")
    
ax1.set(xlabel=labels['eq_width'],xlim=limits['eq_width'])
ax2.set(xlabel=labels['HA/FUV_corr'],xlim=limits['HA/FUV_corr'])
ax3.set(xlabel=labels['Delta_met_scal'],xlim=limits['Delta_met_scal'])
ax4.set(xlabel=labels['logq_D91'],xlim=limits['logq_D91'])
ax1.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=1)
ax2.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=1)
ax3.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=1)
ax4.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=1)

plt.tight_layout()

plt.show()

### Quantify changes in numbers

we compare the change of the EW(Ha) and Ha/FUV to the predicted numbers from Starburst99

In [ ]:
from scipy.stats import binned_statistic

criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=8) #& (catalogue['age']>2)
criteria  &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()

EW_HA_lst, edges, _ = binned_statistic(list(tmp['age']),list(tmp['EW_HA']),statistic='median',bins=8,range=[0.5,8.5])
HaFUV_lst, edges, _ = binned_statistic(list(tmp['age']),list(tmp['HA/FUV_corr']),statistic='median',bins=8,range=[0.5,8.5])
xp = (edges[1:]+edges[:-1])/2


print('    age  EW(Ha)  Ha/FUV')
for age,EW_HA,HaFUV in zip(xp,EW_HA_lst,HaFUV_lst):
    print(f'{age} Myr {EW_HA/EW_HA_lst[0]:>5.2f} % {HaFUV/HaFUV_lst[0]:>5.2f} %')

### log q vs age vs pressure

see also Dopita+2006.

In [ ]:
from astrotools.plot.utils import bin_stat

criteria  = (catalogue['mass']>1e3) 
#criteria &= (catalogue['age']<=8) #& (catalogue['age']>2)
#criteria  &= (catalogue['overlap_asc']==1) 
#criteria  &= (catalogue['logq_D91']>3*catalogue['logq_D91_err']) 

tmp = catalogue[criteria].copy()

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

cmap = mpl.cm.get_cmap('plasma',6)
norm = mpl.colors.Normalize(vmin=6.5,vmax=9.5)
rho = []
n_bins = np.arange(6.5,10,0.5)
for i in range(len(n_bins)-1):
    sub = tmp[(tmp['LOG_SIGMA_STMASS_CHAB']>=n_bins[i]) & (tmp['LOG_SIGMA_STMASS_CHAB']<n_bins[i+1])]
    x,mean,std = bin_stat(sub['age'],sub['logq_D91'],bins=np.linspace(0.5,10.5,5),statistic='median')
    ax.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(n_bins[i]+0.25)))
    
ax.set(xlim=[0,10],ylim=[6.1,7.5],xlabel='age / Myr',ylabel=r'$\log q$ (Diaz+91)')
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label=r'$\Sigma_M$ / M$_\odot$ kpc$^{-2}$')

plt.show()

by metallicity

In [ ]:
from astrotools.plot.utils import bin_stat

criteria  = (catalogue['mass']>1e3) 
#criteria &= (catalogue['age']<=8) #& (catalogue['age']>2)
#criteria  &= (catalogue['overlap_asc']==1) 
#criteria  &= (catalogue['logq_D91']>3*catalogue['logq_D91_err']) 

tmp = catalogue[criteria].copy()

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

cmap = mpl.cm.get_cmap('plasma',3)
norm = mpl.colors.Normalize(vmin=8.2,vmax=8.8)
rho = []
n_bins = [8.2,8.4,8.6,8.8]
for i in range(len(n_bins)-1):
    sub = tmp[(tmp['met_scal']>=n_bins[i]) & (tmp['met_scal']<n_bins[i+1])]
    x,mean,std = bin_stat(sub['age'],sub['logq_D91'],bins=np.linspace(0.5,10.5,5),statistic='median')
    ax.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(n_bins[i]+0.1)))
    
sc = ax.scatter(tmp['age'],tmp['logq_D91'],c=tmp['met_scal'],cmap=cmap,vmin=8.2,vmax=8.8)
ax.set(xlim=[0,10],ylim=[6.1,7.5],xlabel='age / Myr',ylabel=r'$\log q$ (Diaz+91)')
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label=r'$12+\log (\mathrm{O}/\mathrm{H})$')

plt.show()

In [ ]:

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

cmap = mpl.cm.get_cmap('plasma',6)
norm = mpl.colors.Normalize(vmin=6.5,vmax=9.5)
n_bins = np.arange(6.5,10,1)
for i in range(len(n_bins)-1):
    sub = tmp[(tmp['LOG_SIGMA_STMASS_CHAB']>=n_bins[i]) & (tmp['LOG_SIGMA_STMASS_CHAB']<n_bins[i+1])]
    ax.hist(sub['logq_D91'],color=cmap(norm(n_bins[i]+0.25)),alpha=0.6,label=n_bins[i]+0.25)
    
sc = ax.scatter([-1],[-1],c=[8],cmap=cmap,vmin=6.5,vmax=9.5)
ax.set(xlim=[6.1,7.5],xlabel=r'$\log q$ (Diaz+91)')
#fig.colorbar(sc,label=r'$\Sigma_M$ / M$_\odot$ kpc$^{-2}$')
ax.legend()
plt.show()

### compare with starburst99

In [ ]:
from starburst import Cluster, find_model, make_folder, list_available_models

cluster_CSF = Cluster(basedir/'..'/'starburst'/'data'/'others'/'continuous_SF')
cluster_SSF = Cluster(basedir/'..'/'starburst'/'data'/'others'/'single_burst_SF')
cluster_SSF.measure_FUV()
cluster_CSF.measure_FUV()

In [ ]:
# Halpha/FUV
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))

tmp = catalogue[(catalogue['mass']>1e4) & (catalogue['overlap_asc']==1)]
ax1.scatter(tmp['age'],tmp['HA/FUV_corr'],color='gray')

time = cluster_CSF.ewidth['Time']/1e6
Ha  = cluster_CSF.ewidth['Luminosity_H_A'].copy()
FUV = np.interp(cluster_CSF.ewidth['Time'],cluster_CSF.FUV['Time'],cluster_CSF.FUV['FUV'])
ax1.plot(time,Ha/FUV,label='continuous')

time = cluster_SSF.ewidth['Time']/1e6
Ha  = cluster_SSF.ewidth['Luminosity_H_A'].copy()
FUV = cluster_SSF.FUV['FUV'].copy()
ax1.plot(time,Ha/FUV,label='single burst')
ax1.legend()
ax1.set(xlim=[0,10],ylim=[0,100],xlabel='age / Myr',ylabel=r'H$\alpha$/FUV')


ax2.scatter(tmp['age'],tmp['eq_width'],color='gray')
time = cluster_CSF.ewidth['Time']/1e6
eq_width  = cluster_CSF.ewidth['eq_width_H_A']
ax2.plot(time,0.1*eq_width,label='continuous')

time = cluster_SSF.ewidth['Time']/1e6
eq_width  = cluster_SSF.ewidth['eq_width_H_A']
ax2.plot(time,0.1*eq_width,label='singel burst')
ax2.legend()
ax2.set(xlim=[0,10],ylim=[0,400],yscale='linear',xlabel='age / Myr',ylabel=r'EW(H$\alpha$)')

plt.tight_layout()

plt.show()

the distribution of EW observed vs model

In [ ]:
from starburst import Cluster, find_model, make_folder, list_available_models

cluster_solar = Cluster(stellar_model='GENEVAv40',metallicity=0.014)
cluster_subsolar = Cluster(stellar_model='GENEVAv40',metallicity=0.004)


In [ ]:
tmp = catalogue.copy()

eq_width_model_solar = cluster_solar.ewidth['eq_width_H_A']
eq_width_model_subsolar = cluster_subsolar.ewidth['eq_width_H_A']
age_model = cluster_solar.ewidth['Time']

tmp['eq_width_model_solar'] = np.nan
tmp['eq_width_model_subsolar'] = np.nan
for row in tmp:
    idx = np.argmin(np.abs(age_model-row['age']*u.Myr))
    row['eq_width_model_solar'] = eq_width_model_solar[idx].value
    row['eq_width_model_subsolar'] = eq_width_model_subsolar[idx].value

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

bins = np.logspace(0,4,15)

ax.hist(tmp['eq_width'],bins=bins,label='observed',alpha=0.6)
ax.hist(tmp['eq_width_model_solar'],bins=bins,label='model solar',alpha=0.6)
#ax.hist(tmp['eq_width_model_subsolar'],bins=bins,label='model subsolar',alpha=0.6)
ax.legend()
ax.set(xscale='log',xlim=[1,1e4],xlabel=r'$\mathrm{EW}(\mathrm{H}\alpha)$')

plt.show()

### other things like violine plots

In [ ]:
import warnings
from astrotools.plot import corner
from astrotools.plot.corner import corner_scatter, corner_binned_stat, corner_density_scatter,\
                                corner_density_histogram, corner_gaussian_kde_scatter,\
                                corner_binned_stat2d, corner_binned_stat2d_histogram,\
                                corner_binned_percentile,corner_violin, corner_spearmanr
                             
def plot_function(x,y,ax,xlim=None,**kwargs):
    nbins = 7
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=98,color='gray',alpha=0.2)
    corner_binned_stat(x,y,ax,nbins=nbins,color='0.3')
    #corner_density_histogram(x,y,ax,nbins=10,cmap=plt.cm.gray_r)
    #corner_scatter(x,y,ax,s=0.5)
    #corner_gaussian_kde_scatter(x,y,ax,s=0.5,cmap=plt.cm.Reds)
    corner_density_scatter(x,y,ax,nbins=20,s=0.5,cmap=plt.cm.Reds)
    corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=False,fontsize=7)
    
filename = basedir/'reports'/f'corner_{HSTband}_{scalepc}pc.pdf'

criteria  = (catalogue['mass']>1e4) 
#criteria &= (catalogue['HA/FUV']>3*catalogue['HA/FUV_err']) | np.isnan(catalogue['FUV_FLUX'])
#criteria &= catalogue['[SIII]/[SII]']>3*catalogue['[SIII]/[SII]_err']
criteria &= (catalogue['age']<=8) #& (catalogue['age']>2)
# young objects should be associated with a GMC
#criteria &= ((catalogue['GMC_sep']<4) | (catalogue['age']>2))
#criteria &= catalogue['U_dolmag_vega']-catalogue['B_dolmag_vega']<-1.
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria &= (catalogue['overlap_asc']==1) 
#criteria &= (catalogue['neighbors']==0)

tmp = catalogue[criteria].copy()
print(f'sample contains {len(tmp)} objects')
with np.errstate(divide='ignore',invalid='ignore'):
    tmp['log_age'] = 6+np.log10(tmp['age'])
    tmp['log_eq_width'] = np.log10(tmp['eq_width'])
    tmp['log[SIII]/[SII]'] = np.log10(tmp['[SIII]/[SII]'])

columns  = ['logq_D91','Delta_met_scal','HA/FUV_corr','log_eq_width']

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner(tmp,columns,function=plot_function,
           limits=limits,labels=labels,
           filename=None,
           figsize=two_column,aspect_ratio=1)
plt.show()

In [ ]:
fig,ax=plt.subplots()


x = tmp['r_R25']
y = tmp['log_eq_width']

ax.scatter(x,y,c='gray')
corner_binned_stat(x,y,ax)
corner_spearmanr(x,y,ax,pvalue=True)

ax.set(xlim=[0,1],ylim=[0,3])
plt.show()

use a violin plot to show the distribution of points in each age bin

In [ ]:
fig,ax = plt.subplots()

x_label = 'age'
y_label = 'eq_width'

x,y = tmp[x_label], tmp[y_label]
x,y = x[~np.isnan(y)],y[~np.isnan(y)]
x,y = x[(x>=limits[x_label][0]) & (x<=limits[x_label][1]) & (y>=limits[y_label][0]) & (y<=limits[y_label][1])], \
      y[(x>=limits[x_label][0]) & (x<=limits[x_label][1]) & (y>=limits[y_label][0]) & (y<=limits[y_label][1])]

positions = np.arange(0,10)

#binned_data = corner_violin(x,y,ax,positions,showmedians=True)

bins = (positions[1:]+positions[:-1])/2
binned_data = [y[(bins[i]<x) & (x<bins[i+1])] for i in range(len(bins)-1)]
ax.violinplot(binned_data,positions=positions[1:-1],showmeans=True)

ax.set(xlabel=x_label,ylabel=y_label.replace('_',''))

plt.show()

filter age with color-color-diagram

In [ ]:
points = np.array([[-0.33 , -1.662],[-0.307, -1.577],[-0.25 , -1.468],[-0.218, -1.394],[-0.186, -1.321],
                   [-0.138, -1.235],[-0.076, -1.213],[-0.007, -1.206],[ 0.065, -1.197],[ 0.135, -1.187],
                   [ 0.203, -1.198],[ 0.276, -1.201],[ 0.347, -1.209],[ 0.418, -1.219],[ 0.488, -1.233],
                   [ 0.559, -1.251],[ 0.63 , -1.27 ],[ 0.706, -1.264],[ 0.771, -1.287],[ 0.648, -1.119],
                   [ 0.626, -1.026],[ 0.565, -0.989],[ 0.559, -0.87 ],[ 0.502, -0.807],[ 0.481, -0.706],
                   [ 0.457, -0.546],[ 0.505, -0.449],[ 0.487, -0.384],[ 0.488, -0.328],[ 0.468, -0.239],
                   [ 0.517, -0.109],[ 0.555, -0.015],[ 0.6  ,  0.041],[ 0.65 ,  0.132],[ 0.724,  0.174],
                   [ 0.864,  0.169],[ 0.948,  0.166],[ 1.02 ,  0.152],[ 1.138,  0.182],[ 1.203,  0.194],
                   [ 1.253,  0.282],[ 1.331,  0.378],[ 1.354,  0.469]])

x,y=points.T
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(2.3*single_column,1*single_column))

tmp = associations
#ax.scatter(tmp['V_dolmag_vega']-tmp['I_dolmag_vega'],tmp['U_dolmag_vega']-tmp['B_dolmag_vega'],s=0.5)
tmp = catalogue
sc=ax1.scatter(tmp['V_dolmag_vega']-tmp['I_dolmag_vega'],tmp['U_dolmag_vega']-tmp['B_dolmag_vega'],
           c=tmp['GMC_sep'],vmin=0,vmax=10,
           s=0.5)
ax1.plot(x,y,color='red')
ax1.set(xlabel=r'$V-I$',ylabel=r'$U-B$',xlim=[-1.5,2],ylim=[-2.5,1])
ax1.invert_yaxis()
plt.colorbar(sc,label='GMC sep / arcsec',ax=ax1)

sc=ax2.scatter(tmp['V_dolmag_vega']-tmp['I_dolmag_vega'],tmp['U_dolmag_vega']-tmp['B_dolmag_vega'],
           c=tmp['age'],vmin=0,vmax=20,cmap=plt.cm.plasma,
           s=0.5)
ax2.plot(x,y,color='red')
ax2.set(xlabel=r'$V-I$',ylabel=r'$U-B$',xlim=[-1.5,2],ylim=[-2.5,1])
ax2.invert_yaxis()
fig.colorbar(sc,label='age / Myr',ax=ax2)

plt.tight_layout()

plt.savefig('UBVI_matched_associations.png',dpi=600)
plt.show()

In [ ]:
fig,ax=plt.subplots()

bins=np.arange(0,150,10)
tmp = catalogue[(catalogue['age']==1) & (catalogue['overlap_asc']==1)]
mask = tmp['GMC_sep']<4

ax.hist(tmp['eq_width'][mask],bins=bins,label=f'nearby GMC: {np.mean(tmp["eq_width"][mask]):.2f} AA',histtype='step',alpha=0.8)
ax.hist(tmp['eq_width'][~mask],bins=bins,label=f'no GMC {np.mean(tmp["eq_width"][~mask]):.2f} AA',histtype='step',alpha=0.8)
ax.set(xlabel=r'EW(H$\alpha$)')
ax.set_title('young associations (age 1 Myr)')
ax.legend()
plt.show()

In [ ]:
import warnings
from astrotools.plot import corner

# define limits and labels for corner plots
limits   = {'age':(0,20),
            'UB':(-2.0,0),
            'CO':(0,200),
            'GMC_sep':(0,10)}

labels   = {'age':'age / Myr',
            'UB':r'$U-B$',
            'CO':r'CO flux',
            'GMC_sep':r'GMC sep / arcsec'}

                             
tmp = catalogue.copy()
#tmp = tmp[tmp['GMC_sep']<5]
print(f'sample contains {len(tmp)} objects')
with np.errstate(divide='ignore',invalid='ignore'):
    tmp['UB'] = tmp['U_dolmag_vega']-tmp['B_dolmag_vega']

columns  = ['age','GMC_sep','CO','UB']

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner(tmp,columns,function=None,
           limits=limits,labels=labels,
           filename=None,
           figsize=two_column,aspect_ratio=1,s=0.5)
plt.show()

the old nebulae with high EW might come from Wolf Rayet Stars

In [ ]:
# the old nebulae with high EW might come from Wolf Rayet Stars
WR_candidates = catalogue[(catalogue['age']>6) & (catalogue['age']<10) & (catalogue['eq_width']>80) & (catalogue['mass']>5e3)]
print(f'{len(WR_candidates)} WR candidates')
groups = WR_candidates.group_by('gal_name')
spectra = []
for key, group in zip(groups.groups.keys, groups.groups):
    filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spectra'/f'{key["gal_name"]}_VorSpectra.fits'
    with fits.open(filename) as hdul:
        tbl = Table(hdul[1].data)
        spectral_axis = np.exp(Table(hdul[2].data)['LOGLAM'])*u.Angstrom
    tbl['region_ID'] = np.arange(len(tbl))
    tbl.add_index('region_ID')
    for region_ID in group['region_ID']:
        spectra.append((key['gal_name'],region_ID,spectral_axis,tbl.loc[region_ID]['SPEC']))
        
        hdu = fits.BinTableHDU(WR_candidates[['gal_name','region_ID','assoc_ID','x_neb','y_neb','age','mass','eq_width']],
                       name='WR_candidates')
hdu.writeto(basedir/'data'/'WR_candidates.fits',overwrite=True)

from matplotlib.backends.backend_pdf import PdfPages

filename = basedir/'reports'/'WR_candidates_spectra'
nrows=6
ncols=1
width = two_column
N = len(WR_candidates)
Npage = nrows*ncols
Npages = int(np.ceil(N/Npage))
with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for i in range(Npages):
        print(f'working on page {i+1} of {Npages}')

        sub_sample = spectra[i*Npage:(i+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width*1.41))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            gal_name,region_ID,spectral_axis,spectrum = row
            ax = next(axes_iter)
            ax.plot(spectral_axis,spectrum,lw=0.5)
            ax.set(xlim=[4860,6800],yscale='log',ylim=[1e2,1e6])
            t = ax.text(0.01,0.87,f'{gal_name}: {region_ID:.0f}', transform=ax.transAxes,color='black',fontsize=8)
        plt.subplots_adjust(wspace=-0.1, hspace=0)

        # only the last page has subplots that need to be removed
        if i == int(np.ceil(N/Npage))-1:
            h,l = fig.axes[0].get_legend_handles_labels()
            ax = next(axes_iter)
            ax.axis('off')
            #ax.legend(h[::len(h)-1],l[::(len(l)-1)],fontsize=7,loc='center',frameon=False)
            #t = ax.text(0.06,0.87,'galaxy name: region ID', transform=ax.transAxes,color='black',fontsize=8)

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()


use elipses for uncertainties

In [ ]:
from matplotlib.patches import Ellipse

ellipses = [Ellipse((row['age'],row['HA/FUV']),
                     row['age_err'],row['HA/FUV_err'],
                    alpha=0.5) for row in tmp]

fig,ax=plt.subplots(figsize=(two_column,two_column/1.618))

for e in ellipses:
    ax.add_artist(e)

ax.set(xlim=(0, 10),ylim=(0,80),xlabel='age / Myr',ylabel=r'H$\alpha$ / FUV')

plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(two_column,two_column/1.618))

ax.errorbar(tmp['age'],tmp['HA/FUV'],xerr=tmp['age_err'],yerr=tmp['HA/FUV_err'],fmt='o')

ax.set(xlim=(0, 10),ylim=(0,80),xlabel='age / Myr',ylabel=r'H$\alpha$ / FUV')

plt.show()

look for correlations by comparing all columns

In [ ]:
from itertools import combinations
from scipy.stats import spearmanr

criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=8)
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria &= (catalogue['overlap_asc']==1) 
#criteria &= (catalogue['neighbors']==0)

tmp = catalogue[criteria].copy()
with np.errstate(divide='ignore',invalid='ignore'):
    tmp['log_age'] = 6+np.log10(tmp['age'])
    tmp['log_eq_width'] = np.log10(tmp['eq_width'])
    tmp['log[SIII]/[SII]'] = np.log10(tmp['[SIII]/[SII]'])
print(len(tmp))
    
columns = ['HA/FUV','eq_width','Delta_met_scal','logq_D91','age','mass',
           'EBV_balmer','EBV_stars','GMC_sep','density','temperature']

correlation = []
pairs = []
for a,b in combinations(columns,2):
    y,x = tmp[a],tmp[b] 
    not_nan = ~np.isnan(x) & ~np.isnan(y)
    r,p = spearmanr(x[not_nan],y[not_nan])
    correlation.append(r)
    pairs.append(a+'+'+b)
correlation = Table([correlation,pairs],names=['r','pair'])
correlation.sort('r')

correlation

In [ ]:
limits['EBV_balmer'] = (0,1)
limits['EBV_stars'] = (0,1)
limits['GMC_sep'] = (0,10)

corner(tmp,['EBV_balmer','EBV_stars','HA/FUV','eq_width','GMC_sep'],function=plot_function,
       limits=limits,labels=labels,
       filename=None,
       figsize=two_column,aspect_ratio=1)

### Abundance gradients

to help better understand the difference between abundances and local abundance offset

In [ ]:
from scipy.stats import binned_statistic
from astrotools.plot.utils import bin_stat
from scipy.stats import spearmanr


bins = np.logspace(-1.5,-0.2,10)
xlim = (1e-2,1e0)
cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

fig,(ax1,ax2) = plt.subplots(ncols=2,figsize=(two_column,two_column/2))

table = nebulae[nebulae['[SIII]/[SII]']>3*nebulae['[SIII]/[SII]_err']]
bins = np.logspace(*np.log10(np.nanpercentile(table['[SIII]/[SII]'],(2,98))),10)

groups = table.group_by('gal_name')

ax1.scatter(table['[SIII]/[SII]'],table['met_scal'],color='gray',s=0.1)
ax2.scatter(table['[SIII]/[SII]'],table['Delta_met_scal'],color='gray',s=0.1)

rho1,rho2 = [], []
for group in groups.groups:
    gal_name = group[0]['gal_name']

    x,mean,std = bin_stat(group['[SIII]/[SII]'],group['met_scal'],[None,None],nbins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)

    x,mean,std = bin_stat(group['[SIII]/[SII]'],group['Delta_met_scal'],[None,None],nbins=bins,statistic='median')
    ax2.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
  
    r1,p1 = spearmanr(group['[SIII]/[SII]'],group['met_scal'],nan_policy='omit')
    r2,p2 = spearmanr(group['[SIII]/[SII]'],group['Delta_met_scal'],nan_policy='omit')
    #print(f'rho = {r1:.2f}, {r2:.2f}')
    rho1.append(r1)
    rho2.append(r2)
r,p = spearmanr(table['[SIII]/[SII]'],table['met_scal'],nan_policy='omit')
t = ax1.text(0.07,0.9,r'$\rho'+f'={r:.2f}$',transform=ax1.transAxes,fontsize=7)
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

r,p = spearmanr(table['[SIII]/[SII]'],table['Delta_met_scal'],nan_policy='omit')
t = ax2.text(0.07,0.9,r'$\rho'+f'={r:.2f}$',transform=ax2.transAxes,fontsize=7)
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

ax1.set(xscale='log',xlim=[2e-2,1],ylim=[8,8.8],
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$12+\log (\mathrm{O}/\mathrm{H})$')
ax2.set(xscale='log',xlim=[2e-2,1],ylim=[-0.15,0.15],
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\Delta$(O/H)')
plt.tight_layout()

plt.show()

In [ ]:
abundance_gradients = ascii.read(basedir/'data'/'external'/'radial_abundance_gradients.txt',
                                names=['name','R0','g_r25'])
abundance_gradients.add_index('name')

$$
\Delta = 12+\log (O/H) - g
$$

In [ ]:
gal_name = 'NGC0628'
tmp = catalogue[catalogue['gal_name']==gal_name]
R0 = abundance_gradients.loc[gal_name]['R0']
g_r25 = abundance_gradients.loc[gal_name]['g_r25']

fig,ax=plt.subplots(figsize=(6,4))
ax.scatter(tmp['r_R25'],tmp['met_scal'])
r = np.linspace(0,0.5)
ax.plot(r,R0+r*g_r25,color='black')

In [ ]:
for gal_name in np.unique(catalogue['gal_name']):
    tmp = catalogue[catalogue['gal_name']==gal_name]
    plt.scatter(tmp['met_scal'],tmp['Delta_met_scal'],c=tmp['r_R25'])
    #plt.scatter(tmp['galactic_radius'],tmp['Delta_met_scal'])
    break

In [ ]:

gal_name = 'NGC0628'
tmp = catalogue[catalogue['gal_name']==gal_name]
R0 = abundance_gradients.loc[gal_name]['R0']
g_r25 = abundance_gradients.loc[gal_name]['g_r25']

fig,ax=plt.subplots(figsize=(6,4))
sc=ax.scatter(tmp['met_scal'],tmp['Delta_met_scal'],c=tmp['r_R25'])
fig.colorbar(sc,label='r / R25')
ax.set(xlabel='12+logOH',ylabel=r'$\Delta$')
plt.show()
#r = np.linspace(0,0.5)
#ax.plot(r,R0+r*g_r25,color='black')

In [ ]:

gal_name = 'NGC0628'
tmp = catalogue[catalogue['gal_name']==gal_name]
R0 = abundance_gradients.loc[gal_name]['R0']
g_r25 = abundance_gradients.loc[gal_name]['g_r25']

fig,ax=plt.subplots(figsize=(6,4))
sc=ax.scatter(tmp['logq_D91'],tmp['met_scal'],c=tmp['r_R25'])
fig.colorbar(sc,label='r / R25')
ax.set(xlabel='log q',ylabel=r'$\Delta$(O/H)')
plt.show()
#r = np.linspace(0,0.5)
#ax.plot(r,R0+r*g_r25,color='black')

In [ ]:
from cluster.auxiliary import bin_stat

fig,ax=plt.subplots(figsize=(two_column,two_column/1.618))
for gal_name in np.unique(nebulae['gal_name']):
    tmp = nebulae[nebulae['gal_name']==gal_name]
    x,mean,std = bin_stat(tmp['logq_D91'],tmp['Delta_met_scal'],[5,8.5],nbins=10)
    ax.errorbar(x,mean,yerr=std,fmt='-',label=gal_name)
    #ax.scatter(tmp['logq_D91'],tmp['Delta_met_scal'],label=gal_name)
ax.set(xlim=[5,8.5],ylim=[-0.2,0.2],xscale='linear')
#ax.legend()
plt.show()

### Age and Ha/FUV

In [ ]:
from cluster.auxiliary import bin_stat

criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=10) 
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria &= (catalogue['overlap_asc']==1) 
#tmp = catalogue[criteria].copy()
print(f'sample contains {len(tmp)} objects')


xlim = [0.5,10.5]
fig,ax=plt.subplots(figsize=(4,4/1.618))
sc = ax.scatter(tmp['age'],tmp['HA/FUV'],s=10,alpha=0.8,vmin=0,vmax=1)

x,mean,std = bin_stat(tmp['age'],tmp['HA/FUV'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='Age/Myr',ylabel=r'H$\alpha$ / FUV',xlim=xlim,ylim=[0,100])

#fig.colorbar(sc,label=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$')
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

#plt.savefig(basedir/'reports'/f'all_galaxies_HaFUV_over_age.pdf',dpi=600)
plt.show()

In [ ]:

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(8,3))

tmp = catalogue[(catalogue['mass']>1e3) & (catalogue['overlap']=='contained')]
ax1.scatter(tmp['dig/hii'],tmp['HA/FUV'],c=tmp['age'],vmin=0,vmax=20,s=2)

tmp = catalogue[(catalogue['mass']>1e3) & (catalogue['overlap']=='partial')]
sc=ax2.scatter(tmp['dig/hii'],tmp['HA/FUV'],c=tmp['age'],vmin=0,vmax=20,s=2)

ax1.set(xlim=[0,1],ylim=[0,100],xlabel=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$',ylabel=r'H$\alpha$/FUV')
ax2.set(xlim=[0,1],ylim=[0,100],xlabel=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$',ylabel=r'H$\alpha$/FUV')

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.93, 0.11, 0.02, 0.84])
fig.colorbar(sc,cax=cbar_ax,label='age / Myr')

plt.show()

seperate subplot for each galaxy

In [ ]:
from astropy.coordinates import match_coordinates_sky, search_around_sky
from scipy.stats import binned_statistic, pearsonr, spearmanr

# '[SIII]/[SII]' , 'HA/FUV', 'AGE_MINCHISQ', 'AGE_BAYES'
x_name, y_name = 'age', 'HA/FUV'
xlim = [0.5,10.5]
bins = 10
max_sep = 1*u.arcsec

#sample = set(np.unique(nebulae['gal_name'])) & hst_sample
sample = np.unique(catalogue['gal_name'])[:-1]

filename = basedir/'reports'/'all_objects_age_over_SII.pdf'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 3
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

# loop over the galaxies we want to plot
for name in sorted(sample): 

    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    criteria = (catalogue['gal_name']==name) 
    criteria &= (catalogue['mass']>1e3) 
    #criteria &= ~np.isnan(catalogue['HA/FUV'])
    criteria &= (catalogue['overlap'] == 'contained')
    #criteria &= (catalogue['neighbors'] == 0)
    
    tmp = catalogue[criteria]
    print(f'{name}: {len(tmp)} objects')
    
    mean, bin_edges, binnumber = binned_statistic(tmp[x_name],
                                                  tmp[y_name],
                                                  statistic='mean',
                                                  bins=bins,
                                                  range=xlim)
    std, _, _ = binned_statistic(tmp[x_name],
                                  tmp[y_name],
                                  statistic='std',
                                  bins=bins,
                                  range=xlim)

    ax.scatter(tmp[x_name],tmp[y_name],color='tab:blue',s=1)
    # plot the standard divation with yerr=std
    ax.errorbar((bin_edges[1:]+bin_edges[:-1])/2,mean,fmt='-')
    ax.text(0.65,0.85,f'{name}', transform=ax.transAxes,fontsize=7)

    ax.set(xlim=xlim)
    if i==nrows-1:
        ax.set_xlabel(f'{x_name.replace("_"," ")}')
    if j==0:
        ax.set_ylabel(f'{y_name.replace("_"," ")}')

for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(f'{x_name.replace("_"," ")}')


#plt.savefig(filename,dpi=600)
plt.show()

In [ ]:
from astropy.coordinates import match_coordinates_sky, search_around_sky

# '[SIII]/[SII]' , 'HA/FUV'
x_name, y_name, z_name = '[SIII]/[SII]', 'HA/FUV', 'AGE_MINCHISQ'
xlim = [0.5,10.5]
bins = 10
max_sep = 2*u.arcsec

sample = muse_sample & hst_sample & astrosat_sample

filename = basedir/'reports'/'all_objects_FUV_over_SII_with_age.pdf'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 2
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

# loop over the galaxies we want to plot
for name in sorted(sample): 

    # it makes a different if we match the clusters to the nebulae or the other way around
    catalogcoord = clusters[clusters['gal_name']==name].copy()
    matchcoord   = nebulae[nebulae['gal_name']==name].copy()

    idx, sep, _ = match_coordinates_sky(matchcoord['SkyCoord'],catalogcoord['SkyCoord'])

    catalogue = matchcoord.copy()

    for col in catalogcoord.columns:
        if col in catalogue.columns:
            catalogue[f'{col}2'] = catalogcoord[idx][col]
        else:
            catalogue[col] = catalogcoord[idx][col]

    catalogue['sep'] = sep
    catalogue = catalogue[sep.__lt__(max_sep)]
    catalogue = catalogue[~np.isnan(catalogue[x_name]) & ~np.isnan(catalogue[y_name]) & (catalogue['AGE_MINCHISQ']<10)]
    print(f'{name}: {len(catalogue)} objects in joined catalogue')

    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    #catalogue = catalogue[catalogue['HA6562_FLUX']>np.nanpercentile(catalogue['HA6562_FLUX'],50)]
    #catalogue = catalogue[catalogue['FUV_FLUX_CORR']>3*catalogue['FUV_FLUX_CORR_ERR']]
    #catalogue = catalogue[catalogue['SII6716_FLUX_CORR']>3*catalogue['SII6716_FLUX_CORR_ERR']]
    #catalogue = catalogue[catalogue['SIII9068_FLUX_CORR']>3*catalogue['SIII9068_FLUX_CORR_ERR']]

    r,p = spearmanr(catalogue['[SIII]/[SII]'],catalogue['HA/FUV'])
    print(f'{name}: rho={r:.2f}, {len(catalogue)} objects')

    sc = ax.scatter(catalogue['[SIII]/[SII]'],catalogue['HA/FUV'],
                    c=catalogue[z_name],vmin=0, vmax=10,cmap=plt.cm.RdBu_r,
                    s=3,marker='.')
    
    ax.text(0.05,0.9,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.7,0.15,r'$\rho$'+f'={r:.2f}',transform=ax.transAxes,fontsize=7)
    ax.text(0.55,0.05,f'{len(catalogue):.0f} objects', transform=ax.transAxes,fontsize=7)
    
    ax.set(xscale='log',yscale='log',xlim=[1e-2,1],ylim=[2,2e2])
    # https://stackoverflow.com/questions/21920233/matplotlib-log-scale-tick-label-number-formatting/33213196
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    if i==nrows-1:
        ax.set_xlabel('[SIII]/[SII]')
    if j==0:
        ax.set_ylabel(r'H$\alpha$ / FUV')

fig.colorbar(sc,ax=axes.ravel().tolist(),label=f'{z_name.replace("_"," ")}')
        
for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(f'{x_name.replace("_"," ")}')


plt.savefig(filename,dpi=600)
plt.show()


### Catalogue statistics

In [ ]:
fig,(ax1,ax2,ax3,ax4)=plt.subplots(nrows=4,figsize=(single_column,single_column*1.5))

ax1.hist(catalogue['mass'],bins=np.logspace(2,5.5,15))
ax1.axvline(1e4,color='black')
ax1.set(xscale='log',xlabel='mass / Msun',xlim=[1e2,10**5.5])

ax2.hist(catalogue['age'],bins=np.logspace(0,2,11))
ax2.axvline(8,color='black')
ax2.set(xscale='log',xlabel='age / Myr',xlim=[1,100])

ax3.hist(100*catalogue['overlap_asc'],bins=np.linspace(0,100,11))
ax3.axvline(90,color='black')
ax3.set(xlabel='overlap percentage associations',xlim=[0,100])

ax4.hist(100*catalogue['overlap_neb'],bins=np.linspace(0,100,11))
#ax4.axvline(95,color='black')
ax4.set(xlabel='overlap percentage nebulae',xlim=[0,100])

plt.tight_layout()

#plt.savefig(basedir/'reports'/'tmp_catalogue_properties.png',dpi=300)
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))
xlim = [1e2,1e5]
ylim = [1,100]

x,y = catalogue['mass'], catalogue['age']
nbins=11
bins = [np.logspace(*np.log10(xlim),16),np.logspace(*np.log10(ylim),9)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=False)
im = ax.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.Reds)
ax.plot([1e4,1e4,1e5],[1,8,8],color='black',lw=2)

ax.set(xlim=xlim,xscale='log',xlabel=r'mass / M$_\odot$',
       ylim=ylim,yscale='log',ylabel='age / Myr')
fig.colorbar(im,label='count')
plt.savefig(basedir/'reports'/'tmp_catalogue_properties_2D_hist.png',dpi=300)
plt.show()

In [ ]:
fig=plt.figure(figsize=(single_column,single_column/1.618))

gs = fig.add_gridspec(2, 2,  width_ratios=(7, 2), height_ratios=(2, 7),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.03, hspace=0.05)

ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)

ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)


xlim = [63.095734446471226,1.5848931925e5]
ylim = [1,39.81071706] # based on 10 with 9 bins
ylim = [1,38.05457130] # based on 8 with 8 bins
x,y = catalogue['mass'], catalogue['age']
nbins=11
bins = [np.logspace(*np.log10(xlim),18),np.logspace(*np.log10(ylim),8)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=False)
im = ax.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.Reds,zorder=0)
ax.plot([1e4,1e4,xlim[1]],[1,8,8],color='black',lw=2)

ax_histx.hist(x,bins[0],color='#af1d1d',histtype='step')
ax_histy.hist(y,bins[1],orientation='horizontal',color='#af1d1d',histtype='step')

ax_histx.set_yticks([0,100,200,300,400,500],minor=True)
ax_histy.set_xticks([0,250,500,750,1000],minor=True)
ax_histx.set(ylim=[0,600])

ax.set(xlim=xlim,xscale='log',xlabel=r'mass / M$_\odot$',
       ylim=ylim,yscale='log',ylabel='age / Myr')
ax.set_yticks([1,10])
ax.set_yticklabels([1,10])
fig.colorbar(im,label='count',ax=ax_histy)
plt.savefig(basedir/'reports'/'catalogue_properties_2D_hist.pdf',dpi=300)

plt.show()

## HII regions without 1to1 match


### Multiple associations in one nebula

so far we restricted ourselves to HII regions and associations that show a one-to-one relation. This reduces the sample to 4062 HII regions. However we saw that over 8000 HII regions overlap with an associations, so there is a lot of potential. 

In [ ]:
fig,(ax1,ax2) = plt.subplots(ncols=2,figsize=(two_column,two_column/2))

ax1.hist(nebulae['Nassoc'],bins=np.arange(.5,6.5))
ax1.set_title('Number of associations per nebulae')

ax2.hist(associations['Nnebulae'],bins=np.arange(0.5,6.5))
ax2.set_title('Number of nebulae per associations')

plt.show()

for our analysis we limit ourselves to the associations that show a one-to-one relation. However including HII regions that overlap with multiple associations has the potential to increase the sample size by 50%. 
However it is difficult to assess the contribution of multiple associations. We first look at the age. According to stellar models, the ionising photon flux stays almost constant for the first few Myr (<5Myr) and than quickly drops. By the time the cluster is 10 Myr old, it has dropped to less than 1% of its initial value. By the time the cluster is 20 Myr old, the ionising photon flux has dropped to 1/6000 of its initial value. 

In [ ]:
form tqdm import tqdm 

candidates = []

for gal_name in tqdm(np.unique(associations['gal_name'])):

    HIIregion_mask = (nebulae['BPT_NII']==0) & (nebulae['BPT_SII']==0) & (nebulae['BPT_OI']==0)
    neb_tmp = nebulae[(nebulae['gal_name']==gal_name) & HIIregion_mask]
    assoc_tmp = associations[associations['gal_name']==gal_name].copy()
    assoc_tmp.add_index('assoc_ID')

    with open(basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'/f'{gal_name}_{HSTband}_{scalepc}pc_nebulae.yml') as f:
        nebulae_dict = yaml.load(f,Loader=yaml.SafeLoader)
    with open(basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'/f'{gal_name}_{HSTband}_{scalepc}pc_associations.yml') as f:
        associations_dict = yaml.load(f,Loader=yaml.SafeLoader)


    # we select each nebulae with more than one HII region
    for region_ID in neb_tmp[neb_tmp['Nassoc']>1]['region_ID']:
        sub = assoc_tmp.loc[nebulae_dict[region_ID]]
        # first we check if we can ignore the other associations in the nebulae
        if np.sum((sub['age']<10) & (sub['mass']>1e4)) == 1:
            # then we make sure that the one associations also overlaps sufficiently
            if np.sum((sub['age']<10) & (sub['mass']>1e4) & (sub['overlap_asc']>0.5)) == 1:
                candidates.append(region_ID)
        
print(f'{len(candidates)} of {np.sum(nebulae_tmp["Nassoc"]>1)} match the criteria')

In [ ]:
from imf import make_cluster

cluster = make_cluster(5e3)
cluster.sort()
cluster = cluster[::-1]

### Isolated HII regions

#### Compare with DOLPHONT peaks

the following code goes through each compact cluster and DOLPHOT peak and determines if they fall inside an HII region. The `region_ID` is recorded (nan if outside) and the result is saved in a new catalogue file.

In [ ]:
def get_value(matrix, index, default_value=np.nan):
    '''
    The `to_pixel` method returns the x,y coordinates. However in the 
    image they correspond to img[y,x]
    '''
    result = np.zeros(len(index))+default_value
    mask = (index[:,1] < matrix.shape[0]) & (index[:,0] < matrix.shape[1])
    mask &= (index[:,1] >= 0) & (index[:,0] >=0)

    valid = index[mask]
    result[mask] = matrix[valid[:,1], valid[:,0]]
    return result

isolated = nebulae[nebulae['overlap_neb']==0]

compact_clusters_list = []
dolphot_peaks_list = []

for gal_name in np.unique(sample_table['name']):

    # we are only interested in peaks that overlap with isolated HII regions
    sub_nebulae = nebulae[(nebulae['gal_name']==gal_name) & (nebulae['in_frame'])] # & (nebulae['overlap_neb']==0)]
    
    print(f'working on {gal_name}')
    
    # read the nebula mask
    filename = next((data_ext/'MUSE'/'DR2.1'/'copt'/'MUSEDAP').glob(f'{gal_name}*.fits'))
    copt_res = float(filename.stem.split('-')[1].split('asec')[0])
    with fits.open(filename) as hdul:
        Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                        mask=np.isnan(hdul['HA6562_FLUX'].data),
                        meta=hdul['HA6562_FLUX'].header,
                        wcs=WCS(hdul['HA6562_FLUX'].header))

    filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
    with fits.open(filename) as hdul:
        nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
        nebulae_mask.data[nebulae_mask.data==-1] = np.nan
    
    # we only need the wcs to convert from pixel to skycoord
    filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
    if not filename.is_file():
        print(f'no F275W image for {gal_name}')
        continue
    with fits.open(filename) as hdul:
        #F275 = NDData(hdul[0].data,
        #              mask=hdul[0].data==0,
        #              meta=hdul[0].header,
        #              wcs=WCS(hdul[0].header))
        wcs_F275=WCS(hdul[0].header)
    
    # read the compact cluster and DOLPHOT catalogues
    filename = data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_ml_class12.fits'
    if not filename.is_file():
        print(f'no compact clusters for {gal_name}')
    else:     
        with fits.open(filename) as hdul:
            compact_clusters = Table(hdul[1].data)
        compact_clusters['SkyCoord'] = SkyCoord(compact_clusters['PHANGS_RA']*u.deg,compact_clusters['PHANGS_DEC']*u.deg)
        compact_clusters.add_index('ID_PHANGS_CLUSTERS')

        compact_clusters['region_ID'] = get_value(nebulae_mask.data,np.array(compact_clusters['SkyCoord'].to_pixel(nebulae_mask.wcs)).T.astype(int))

        # we save all objects that fall inside one of our empty HII regions
        compact_clusters.add_column(gal_name,index=0,name='gal_name')
        del compact_clusters['SkyCoord']
        compact_clusters_list.append(compact_clusters[np.isin(compact_clusters["region_ID"],sub_nebulae["region_ID"])])
        #compact_clusters_list.append(compact_clusters)

    # and now for the peak catalogue
    filename = data_ext/'Products'/'stellar_associations'/'DOLPHOT'/f'{gal_name.lower()}_uvis_dolphot.fits'
    if not filename.is_file():
        print(f'no DOLPHOT catalogue for {gal_name}')
        continue  
    with fits.open(filename) as hdul:
        dolphot_peaks = Table(hdul[1].data)
    dolphot_peaks.add_index('Dolphot_ID')
    dolphot_peaks['SkyCoord'] = SkyCoord.from_pixel(dolphot_peaks['Dolphot_x'],dolphot_peaks['Dolphot_y'],wcs=wcs_F275)
    dolphot_peaks['region_ID'] = get_value(nebulae_mask.data,np.array(dolphot_peaks['SkyCoord'].to_pixel(nebulae_mask.wcs)).T.astype(int))

    dolphot_peaks.add_column(gal_name,index=0,name='gal_name')
    dolphot_peaks.add_column(dolphot_peaks['SkyCoord'].ra.deg,index=3,name='RA')
    dolphot_peaks.add_column(dolphot_peaks['SkyCoord'].dec.deg,index=4,name='DEC')
    del dolphot_peaks['SkyCoord']
    dolphot_peaks_list.append(dolphot_peaks[np.isin(dolphot_peaks["region_ID"],sub_nebulae["region_ID"])])

compact_clusters = vstack(compact_clusters_list)
dolphot_peaks = vstack(dolphot_peaks_list)

primary_hdu = fits.PrimaryHDU()
table_hdu   = fits.BinTableHDU(compact_clusters)
hdul = fits.HDUList([primary_hdu, table_hdu])
hdul.writeto(basedir/'data'/'interim'/f'compact_clusters_in_isolated_HIIregions.fits',overwrite=True)

primary_hdu = fits.PrimaryHDU()
table_hdu   = fits.BinTableHDU(dolphot_peaks)
hdul = fits.HDUList([primary_hdu, table_hdu])
hdul.writeto(basedir/'data'/'interim'/f'dolphot_peaks_in_isolated_HIIregions.fits',overwrite=True)

the result from the previous step is saved in a file

In [ ]:
dolphot_peaks = Table(fits.getdata(basedir/'data'/'interim'/f'dolphot_peaks_in_isolated_HIIregions.fits'))
compact_clusters = Table(fits.getdata(basedir/'data'/'interim'/f'compact_clusters_in_isolated_HIIregions.fits'))
compact_clusters['SkyCoord'] = SkyCoord(compact_clusters['PHANGS_RA']*u.deg,compact_clusters['PHANGS_DEC']*u.deg)
dolphot_peaks['SkyCoord'] = SkyCoord(dolphot_peaks['RA']*u.deg,dolphot_peaks['DEC']*u.deg)
dolphot_peaks['S2N_WFC3_F555W'][dolphot_peaks['gal_name']=='NGC0628'] = dolphot_peaks['S2N_WFC3_F555W'][dolphot_peaks['gal_name']=='NGC0628'] = dolphot_peaks['S2N_ACS_F555W'][dolphot_peaks['gal_name']=='NGC0628']

S2N = 5
dolphot_peaks['detection'] = np.sum([(dolphot_peaks[f'S2N_WFC3_F{band}W']>S2N) for band in [275,336,438,555,814]],axis=0)

record how many peaks fall in each HII region

In [ ]:
nebulae['Ncluster'] = False
for gal_name in sample_table['name']:  
    nebulae['Ncluster'][nebulae['gal_name']==gal_name] = np.isin(nebulae[nebulae['gal_name']==gal_name]['region_ID'],compact_clusters[compact_clusters['gal_name']==gal_name]['region_ID'])

# there is no nebulae catalogue for NGC1300 and NGC4254
criteria = nebulae['in_frame'] & np.isin(nebulae['gal_name'],np.unique(compact_clusters['gal_name']))
N_isolated = np.sum(criteria)
print(f"{np.sum(nebulae['Ncluster'])} ({100*np.sum(nebulae['Ncluster']) / N_isolated:.1f} %) have a compact cluster")    

nebulae['Ndolphot'] = np.nan
for gal_name in sample_table['name']:
    tmp = dolphot_peaks[(dolphot_peaks['S2N_WFC3_F275W']>S2N) &         # detection in NUV
                        (dolphot_peaks['detection']>=2) &               # detection in at least one other band
                        (dolphot_peaks['gal_name']==gal_name)]    
    nebulae['Ndolphot'][nebulae['gal_name']==gal_name] = [np.sum(tmp['region_ID']==region_ID) for region_ID in nebulae[nebulae['gal_name']==gal_name]['region_ID']]

# same criteria as above but also with no clusters
criteria = (nebulae['overlap_neb']==0) & (~nebulae['Ncluster']) & nebulae['in_frame'] & np.isin(nebulae['gal_name'],np.unique(compact_clusters['gal_name']))
N_isolated = np.sum(criteria)
N_dolphot = np.sum((nebulae['Ndolphot']>0) & criteria)
print(f"{N_dolphot} ({100*N_dolphot / N_isolated:.1f} %) have a DOLPHOT peak")    

In [ ]:
fig,ax=plt.subplots(figsize=(two_column,two_column/1.618))

bins = np.logspace(35,40,20)

gal_name = 'NGC2835'
sub = nebulae #[(nebulae['gal_name']==gal_name)]


tmp = sub[(sub['overlap_neb']>0) & (sub['in_frame'])]
label = f"stellar association: {np.mean(tmp['HA6562_LUM_CORR']):.2g} erg s-1"
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,histtype='step',alpha=0.6,label=label)
print(label)

tmp = sub[(sub['overlap_neb']==0) & (sub['in_frame']) & (sub['Ncluster']>0)]
label = f"Compact cluster: {np.mean(tmp['HA6562_LUM_CORR']):.2g} erg s-1"
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,histtype='step',alpha=0.6,label=label)
print(label)

tmp = sub[(sub['overlap_neb']==0) & (sub['in_frame']) & (sub['Ncluster']==0) & (sub['Ndolphot']>0)]
label = f"DOLPHOT: {np.mean(tmp['HA6562_LUM_CORR']):.2g} erg s-1"
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,histtype='step',alpha=0.6,label=label)
print(label)


tmp = sub[(sub['overlap_neb']==0) & (sub['in_frame']) & (sub['Ndolphot']==0)]
label = f"isolated: {np.mean(tmp['HA6562_LUM_CORR']):.2g} erg s-1"
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,histtype='step',alpha=0.6,label=label)
print(label)

ax.legend()

ax.set(xscale='log',xlim=[2e35,9e39],xlabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$',
       yscale='linear',ylim=[1,None],ylabel='N')
plt.savefig(basedir/'reports'/'Ha_luminosity_function_isolated_vs_matched.png',dpi=300)
plt.show()


create a joined HII region and dolphot peak catalogue for single peaks

In [ ]:
print(f'{len(dolphot_peaks)} peaks in initial catalogue')
lst = []
S2N = 10
for gal_name in sample_table['name']:
    tmp = dolphot_peaks[dolphot_peaks['gal_name']==gal_name]
    tmp = tmp[(tmp['S2N_WFC3_F275W']>S2N) & (tmp['S2N_WFC3_F555W']>S2N)]
    ids, counts = np.unique(tmp['region_ID'],return_counts=True)
    ids = ids[counts==1]
    tmp = tmp[np.isin(tmp['region_ID'],ids)]
    HII_sub = nebulae[(nebulae['gal_name']==gal_name)]
    del HII_sub['SkyCoord_neb']
    
    if len(tmp)>0:
        tmp = join(tmp,HII_sub,keys=['gal_name','region_ID'])
        tmp['MV'] = tmp['Mag_WFC3_F555W'] - Distance(tmp['distance']*u.Mpc).distmod.value
        lst.append(tmp)

single_peaks = vstack(lst)
single_peaks['SkyCoord'] = SkyCoord(single_peaks['RA']*u.deg,single_peaks['DEC']*u.deg)
print(f'{len(single_peaks)} single peaks in final catalogue')

show color-color-diagram and cmd

In [ ]:
points = np.array([[-0.33 , -1.662],[-0.307, -1.577],[-0.25 , -1.468],[-0.218, -1.394],[-0.186, -1.321],
                   [-0.138, -1.235],[-0.076, -1.213],[-0.007, -1.206],[ 0.065, -1.197],[ 0.135, -1.187],
                   [ 0.203, -1.198],[ 0.276, -1.201],[ 0.347, -1.209],[ 0.418, -1.219],[ 0.488, -1.233],
                   [ 0.559, -1.251],[ 0.63 , -1.27 ],[ 0.706, -1.264],[ 0.771, -1.287],[ 0.648, -1.119],
                   [ 0.626, -1.026],[ 0.565, -0.989],[ 0.559, -0.87 ],[ 0.502, -0.807],[ 0.481, -0.706],
                   [ 0.457, -0.546],[ 0.505, -0.449],[ 0.487, -0.384],[ 0.488, -0.328],[ 0.468, -0.239],
                   [ 0.517, -0.109],[ 0.555, -0.015],[ 0.6  ,  0.041],[ 0.65 ,  0.132],[ 0.724,  0.174],
                   [ 0.864,  0.169],[ 0.948,  0.166],[ 1.02 ,  0.152],[ 1.138,  0.182],[ 1.203,  0.194],
                   [ 1.253,  0.282],[ 1.331,  0.378],[ 1.354,  0.469]])

x,y=points.T

fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/3))

S2N = 10
criteria  =  single_peaks['S2N_WFC3_F275W']>S2N
for band in [275,336,438,555,814]:
    criteria &= single_peaks[f'S2N_WFC3_F{band}W']>S2N
#criteria &= single_peaks['HA6562_LUM_CORR']<1.5e37

tmp = single_peaks[criteria]
print(f'{len(tmp)} objects in sample')

U = tmp['Mag_WFC3_F336W']
B = tmp['Mag_WFC3_F438W']
V = tmp['Mag_WFC3_F555W']
I = tmp['Mag_WFC3_F814W']

ax1.scatter(V-I,U-B)
ax1.plot(x,y,color='black')
ax1.set(xlim=[-1,2],ylim=[-2.5,1],
      ylabel=r'F336W$-$F438W',xlabel=r'F555W$-$F814W')
ax1.invert_yaxis()

ax2.scatter(B-V,V)
ax2.set(xlim=[-1,2.5],ylim=[18,28],
      ylabel=r'F555W',xlabel=r'F438W$-$F555W')
ax2.invert_yaxis()

bins = np.logspace(35,39,20)
ax3.hist(tmp['HA6562_LUM_CORR'],bins=bins)
ax3.set(xscale='log',xlabel=r'H$\alpha$ / erg s$^{-1}$')

plt.tight_layout()
plt.show()

be careful. For NGC0628, the S/N for some filters is weird (F275W and F336W look fine)

In [ ]:
n1 = 0  # isolated
n2 = 0  # cluster
n3 = 0  # dolphot peaks
S2N = 3
for gal_name in np.unique(sample_table['name']):
    
    sub_nebulae = nebulae[(nebulae['gal_name']==gal_name)  & (nebulae['in_frame'])]
    sub_dolphot_peaks = dolphot_peaks[(dolphot_peaks['gal_name']==gal_name) & (dolphot_peaks['S2N_WFC3_F275W']>S2N) & (dolphot_peaks['S2N_WFC3_F555W']>S2N)]
    #sub_dolphot_peaks = dolphot_peaks[(dolphot_peaks['gal_name']==gal_name) & (dolphot_peaks['Mag_WFC3_F275W']<25)] # & (dolphot_peaks['Mag_WFC3_F555W']<25)]
    sub_compact_clusters = compact_clusters[compact_clusters['gal_name']==gal_name]
    
    if len(sub_dolphot_peaks)==0:
        continue
    
    isolated_nebulae = sub_nebulae[(sub_nebulae['overlap_neb']==0)]
    n_overlap_with_cluster = np.sum(np.isin(isolated_nebulae['region_ID'],sub_compact_clusters['region_ID']))
    n_overlap_with_peak = np.sum(np.isin(isolated_nebulae['region_ID'],sub_dolphot_peaks['region_ID']))
       
    n1+=len(isolated_nebulae)
    n2+=n_overlap_with_cluster
    n3+=n_overlap_with_peak
    print(f'{gal_name:>8}: {len(isolated_nebulae):>5} isolated HII regions. {n_overlap_with_cluster:>3} ({100*n_overlap_with_cluster/len(isolated_nebulae):>4.1f}%), {n_overlap_with_peak:>3} ({100*n_overlap_with_peak/len(isolated_nebulae):>4.1f}%)')
        
print(f'{"all":>8}: {n1:>5} isolated HII regions. {n2:>3} ({100*n2/n1:>4.1f}%), {n3:>3} ({100*n3/n1:>4.1f}%)')

#### Account for associations of different scales

In [ ]:
# the nebulae catalogue with additional information
folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'16pc'
lst = []
for file in folder.glob(f'*16pc_nebulae.fits'):
    gal_name = file.stem.split('_')[0]
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    lst.append(tbl)
nebulae_tmp16 = vstack(lst)
nebulae_tmp16.rename_column('overlap_neb','overlap_16pc')
folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'32pc'
lst = []
for file in folder.glob(f'*32pc_nebulae.fits'):
    gal_name = file.stem.split('_')[0]
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    lst.append(tbl)
nebulae_tmp32 = vstack(lst)
nebulae_tmp32.rename_column('overlap_neb','overlap_32pc')
folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'64pc'
lst = []
for file in folder.glob(f'*64pc_nebulae.fits'):
    gal_name = file.stem.split('_')[0]
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    lst.append(tbl)
nebulae_tmp64 = vstack(lst)
nebulae_tmp64.rename_column('overlap_neb','overlap_64pc')

multiscale_nebulae = join(nebulae_tmp16[['gal_name','region_ID','overlap_16pc']],nebulae_tmp32[['gal_name','region_ID','overlap_32pc']],keys=['gal_name','region_ID'])
multiscale_nebulae = join(multiscale_nebulae,nebulae_tmp64[['gal_name','region_ID','overlap_64pc']],keys=['gal_name','region_ID'])
multiscale_nebulae = join(multiscale_nebulae,nebulae[['gal_name','region_ID','in_frame','SkyCoord_neb']],keys=['gal_name','region_ID'])

np.sum(multiscale_nebulae['in_frame'] & (multiscale_nebulae['overlap_16pc']==0) & (multiscale_nebulae['overlap_32pc']==0) & (multiscale_nebulae['overlap_64pc']==0))

multiscale_nebulae['overlap_any'] = (multiscale_nebulae['overlap_16pc']>0) | (multiscale_nebulae['overlap_32pc']>0) | (multiscale_nebulae['overlap_64pc']>0)

# there is no nebulae catalogue for NGC1300 and NGC4254
N_isolated = np.sum(multiscale_nebulae['in_frame'] & (multiscale_nebulae['overlap_32pc']==0))
N_multi = np.sum(multiscale_nebulae['overlap_any'] & (multiscale_nebulae['overlap_32pc']==0))

print(f'{N_isolated} HII regions do not overlap with association')
print(f"{N_multi} ({100*N_multi / N_isolated:.1f} %) have association of different scale")    

multiscale_nebulae['Ncluster'] = False
for gal_name in sample_table['name']:  
    multiscale_nebulae['Ncluster'][multiscale_nebulae['gal_name']==gal_name] = np.isin(multiscale_nebulae[multiscale_nebulae['gal_name']==gal_name]['region_ID'],compact_clusters[compact_clusters['gal_name']==gal_name]['region_ID'])

# there is no nebulae catalogue for NGC1300 and NGC4254
criteria = ~multiscale_nebulae['overlap_any'] & multiscale_nebulae['in_frame'] & np.isin(multiscale_nebulae['gal_name'],np.unique(compact_clusters['gal_name']))
N_isolated = np.sum(criteria)
N_cluster = np.sum(multiscale_nebulae['Ncluster'] & ~multiscale_nebulae['overlap_any'])
print(f"{N_cluster} ({100*N_cluster / N_isolated:.1f} %) have a compact cluster")    

multiscale_nebulae['Ndolphot'] = np.nan
for gal_name in sample_table['name']:
    tmp = dolphot_peaks[(dolphot_peaks['S2N_WFC3_F275W']>S2N) &         # detection in NUV
                        (dolphot_peaks['detection']>=2) &               # detection in at least one other band
                        (dolphot_peaks['gal_name']==gal_name)]    
    multiscale_nebulae['Ndolphot'][multiscale_nebulae['gal_name']==gal_name] = [np.sum(tmp['region_ID']==region_ID) for region_ID in multiscale_nebulae[multiscale_nebulae['gal_name']==gal_name]['region_ID']]

# same criteria as above but also with no clusters
criteria = ~multiscale_nebulae['overlap_any'] & (~multiscale_nebulae['Ncluster']) & multiscale_nebulae['in_frame'] & np.isin(multiscale_nebulae['gal_name'],np.unique(compact_clusters['gal_name']))
N_isolated = np.sum(criteria)
N_dolphot = np.sum((multiscale_nebulae['Ndolphot']>0) & criteria)
print(f"{N_dolphot} ({100*N_dolphot / N_isolated:.1f} %) have a DOLPHOT peak")    

N_truley_isolated = np.sum(~multiscale_nebulae['overlap_any'] & ~multiscale_nebulae['Ncluster'] & multiscale_nebulae['in_frame'] & (multiscale_nebulae['Ndolphot']==0))
N_tot = np.sum(multiscale_nebulae['in_frame'])
print(f"{100*N_truley_isolated/N_tot:.2f}% of the HII regions are truely isolated")


In [ ]:
np.sum(~multiscale_nebulae['overlap_any'] & ~multiscale_nebulae['Ncluster'] & multiscale_nebulae['in_frame']) / np.sum(multiscale_nebulae['in_frame'])

cutouts for isolated objects

In [ ]:
from cluster.io import read_associations
from matplotlib.backends.backend_pdf import PdfPages
from cluster.plot import single_cutout

S2N = 10

#tmp = compact_clusters[(compact_clusters['PHANGS_MASS_MINCHISQ']>1e4) & (compact_clusters['PHANGS_AGE_MINCHISQ']<10)]
#criteria =  single_peaks['S2N_WFC3_F275W']>S2N
#criteria &= single_peaks['S2N_WFC3_F555W']>S2N
#criteria &= single_peaks['Mag_WFC3_F336W']-single_peaks['Mag_WFC3_F438W']<-1
#tmp = single_peaks[criteria]

tmp = nebulae[(nebulae['Ndolphot']==0) & (nebulae['overlap_neb']==0) & (nebulae['in_frame'])]
tmp = nebulae[(nebulae['Ndolphot']==0) & (nebulae['overlap_neb']==0) & (nebulae['in_frame']) & 
              (nebulae['gal_name']=='NGC1672')]
tmp = nebulae_with_single_peak[:50]

print(f'{len(tmp)} objects in sample')

size=8*u.arcsec
nrows=5
ncols=4
filename = basedir/'reports'/f'nebulae_with_single_peak'
    
width = 8.27
N = len(tmp) 
Npage = nrows*ncols-1
if N%Npage==0:
    print('sample size % subplots = 0: no subplot for legend')
Npages = int(np.ceil(N/Npage))
gal_name = None

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for i in range(Npages):
        print(f'working on page {i+1} of {Npages}')

        sub_sample = tmp[i*Npage:(i+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            
            # for a new galaxy we need to read in the masks/images
            if row['gal_name'] != gal_name:
                
                gal_name = row['gal_name']
                
                # HST image for the background
                filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
                with fits.open(filename) as hdul:
                    F275 = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                    
                '''
                # HST image for the background
                filename = data_ext / 'HST' / 'narrowband_Halpha' / f'{gal_name}_ha_sub.fits'
                with fits.open(filename) as hdul:
                    hst_halpha = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                '''
                
                filename = data_ext / 'MUSE' / 'DR2.1' / 'MUSEDAP' / f'{gal_name}_MAPS.fits'
                with fits.open(filename) as hdul:
                    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                                    meta=hdul['HA6562_FLUX'].header,
                                    wcs=WCS(hdul['HA6562_FLUX'].header))
            
                # nebulae mask
                filename = data_ext / 'Products' / 'Nebulae_catalogs' / 'Nebulae_catalogue_v2' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
                with fits.open(filename) as hdul:
                    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
                    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
                
                # association mask
                associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',
                                                      target=gal_name.lower(),
                                                      scalepc=scalepc,
                                                      data='mask')

            
            ax = next(axes_iter)
            ax = single_cutout(ax,
                             position = row['SkyCoord_neb'],
                             image = F275,
                             mask1 = nebulae_mask,
                             mask2 = associations_mask,
                             points = single_peaks,
                             label = f"{row['gal_name']}: {row['region_ID']:.0f}",
                             size  = size)

        plt.subplots_adjust(wspace=-0.01, hspace=0.05)

        # only the last page has subplots that need to be removed
        h,l = fig.axes[0].get_legend_handles_labels()
        ax = next(axes_iter)
        ax.axis('off')
        ax.legend(h[::len(h)-1],l[::(len(l)-1)],fontsize=7,loc='center',frameon=False)
        t = ax.text(0.07,0.87,'name: region ID/assoc ID', transform=ax.transAxes,color='black',fontsize=8)

        if i == int(np.ceil(N/Npage))-1:

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()


#### Truly isolated HII regions

are the HII regions without a peak more extincted?

In [ ]:
fig,ax=plt.subplots()

bins = np.linspace(0,1,10)

tmp = nebulae[(nebulae['overlap_neb']>0) & (nebulae['in_frame'])]
#ax.hist(tmp['EBV_balmer'],bins=bins,alpha=0.5,label='association')

tmp = nebulae[(nebulae['Ndolphot']>0) & (nebulae['overlap_neb']==0) & (nebulae['in_frame'])]
ax.hist(tmp['EBV_balmer'],bins=bins,alpha=0.5,label='overlap')

tmp = nebulae[(nebulae['Ndolphot']==0) & (nebulae['overlap_neb']==0) & (nebulae['in_frame'])]
ax.hist(tmp['EBV_balmer'],bins=bins,alpha=0.5,label='isolated')

ax.legend()
ax.set(xlabel=r'$E(B-V)$')
plt.show()

looks at the brightest peak inside of them

with pandas it is easy to select the min/max row in a group based on a column so we convert to a DataFrame and back to a Table

In [ ]:
import pandas as pd

faint_peaks = []
for gal_name in sample_table['name']:
    
    sub_nebulae = nebulae[(nebulae['gal_name']==gal_name) & (nebulae['Ndolphot']==0)]
    sub_peaks = dolphot_peaks[dolphot_peaks['gal_name']==gal_name]

    df = sub_peaks.to_pandas()
    faint_peaks.append(Table.from_pandas(df.loc[df.groupby('region_ID')['Mag_WFC3_F275W'].idxmin()]))
    break
faint_peaks=faint_peaks[0]

In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))


ax1.hist(faint_peaks['Mag_WFC3_F275W'],bins=np.linspace(20,28,16))
ax1.set(xlim=[20,28],xlabel=r'm NUV',yscale='linear',ylabel='N')
ax2.hist(faint_peaks['S2N_WFC3_F275W'],bins=np.linspace(-0.5,10.5,11))
ax2.set(xlim=[0,10],xlabel=r'S/N NUV',yscale='linear',ylabel='N')

plt.show()


In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

bins = np.logspace(35,40,20)

tmp = nebulae[(nebulae['overlap_neb']>0) & (nebulae['in_frame'])]
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='matched')

ax.hist(isolated_HIIregions['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='isolated')

ax.hist(single_peaks['HA6562_LUM_CORR'],bins=bins,alpha=0.7,label='O-Star?')

ax.legend()

ax.set(xscale='log',xlim=[2e35,9e39],xlabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$',
       yscale='linear',ylim=[1,2e3],ylabel='N')
plt.show()


look at the concentration index of the new objects to see why they were rejected

In [ ]:
S2N = 5

gal_name = 'NGC4535'
tmp = dolphot_peaks[(dolphot_peaks['detection']>=2) & 
                    (dolphot_peaks['Mag_WFC3_F275W']<25) & 
                    (dolphot_peaks['S2N_WFC3_F275W']>S2N) &
                    (dolphot_peaks['gal_name']==gal_name)] 

dolphot_augmented = Table.read(basedir/'data'/'external'/'MCI'/'ngc4535_uvis_f555w_exp_drc_augmenteddolcat.fits')
dolphot_augmented.rename_column('dolphot_id','Dolphot_ID')
dolphot_augmented = dolphot_augmented[~dolphot_augmented['Dolphot_ID'].mask]
tmpIC5332 = join(tmp,dolphot_augmented,keys='Dolphot_ID')

gal_name = 'NGC1433'
tmp = dolphot_peaks[(dolphot_peaks['detection']>=2) & 
                    (dolphot_peaks['Mag_WFC3_F275W']<25) & 
                    (dolphot_peaks['S2N_WFC3_F275W']>S2N) &
                    (dolphot_peaks['gal_name']==gal_name)] 

dolphot_augmented = Table.read(basedir/'data'/'external'/'MCI'/'ic5332_augmented_dolphot_cat_pre_candidates_v1_1.fits')
dolphot_augmented.rename_column('dolphot_id','Dolphot_ID')
dolphot_augmented = dolphot_augmented[~dolphot_augmented['Dolphot_ID'].mask]
tmpNGC1433 = join(tmp,dolphot_augmented,keys='Dolphot_ID')

gal_name = 'NGC1672'
tmp = dolphot_peaks[(dolphot_peaks['detection']>=2) & 
                    (dolphot_peaks['Mag_WFC3_F275W']<25) & 
                    (dolphot_peaks['S2N_WFC3_F275W']>S2N) &
                    (dolphot_peaks['gal_name']==gal_name)] 

dolphot_augmented = Table.read(basedir/'data'/'external'/'MCI'/'ngc1672_uvis_f555w_exp_drc_augmenteddolcat_v1p1.fits')
dolphot_augmented.rename_column('dolphot_id','Dolphot_ID')
dolphot_augmented = dolphot_augmented[~dolphot_augmented['Dolphot_ID'].mask]
tmpNGC1672 = join(tmp,dolphot_augmented,keys='Dolphot_ID')

gal_name = 'NGC4535'
tmp = dolphot_peaks[(dolphot_peaks['detection']>=2) & 
                    (dolphot_peaks['Mag_WFC3_F275W']<25) & 
                    (dolphot_peaks['S2N_WFC3_F275W']>S2N) &
                    (dolphot_peaks['gal_name']==gal_name)] 

dolphot_augmented = Table.read(basedir/'data'/'external'/'MCI'/'ngc4535_uvis_f555w_exp_drc_augmenteddolcat.fits')
dolphot_augmented.rename_column('dolphot_id','Dolphot_ID')
dolphot_augmented = dolphot_augmented[~dolphot_augmented['Dolphot_ID'].mask]
tmpNGC4535 = join(tmp,dolphot_augmented,keys='Dolphot_ID')

In [ ]:
X=[-0.3283853837400431,0.00882538879757222,0.10767480085977987,0.2926033632570486,0.04397521810595495,-0.3283853837400431]
Y=[-1.1139840687824003,-1.1377544569477822,-0.7350486787204453,0.5399544822354279,0.5598052851182196,-1.1139840687824003]
cmap = plt.cm.get_cmap('viridis',8)
vmin,vmax=22,26

fig,((ax1,ax2),(ax3,ax4))=plt.subplots(figsize=(two_column,two_column),ncols=2,nrows=2)

s = tmpIC5332[tmpIC5332['S2N_WFC3_F555W']<10]
ax1.scatter(s['mci_in'],s['mci_out'],label=r'S/N$_\mathrm{V}<10$',color='0.8',s=1.5,rasterized=True)
s = tmpIC5332[tmpIC5332['S2N_WFC3_F555W']>=10]
sc=ax1.scatter(s['mci_in'],s['mci_out'],c=s['Mag_WFC3_F555W'],vmin=vmin,vmax=vmax,cmap=cmap,
            label=r'S/N$_\mathrm{V}>10$',s=1.5,rasterized=True)
ax1.legend()
ax1.set_title('IC5332')

s = tmpNGC1433[tmpNGC1433['S2N_WFC3_F555W']<10]
ax2.scatter(s['mci_in'],s['mci_out'],label=r'S/N$<10$',color='0.8',s=1.5,rasterized=True)
s = tmpNGC1433[tmpNGC1433['S2N_WFC3_F555W']>=10]
ax2.scatter(s['mci_in'],s['mci_out'],c=s['Mag_WFC3_F555W'],vmin=vmin,vmax=vmax,cmap=cmap,
            label=r'S/N$_\mathrm{V}>10$',s=1.5,rasterized=True)
ax2.set_title('NGC1433')

s = tmpNGC1672[tmpNGC1672['S2N_WFC3_F555W']<10]
ax3.scatter(s['mci_in'],s['mci_out'],label=r'S/N$<10$',color='0.8',s=1.5,rasterized=True)
s = tmpNGC1672[tmpNGC1672['S2N_WFC3_F555W']>=10]
ax3.scatter(s['mci_in'],s['mci_out'],c=s['Mag_WFC3_F555W'],vmin=vmin,vmax=vmax,cmap=cmap,
            label=r'S/N$>10$',s=1.5,rasterized=True)
ax3.set_title('NGC1672')

s = tmpNGC4535[tmpNGC4535['S2N_WFC3_F555W']<10]
ax4.scatter(s['mci_in'],s['mci_out'],label=r'S/N$<10$',color='0.8',s=1.5,rasterized=True)
s = tmpNGC4535[tmpNGC4535['S2N_WFC3_F555W']>=10]
ax4.scatter(s['mci_in'],s['mci_out'],c=s['Mag_WFC3_F555W'],vmin=vmin,vmax=vmax,cmap=cmap,
            label=r'S/N$>10$',s=1.5,rasterized=True)
ax4.set_title('NGC4535')

for ax in (ax1,ax2,ax3,ax4):
    ax.plot(X,Y,color='black')
    ax.set(xlim=[-0.6,0.5],ylim=[-2,1],xlabel=r'MCI$_\mathrm{in}$',ylabel=r'MCI$_\mathrm{in}$')
plt.tight_layout()

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
fig.colorbar(sc,cax=cbar_ax,label=r'$m_V$')

plt.savefig(basedir/'reports'/'concentration_index.png',dpi=300)
plt.show()

NGC0628 consists of two pointings. Here we merge the two catalogues

In [ ]:
gal_name = 'NGC0628c'
with fits.open(data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_ml_class12.fits') as hdul:
    compact_clusters1 = Table(hdul[1].data)
gal_name = 'NGC0628e'
with fits.open(data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_ml_class12.fits') as hdul:
    compact_clusters2 = Table(hdul[1].data)
    
compact_clusters2['ID_PHANGS_CLUSTERS'][np.isin(compact_clusters2['ID_PHANGS_CLUSTERS'],compact_clusters1['ID_PHANGS_CLUSTERS'])] += np.max(compact_clusters1['ID_PHANGS_CLUSTERS'])

merged = vstack([compact_clusters1,compact_clusters2])

primary_hdu = fits.PrimaryHDU()
table_hdu   = fits.BinTableHDU(merged)
hdul = fits.HDUList([primary_hdu, table_hdu])
hdul.writeto(data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_ngc0628_phangs-hst_v1p1_ml_class12.fits',overwrite=True)

In [ ]:
gal_name = 'NGC0628c'
with fits.open(data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_human_class12.fits') as hdul:
    compact_clusters1 = Table(hdul[1].data)
gal_name = 'NGC0628e'
with fits.open(data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_human_class12.fits') as hdul:
    compact_clusters2 = Table(hdul[1].data)
    
compact_clusters2['ID_PHANGS_CLUSTERS'][np.isin(compact_clusters2['ID_PHANGS_CLUSTERS'],compact_clusters1['ID_PHANGS_CLUSTERS'])] += np.max(compact_clusters1['ID_PHANGS_CLUSTERS'])

merged = vstack([compact_clusters1,compact_clusters2])

primary_hdu = fits.PrimaryHDU()
table_hdu   = fits.BinTableHDU(merged)
hdul = fits.HDUList([primary_hdu, table_hdu])
hdul.writeto(data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_ngc0628_phangs-hst_v1p1_human_class12.fits',overwrite=True)

#### Models of single stars

we can use the stellar atmosphere models from cloudy
https://gitlab.nublado.org/cloudy/cloudy/-/wikis/StellarAtmospheres

they use the following models
http://tlusty.oca.eu/Tlusty2002/tlusty-frames-OS02.html

see Figure 1 to estimate radius based on log g
https://ui.adsabs.harvard.edu/abs/2003ApJS..146..417L/abstract

Example: G35000g400v10. The first letter indicates the composition (G=solar), followed by the effective temperature, the gravity and the turbulent velocity.

In [ ]:
import speclite.filters

filter_path = basedir/'data'/'external'/'UVIS'

wlen,response=np.loadtxt(filter_path/'wfc3_uvis1_f275w.txt',delimiter=' ').T
uvis_f275w = speclite.filters.FilterResponse(
    wavelength=wlen*u.Angstrom,response = response, 
    meta=dict(group_name='uvis', band_name='f275w'))

wlen,response=np.loadtxt(filter_path/'wfc3_uvis1_f336w.txt',delimiter=' ').T
uvis_f336w = speclite.filters.FilterResponse(
    wavelength=wlen*u.Angstrom,response = response, 
    meta=dict(group_name='uvis', band_name='f336w'))

wlen,response=np.loadtxt(filter_path/'wfc3_uvis1_f438w.txt',delimiter=' ').T
uvis_f438w = speclite.filters.FilterResponse(
    wavelength=wlen*u.Angstrom,response = response, 
    meta=dict(group_name='uvis', band_name='f438w'))

wlen,response=np.loadtxt(filter_path/'wfc3_uvis1_f555w.txt',delimiter=' ').T
uvis_f555w = speclite.filters.FilterResponse(
    wavelength=wlen*u.Angstrom,response = response, 
    meta=dict(group_name='uvis', band_name='f555w'))

wlen,response=np.loadtxt(filter_path/'wfc3_uvis1_f814w.txt',delimiter=' ').T
uvis_f814w = speclite.filters.FilterResponse(
    wavelength=wlen*u.Angstrom,response = response, 
    meta=dict(group_name='uvis', band_name='f814w'))

uvis = speclite.filters.load_filters('uvis-f275w', 'uvis-f336w','uvis-f438w','uvis-f555w','uvis-f814w')
#speclite.filters.plot_filters(uvis)

In [ ]:
import re 

mass = 120*u.Msun
logg = 300
temp = 30000

model = f'G{temp}g{logg}v10'
result = re.search('([A-Z])(\d+)g(\d+)',model)
Z,temp,logg = result.groups()
g = 10**(float(logg)/100) * u.cm/u.s**2
radius = np.sqrt(c.G*mass / g).to(u.Rsun)

OBstars = ascii.read(basedir/'data'/'external'/'OBstars'/f'{model}.vis.17',
                     data_start=0,names=['wavelength','flux'])
wlen = OBstars['wavelength'] * u.Angstrom
flux = OBstars['flux'] * u.erg / (u.cm**2 * u.s * u.Angstrom)
flux, wlen = uvis.pad_spectrum(flux, wlen)

#uvis.get_ab_magnitudes(flux*(radius/u.pc)**2,wlen)

In [ ]:
import re

mass = np.array([15,120]) * u.Msun
logg = [400,300]
temp = [55000,30000]


def AB_mag_from_spec(mass,logg,temp,mu):
    
    # https://www.astronomy.ohio-state.edu/martini.10/usefuldata.html
    AB_to_vega = {
        'bessell-U' : 0.79,
        'bessell-B' : -0.09,
        'bessell-V' : 0.02
    }
    
    model = f'G{temp}g{logg}v10'
    result = re.search('([A-Z])(\d+)g(\d+)',model)
    Z,temp,logg = result.groups()
    g = 10**(float(logg)/100) * u.cm/u.s**2
    radius = np.sqrt(c.G*mass / g).to(u.Rsun)

    OBstars = ascii.read(basedir/'data'/'external'/'OBstars'/f'{model}.vis.17',
                         data_start=0,names=['wavelength','flux'])
    wlen = OBstars['wavelength'] * u.Angstrom
    flux = OBstars['flux'] * u.erg / (u.cm**2 * u.s * u.Angstrom)
    flux, wlen = uvis.pad_spectrum(flux, wlen)

    mags = uvis.get_ab_magnitudes(flux*(radius/u.pc)**2,wlen)
    print(f"M={mass:>5}, R={radius:>5.1f}, T={temp} K, g={g:>8.2g}: M_U={mags['uvis-f336w'][0]+ AB_to_vega['bessell-U']:>5.1f}, m_U={mags['uvis-f336w'][0]+mu+AB_to_vega['bessell-U']:.1f}")
    
mu =30
AB_mag_from_spec(120*u.Msun,400,55000,mu)
AB_mag_from_spec(120*u.Msun,300,30000,mu)
AB_mag_from_spec(40*u.Msun,400,40000,mu)
AB_mag_from_spec(40*u.Msun,300,27500,mu)
AB_mag_from_spec(20*u.Msun,450,35000,mu)
AB_mag_from_spec(20*u.Msun,350,27500,mu)
AB_mag_from_spec(15*u.Msun,450,27500,mu)

In [ ]:
model = 'G50000g400v10'
OBstars = ascii.read(basedir/'data'/'external'/'OBstars'/f'{model}.vis.7',
                     data_start=0,names=['wavelength','flux'])
    
fig,ax=plt.subplots(figsize=(8,3))
ax.plot(OBstars['wavelength'],OBstars['flux'])

ax.set(xscale='log',yscale='log')

plt.show()

using the calibration from Martins+2015 we can estimate the HII region that a single star could ionise

In [ ]:
O_stars_1 = ascii.read(basedir/'data'/'external'/'martins+2015'/'table1_OV_theo.tex',format='latex')
O_stars_1['LHa'] = 10**O_stars_1['Q0'] / 7.31e11

O_stars_2 = ascii.read(basedir/'data'/'external'/'martins+2015'/'table2_Oiii_theo.tex',format='latex')
O_stars_2['LHa'] = 10**O_stars_2['Q0'] / 7.31e11

O_stars_3 = ascii.read(basedir/'data'/'external'/'martins+2015'/'table3_Oi_theo.tex',format='latex')
O_stars_3['LHa'] = 10**O_stars_3['Q0'] / 7.31e11

O_stars_4 = ascii.read(basedir/'data'/'external'/'martins+2015'/'table4_OV_obs.tex',format='latex')
O_stars_4['LHa'] = 10**O_stars_4['Q0'] / 7.31e11

O_stars_5 = ascii.read(basedir/'data'/'external'/'martins+2015'/'table5_Oiii_obs.tex',format='latex')
O_stars_5['LHa'] = 10**O_stars_5['Q0'] / 7.31e11

O_stars_6 = ascii.read(basedir/'data'/'external'/'martins+2015'/'table6_Oi_obs.tex',format='latex')
O_stars_6['LHa'] = 10**O_stars_6['Q0'] / 7.31e11

In [ ]:
fig, ((ax1,ax2),(ax3,ax4))=plt.subplots(ncols=2,nrows=2,figsize=(two_column,two_column))

ax1.plot(O_stars_1['mass'],O_stars_1['LHa'],label='V')
ax1.plot(O_stars_2['mass'],O_stars_2['LHa'],label='III')
ax1.plot(O_stars_3['mass'],O_stars_3['LHa'],label='I')
ax1.set(xlabel=r'Mass / M$_\odot$',ylabel=r'L(H$\alpha$) / erg s$^{-1}$',
        xlim=[15,60],ylim=[4e35,1e38],yscale='log')
ax1.set_title(r'theoretical $T_\mathrm{eff}$')

ax2.plot(O_stars_4['mass'],O_stars_4['LHa'],label='V')
ax2.plot(O_stars_5['mass'],O_stars_5['LHa'],label='III')
ax2.plot(O_stars_6['mass'],O_stars_6['LHa'],label='I')
ax2.set(xlabel=r'Mass / M$_\odot$',ylabel=r'L(H$\alpha$) / erg s$^{-1}$',
        xlim=[15,60],ylim=[4e35,1e38],yscale='log')
ax2.set_title(r'observational $T_\mathrm{eff}$')
ax2.legend(loc=4)

ax3.plot(O_stars_1['mass'],O_stars_1['Mv'],label='V')
ax3.plot(O_stars_2['mass'],O_stars_2['Mv'],label='III')
ax3.plot(O_stars_3['mass'],O_stars_3['Mv'],label='I')
ax3.set(xlabel=r'Mass / M$_\odot$',ylabel=r'$M_V$',
        xlim=[15,60],ylim=[-7,-3.5],yscale='linear')
ax3.invert_yaxis()

ax4.plot(O_stars_4['mass'],O_stars_4['Mv'],label='V')
ax4.plot(O_stars_5['mass'],O_stars_5['Mv'],label='III')
ax4.plot(O_stars_6['mass'],O_stars_6['Mv'],label='I')
ax4.set(xlabel=r'Mass / M$_\odot$',ylabel=r'$M_V$',
        xlim=[15,60],ylim=[-7,-3.5],yscale='linear')
ax4.invert_yaxis()

plt.tight_layout()

plt.show()

In [ ]:
points = np.array([[-0.33 , -1.662],[-0.307, -1.577],[-0.25 , -1.468],[-0.218, -1.394],[-0.186, -1.321],
                   [-0.138, -1.235],[-0.076, -1.213],[-0.007, -1.206],[ 0.065, -1.197],[ 0.135, -1.187],
                   [ 0.203, -1.198],[ 0.276, -1.201],[ 0.347, -1.209],[ 0.418, -1.219],[ 0.488, -1.233],
                   [ 0.559, -1.251],[ 0.63 , -1.27 ],[ 0.706, -1.264],[ 0.771, -1.287],[ 0.648, -1.119],
                   [ 0.626, -1.026],[ 0.565, -0.989],[ 0.559, -0.87 ],[ 0.502, -0.807],[ 0.481, -0.706],
                   [ 0.457, -0.546],[ 0.505, -0.449],[ 0.487, -0.384],[ 0.488, -0.328],[ 0.468, -0.239],
                   [ 0.517, -0.109],[ 0.555, -0.015],[ 0.6  ,  0.041],[ 0.65 ,  0.132],[ 0.724,  0.174],
                   [ 0.864,  0.169],[ 0.948,  0.166],[ 1.02 ,  0.152],[ 1.138,  0.182],[ 1.203,  0.194],
                   [ 1.253,  0.282],[ 1.331,  0.378],[ 1.354,  0.469]])

x,y=points.T

fig,ax = plt.subplots(figsize=(single_column,single_column))

U = single_peaks['Mag_WFC3_F336W']
B = single_peaks['Mag_WFC3_F438W']
V = single_peaks['Mag_WFC3_F555W']
I = single_peaks['Mag_WFC3_F814W']

ax.scatter(V-I,U-B)
ax.plot(x,y,color='black')
ax.set(xlim=[-1,2],ylim=[-2.5,1],
      ylabel=r'F336W$-$F438W',xlabel=r'F555W$-$F814W')
ax.invert_yaxis()

plt.show()

In [ ]:
fig,ax=plt.subplots(ncols=1,figsize=(single_column,single_column/1.))

ax.scatter(single_peaks['MV'],single_peaks['HA6562_LUM_CORR'],color='0.8',s=0.6,zorder=0,rasterized=True)

ax.plot(O_stars_1['Mv'],O_stars_1['LHa'],label='class V stars',color=tab10[0])
ax.plot(O_stars_2['Mv'],O_stars_2['LHa'],label='class III stars',color=tab10[1])
ax.plot(O_stars_3['Mv'],O_stars_3['LHa'],label='class I stars',color=tab10[2])
ax.plot(O_stars_4['Mv'],O_stars_4['LHa'],ls='--',color=tab10[0])
ax.plot(O_stars_5['Mv'],O_stars_5['LHa'],ls='--',color=tab10[1])
ax.plot(O_stars_6['Mv'],O_stars_6['LHa'],ls='--',color=tab10[2])

t = ax.text(-5.79,5.9e+37,r'60 M$_\odot$',color='black',fontsize=7,ha='right',va='bottom')
t = ax.text(-3.9,4.2e+35,r'16 M$_\odot$',color='black',fontsize=7,ha='center',va='top')

ax.legend(loc=2)
ax.set(xlabel='$M_V$ / mag',ylabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$',
       yscale='log',ylim=[1e35,1e39],xlim=[-9,-3])
ax.invert_xaxis()

axin = ax.inset_axes([0.7,0.12,0.25,1.*0.25])

axin.scatter(V-I,U-B,marker='p',s=0.5,ec=None,c='0.8',zorder=0,rasterized=True)
axin.plot(x,y,color='black',lw=0.4)
axin.set(xlim=[-1.5,1.5],ylim=[-2.5,0.9])
axin.set_xlabel(r'$V-I$',fontsize=5)
axin.set_ylabel(r'$U-B$',fontsize=6)
axin.set_xticks([-1,0,1])
axin.tick_params(axis='both', labelsize=5)
axin.invert_yaxis()

plt.savefig(basedir/'reports'/'dolphot_peaks_stellar_models_new.pdf',dpi=600)
plt.show()

In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))

ax1.hist(single_peaks['HA6562_LUM_CORR'],np.logspace(35,40,20))
ax1.set(xlabel=r'H$\alpha$ / erg s$^{-1}$',xscale='log',xlim=[1e35,1e40])

ax2.hist(single_peaks['MV'],bins=np.linspace(-8,-3,10))
ax2.set(xlabel='$M_V$',xlim=[-3,-8])
ax2.invert_xaxis()

plt.show()

#### Model single stars

https://beast.readthedocs.io/en/latest/index.html

#### Use compact clusters for age trends

In [ ]:
print(f'{len(compact_clusters)} peaks in initial catalogue')
lst = []
for gal_name in sample_table['name']:
    tmp = compact_clusters[compact_clusters['gal_name']==gal_name]
    ids, counts = np.unique(tmp['region_ID'],return_counts=True)
    ids = ids[counts==1]
    tmp = tmp[np.isin(tmp['region_ID'],ids)]
    HII_sub = nebulae[(nebulae['gal_name']==gal_name)]
    del HII_sub['SkyCoord_neb']
    if len(tmp)>0:
        tmp = join(tmp,HII_sub,keys=['gal_name','region_ID'])
        lst.append(tmp)

compact_clusters_1to1 = vstack(lst)
compact_clusters_1to1['SkyCoord'] = SkyCoord(compact_clusters_1to1['PHANGS_RA']*u.deg,compact_clusters_1to1['PHANGS_DEC']*u.deg)
compact_clusters_1to1.rename_columns(['PHANGS_AGE_MINCHISQ','PHANGS_MASS_MINCHISQ'],['age','mass'])
print(f'{len(compact_clusters_1to1)} single peaks in final catalogue')

In [ ]:
from astrotools.plot.corner import corner_binned_stat, corner_density_scatter,\
                                corner_binned_percentile, corner_spearmanr
from scipy.stats import binned_statistic

    
nbins = 8
criteria  = (compact_clusters_1to1['mass']>1e4) 
criteria &= (compact_clusters_1to1['age']<=8) 
#criteria &= (compact_clusters_1to1['overlap_neb']==0) 

tmp = compact_clusters_1to1[criteria].copy()
tmp['HA/FUV_corr'][tmp['FUV_FLUX_CORR']/tmp['FUV_FLUX_CORR_ERR']<3] = np.nan
print(f'sample contains {len(tmp)} objects')

columns  = ['age','EW_HA','HA/FUV_corr','logq_D91']

fig,(ax1,ax2,ax3) = plt.subplots(nrows=3,figsize=(single_column,single_column*1.5),sharex=True)

for col,ax in zip(['EW_HA','HA/FUV_corr','logq_D91'],[ax1,ax2,ax3]):
    ax.set(ylim=limits[col])
    x,y = tmp['age'],tmp[col]
    x,y = x[np.isfinite(x) & np.isfinite(y)], y[np.isfinite(x) & np.isfinite(y)]
    corner_binned_percentile(x,y,ax,nbins=nbins,range=limits['age'],n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=nbins,range=limits['age'],n=98,color='gray',alpha=0.2)

    mean, edges, _ = binned_statistic(x,y,statistic='median',bins=nbins,range=limits['age'])
    xp = (edges[1:]+edges[:-1])/2
    ax.errorbar(xp,mean,fmt='o-',ms=2,color='0.3')
    
    corner_density_scatter(x,y,ax,nbins=20,s=0.5,cmap=plt.cm.Reds)
    corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=False,fontsize=7)
    ax.set(ylabel=labels[col])
ax3.set(xlabel='age / Myr')
plt.tight_layout()

filename = basedir/'reports'/f'age_tracers_clusters.pdf'
#plt.savefig(filename,dpi=300)

plt.show()

In [ ]:
fig,ax=plt.subplots()

bins=np.arange(0.5,11.5,1)

ax.hist(compact_clusters_1to1['age'],bins=bins,label='compact clusters',histtype='step')
ax.hist(catalogue['age'],bins=bins,label='stellar associations',histtype='step')
ax.set(xlabel='age / Myr')
ax.legend()

plt.show()

### Age difference between multiple associations

In [ ]:
diff = []

for name in sample_table['name']:
    print(name)
    nebulae_sub = nebulae[nebulae['gal_name']==name]
    associations_sub = associations[associations['gal_name']==name]
    associations_sub.add_index('assoc_ID')

    with open(basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'/f'{name}_{HSTband}_{scalepc}pc_nebulae.yml') as f:
        nebulae_dict = yaml.load(f,Loader=yaml.SafeLoader)
    with open(basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'/f'{name}_{HSTband}_{scalepc}pc_associations.yml') as f:
        associations_dict = yaml.load(f,Loader=yaml.SafeLoader)

    multiple = nebulae_sub[nebulae_sub['Nassoc']>1]['region_ID']

    for region_ID in multiple:
        ages = associations_sub.loc[nebulae_dict[region_ID]]['age']
        diff.append(np.max(ages)-np.min(ages))

## Match with compact clusters

In [ ]:
def get_value(matrix, index, default_value=np.nan):
    '''
    The `to_pixel` method returns the x,y coordinates. However in the 
    image they correspond to img[y,x]
    '''
    result = np.zeros(len(index))+default_value
    mask = (index[:,1] < matrix.shape[0]) & (index[:,0] < matrix.shape[1])
    mask &= (index[:,1] >= 0) & (index[:,0] >=0)

    valid = index[mask]
    result[mask] = matrix[valid[:,1], valid[:,0]]
    return result


compact_clusters_list = []

N_neb = 0
N_ovl = 0
for gal_name in np.unique(sample_table['name']):

    # we are only interested in peaks that overlap with isolated HII regions
    sub_nebulae = nebulae[(nebulae['gal_name']==gal_name) & (nebulae['in_frame'])]
    N_neb += len(sub_nebulae)
    
    print(f'working on {gal_name}')
    
    # read the nebula mask
    filename = next((data_ext/'MUSE'/'DR2.1'/'copt'/'MUSEDAP').glob(f'{gal_name}*.fits'))
    copt_res = float(filename.stem.split('-')[1].split('asec')[0])
    with fits.open(filename) as hdul:
        Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                        mask=np.isnan(hdul['HA6562_FLUX'].data),
                        meta=hdul['HA6562_FLUX'].header,
                        wcs=WCS(hdul['HA6562_FLUX'].header))

    filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
    with fits.open(filename) as hdul:
        nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
        nebulae_mask.data[nebulae_mask.data==-1] = np.nan
    
    # we only need the wcs to convert from pixel to skycoord
    filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
    if not filename.is_file():
        print(f'no F275W image for {gal_name}')
        continue
    with fits.open(filename) as hdul:
        #F275 = NDData(hdul[0].data,
        #              mask=hdul[0].data==0,
        #              meta=hdul[0].header,
        #              wcs=WCS(hdul[0].header))
        wcs_F275=WCS(hdul[0].header)
    
    # read the compact cluster and DOLPHOT catalogues
    filename = data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_ml_class12.fits'
    if not filename.is_file():
        print(f'no compact clusters for {gal_name}')
    else:     
        with fits.open(filename) as hdul:
            compact_clusters = Table(hdul[1].data)
        compact_clusters['SkyCoord'] = SkyCoord(compact_clusters['PHANGS_RA']*u.deg,compact_clusters['PHANGS_DEC']*u.deg)
        compact_clusters.add_index('ID_PHANGS_CLUSTERS')

        compact_clusters['region_ID'] = get_value(nebulae_mask.data,np.array(compact_clusters['SkyCoord'].to_pixel(nebulae_mask.wcs)).T.astype(int))

        # we save all objects that fall inside one of our empty HII regions
        compact_clusters.add_column(gal_name,index=0,name='gal_name')
        del compact_clusters['SkyCoord']
        compact_clusters_list.append(compact_clusters[np.isin(compact_clusters["region_ID"],sub_nebulae["region_ID"])])
    N_ovl += np.sum(np.isin(sub_nebulae['region_ID'],compact_clusters['region_ID']))
    
    
compact_clusters = vstack(compact_clusters_list)

primary_hdu = fits.PrimaryHDU()
table_hdu   = fits.BinTableHDU(compact_clusters)
hdul = fits.HDUList([primary_hdu, table_hdu])
#hdul.writeto(basedir/'data'/'interim'/f'compact_clusters_in_isolated_HIIregions.fits',overwrite=True)

In [ ]:
N_ovl / N_neb

## How many matches are by change?

we rotate one of the masks by 90 degree and see how many still overlap

In [ ]:
from reproject import reproject_interp
from cluster.io import read_associations


version = 'v1p2'
HSTband = 'nuv'
scalepc = 32

gal_name = 'NGC1365'


# 1 = 90 deg, 2 = 180 deg ...
print(9*' '+'Nneb ovlp Nasc cont part 1to1')

# these are the HST images with a single pointing
#for gal_name in ['IC5332','NGC1087','NGC1365','NGC1385','NGC1433','NGC1566','NGC2835','NGC4303','NGC4321','NGC4535','NGC7496']
for gal_name in ['NGC4535','NGC7496']:
    
    print(f'{gal_name}')
    filename = next((data_ext/'MUSE'/'DR2.1'/'copt'/'MUSEDAP').glob(f'{gal_name}*.fits'))
    copt_res = float(filename.stem.split('-')[1].split('asec')[0])
    with fits.open(filename) as hdul:
        Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                        uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                        mask=np.isnan(hdul['HA6562_FLUX'].data),
                        meta=hdul['HA6562_FLUX'].header,
                        wcs=WCS(hdul['HA6562_FLUX'].header))

    filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
    with fits.open(filename) as hdul:
        nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
        nebulae_mask.data[nebulae_mask.data==-1] = np.nan


    # filter image with uncertainties
    filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
    with fits.open(filename) as hdul:
        F275 = NDData(hdul[0].data,
                      mask=hdul[0].data==0,
                      meta=hdul[0].header,
                      wcs=WCS(hdul[0].header))

    rot_img = np.rot90(F275.data,k=1)
    if np.sum((F275.data!=0) ^ (rot_img!=0)) / np.sum(F275.data!=0) > 0.1:
        print(np.sum((F275.data!=0) ^ (rot_img!=0)) / np.sum(F275.data!=0))
        print('warning: rotated image does not overlap')

    associations, associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',target=gal_name.lower(),
                                                        HSTband=HSTband,scalepc=scalepc,version=version)

    nebulae_hst, _  = reproject_interp(nebulae_mask,
                                       output_projection=associations_mask.wcs,
                                       shape_out=associations_mask.data.shape,
                                       order='nearest-neighbor') 
    # we need this to determine which 
    _ , footprint  = reproject_interp(Halpha,
                                       output_projection=associations_mask.wcs,
                                       shape_out=associations_mask.data.shape,
                                       order='nearest-neighbor') 

    for k in [0,1]:

        region_ID = np.unique(nebulae_hst[~np.isnan(nebulae_hst)])
        #print(f'neb total: {len(region_ID)}')

        not_in_frame_region_ID = np.rot90(nebulae_hst,k=k)[F275.data==k]
        not_in_frame_region_ID = np.unique(not_in_frame_region_ID[~np.isnan(not_in_frame_region_ID)])
        #print(f'not in frame: {len(not_in_frame_region_ID)}')

        in_frame_region_ID = np.setdiff1d(region_ID,not_in_frame_region_ID)
        #print(f'in frame: {len(in_frame_region_ID)}')

        assoc_ID = np.unique(associations_mask.data[~np.isnan(associations_mask.data)])
        #print(f'assoc total: {len(assoc_ID)}')

        not_in_frame_assoc_ID = associations_mask.data[~np.rot90(footprint.astype(bool),k=k)]
        not_in_frame_assoc_ID = np.unique(not_in_frame_assoc_ID[~np.isnan(not_in_frame_assoc_ID)])
        #print(f'not in frame: {len(not_in_frame_assoc_ID)}')

        in_frame_assoc_ID = np.setdiff1d(assoc_ID,not_in_frame_assoc_ID)
        #print(f'in frame: {len(in_frame_assoc_ID)}')

        # we scale the associations such that the the id is in the decimal
        scale = 10**np.ceil(np.log10(max(associations_mask.data[~np.isnan(associations_mask.data)])))
        s_arr = associations_mask.data/scale+np.rot90(nebulae_hst,k=k)

        # ids of associations, nebulae and combination (sum) of both
        a_id = np.unique(associations_mask.data[~np.isnan(associations_mask.data)]).astype(int)
        n_id = np.unique(nebulae_mask.data[~np.isnan(nebulae_mask.data)]).astype(int)
        s_id = np.unique(s_arr[~np.isnan(s_arr)])

        # this splits the sum into two parts (nebulae and associations)
        a_modf,n_modf = np.modf(s_id)
        n_modf = n_modf.astype(int)
        a_modf = np.round(a_modf*scale).astype(int)

        unique_a, count_a = np.unique(a_modf,return_counts=True)
        unique_n, count_n = np.unique(n_modf,return_counts=True)

        nebulae_dict = {int(n) : a_modf[n_modf==n].tolist() for n in n_id}     
        associations_dict = {int(a) : n_modf[a_modf==a].tolist() for a in a_id}     

        # so far we ensured that the nebulae in unique_n have only one association,
        # but it is possible that this association goes beyond the nebulae and into
        # a second nebulae. Those objects are excluded here
        isolated_nebulae = set()
        isolated_assoc   = set()
        for n,v in nebulae_dict.items():
            if len(v)==1:
                if len(associations_dict[v[0]])==1:
                    isolated_nebulae.add(n)
                    isolated_assoc.add(v[0])

        # find all assoc that have at least one pixel outside of the nebulae masks
        mask = associations_mask.data.copy()
        mask[~np.isnan(np.rot90(nebulae_hst,k=k))] = np.nan
        outside = np.unique(mask[~np.isnan(mask)].astype(int))

        # find all assoc that have at least one pixel inside of the nebulea masks
        mask = associations_mask.data.copy()
        mask[np.isnan(np.rot90(nebulae_hst,k=k))] = np.nan
        inside = np.unique(mask[~np.isnan(mask)].astype(int))

        contained = np.setdiff1d(inside,outside)
        partial   = np.intersect1d(inside,outside)

        # we only use the nebulae that fall in the FOV
        in_frame_region_ID = np.rot90(nebulae_hst,k=k)[F275.data!=0]
        in_frame_region_ID = np.unique(in_frame_region_ID[~np.isnan(in_frame_region_ID)])
        n_over = np.sum([len(a_modf[n_modf==n].tolist())>0 for n in n_id])

        print(f'{90*k:>3} deg: {len(in_frame_region_ID):>4} {n_over:>4} {len(in_frame_assoc_ID):>4} {len(contained):>4} {len(partial):>4} {len(isolated_nebulae):>4}')


In [ ]:
names =  ['IC5332','NGC1087','NGC1365','NGC1385','NGC1433','NGC1566','NGC2835','NGC4303','NGC4535','NGC7496']
# without rotation
text = '''Nneb ovlp Nasc cont part 1to1
777  295  393   45  215  154
1006  299  487  280  137  182
926  191  489  286   95  112
1029  310  525  341  139  133
1024  355  494  134  254  250
2119  816 1578  707  579  373
1039  365  641  232  244  223
2696  973 1735  853  584  402
1567  385  465  181  207  197
743  163  264  185   54   96
'''

t0 = ascii.read(text)
t0.add_column(names,name='gal_name',index=0)

t0.add_row(['total',np.sum(t0['Nneb']),np.sum(t0['ovlp']),np.sum(t0['Nasc']),np.sum(t0['cont']),np.sum(t0['part']),np.sum(t0['1to1'])])
t0

In [ ]:
# rotation
text = '''Nneb ovlp Nasc cont part 1to1
783  144  378    4  131   81
1005  139  447   51  126   68
908   66  493   25   61   46
1029  193  521   98  173   82
982   82  438    4   87   57
2138  463 1540   86  507  203
1046  208  658   39  213  129
2691  625 1738  192  616  290
1561   86  383    9   74   59
722   37  123    6   35   29
'''

t90 = ascii.read(text)
t90.add_column(names,name='gal_name',index=0)

t90.add_row(['total',np.sum(t90['Nneb']),np.sum(t90['ovlp']),np.sum(t90['Nasc']),np.sum(t90['cont']),np.sum(t90['part']),np.sum(t90['1to1'])])
t90

here is the result
```        Nneb ovlp Nasc cont part 1to1
IC5332
  0 deg:  777  295  393   45  215  154
 90 deg:  783  144  378    4  131   81
NGC1087
  0 deg: 1006  299  487  280  137  182
 90 deg: 1005  139  447   51  126   68
NGC1365
  0 deg:  926  191  489  286   95  112
 90 deg:  908   66  493   25   61   46
NGC1385
  0 deg: 1029  310  525  341  139  133
 90 deg: 1029  193  521   98  173   82
NGC1433
  0 deg: 1024  355  494  134  254  250
 90 deg:  982   82  438    4   87   57
NGC1566
  0 deg: 2119  816 1578  707  579  373
 90 deg: 2138  463 1540   86  507  203
NGC2835
  0 deg: 1039  365  641  232  244  223
 90 deg: 1046  208  658   39  213  129
NGC4303
  0 deg: 2696  973 1735  853  584  402
 90 deg: 2691  625 1738  192  616  290
NGC4535
  0 deg: 1567  385  465  181  207  197
 90 deg: 1561   86  383    9   74   59
NGC7496
  0 deg:  743  163  264  185   54   96
 90 deg:  722   37  123    6   35   29```

In [ ]:
from astropy.table import hstack
tbl = join(t0,t90,keys='gal_name')

tbl['by_chance_part'] = 100*tbl['part_2']/tbl['Nasc_2'] / (tbl['part_1']/tbl['Nasc_1'])
tbl['by_chance_cont'] = 100*tbl['cont_2']/tbl['Nasc_2'] / (tbl['cont_1']/tbl['Nasc_1'])

tbl

In [ ]:
print(f"contained decrease by {100*(tbl['cont_1'][-1]-tbl['cont_2'][-1])/tbl['cont_1'][-1]:.1f} %")
print(f"partial decrease by {100*(tbl['part_1'][-1]-tbl['part_2'][-1])/tbl['part_1'][-1]:.1f} %")

In [ ]:
A_mask_total = []
A_FOV_total = []

for gal_name in tbl['gal_name'][:-1]:
    
    filename = next((data_ext/'MUSE'/'DR2.1'/'copt'/'MUSEDAP').glob(f'{gal_name}*.fits'))
    copt_res = float(filename.stem.split('-')[1].split('asec')[0])
    with fits.open(filename) as hdul:
        Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                        uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                        mask=np.isnan(hdul['HA6562_FLUX'].data),
                        meta=hdul['HA6562_FLUX'].header,
                        wcs=WCS(hdul['HA6562_FLUX'].header))

    filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
    with fits.open(filename) as hdul:
        nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
        nebulae_mask.data[nebulae_mask.data==-1] = np.nan
        
    A_mask = np.sum(~np.isnan(nebulae_mask.data))
    A_FOV  = np.sum(~np.isnan(Halpha.data))
    print(f'{gal_name}: {A_mask/A_FOV*100:.1f}%')
    
    A_mask_total.append(A_mask)
    A_FOV_total.append(A_FOV)
    
A_mask_total.append(np.sum(A_mask_total))
A_FOV_total.append(np.sum(A_FOV_total))
tbl['A_mask'] = A_mask_total 
tbl['A_FOV'] = A_FOV_total 

#print(f'Total: {A_mask_total/A_FOV_total*100:.1f}%')
    

## Ages of the association catalogue

In [ ]:
fig,axes=plt.subplots(nrows=2,ncols=3,figsize=(two_column,two_column*2/3))
axes = iter(axes.flatten())

version = 'v1p2'
for HSTband in ['nuv','v']:
    for scalepc in [16,32,64]:
        folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'
        lst = []
        for file in folder.glob(f'*{scalepc}pc_associations.fits'):
            gal_name = file.stem.split('_')[0]
            tbl = Table(fits.getdata(file,ext=1))
            tbl.add_column(gal_name,name='gal_name',index=0)
            lst.append(tbl)
        assoc_tmp = vstack(lst)
        
        with fits.open(basedir/'data'/'interim'/f'phangshst_associations_{HSTband}_ws{scalepc}pc_{version}.fits') as hdul:
            associations = Table(hdul[1].data)
        associations = join(associations,assoc_tmp,keys=['gal_name','assoc_ID'])

        
        ax = next(axes)

        tmp=associations[associations['1to1']]
        ax.hist(tmp['reg_dolflux_Age_MinChiSq'],bins=np.arange(0,20))
        ax.set_title(f'{HSTband}, {scalepc}pc, {np.mean(tmp["reg_dolflux_Age_MinChiSq"]):.1f} Myr')

plt.show()

## BPASS

https://flexiblelearning.auckland.ac.nz/bpass/8/files/bpassv2_1_manual.pdf

In [ ]:

def read_bpass(binary='bin',imf='imf135',upper_mass='300',metallicity='z014'):
    '''read the output from BPASS
    
    
    Parameters
    ----------
    
    binary : str (`bin` or `sin`)
        use binarys or single stars
        
    imf : str
        the slope of the IMF (the model)
        
    upper_mass : str
        mass of the most massive stars
    
    metallicity : str
    '''
    
    filename = basedir/'..'/'BPASS'/f'bpass_v2.2.1_{imf}_{upper_mass}'/binary/f'ionizing-{binary}-{imf}_{upper_mass}.{metallicity}.dat'

    if not filename.is_file():
        print('file does not exist')
        return filename
    
    units = [u.LogUnit(u.year),u.LogUnit(1/u.s),u.LogUnit(u.erg/u.s),u.LogUnit(u.erg/u.s/u.A),u.LogUnit(u.erg/u.s/u.A)]
    names = ['log_age','log_Q','log_Halpha','log_FUV','log_NUV']
    bpass = ascii.read(filename,names=names)
    bpass = QTable(bpass)

    for unit, col in zip(units,bpass.columns):
        bpass[col].unit = unit 
    for col in list(bpass.columns):
        if col.startswith('log'):
            bpass[col] = bpass[col].physical
            bpass.rename_column(col,col[4:])
    return bpass

bpass = read_bpass(metallicity='z020')

In [ ]:
fig,ax=plt.subplots()

ax.plot(bpass['age'],bpass['Halpha']/bpass['FUV'])
ax.set(xlim=[1e6,1e7],xlabel='age / Myr',ylabel=r'H$\alpha$ / FUV')
plt.show()

In [ ]:
from tqdm import tqdm

catalogue['Qpredicted'] = np.nan
HI_rate = bpass['Q'].value
time = bpass['age']
for row in tqdm(catalogue):
    idx = np.argmin(np.abs(time-row['age']*u.Myr))
    row['Qpredicted'] = HI_rate[idx] * row['mass'] / 1e6
    
catalogue['distance'] = np.nan
for gal_name in catalogue['gal_name']:
    distance = Distance(distmod=sample_table.loc[gal_name]['(m-M)'])
    catalogue['distance'][catalogue['gal_name']==gal_name] = distance
    
catalogue['L(Ha)'] = (catalogue['HA6562_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2 *4*np.pi*(catalogue['distance']*u.Mpc)**2).to(u.erg/u.s)
catalogue['Qobserved'] = 7.31e11*catalogue['L(Ha)']/u.erg
fesc_classic = (catalogue['Qpredicted']-catalogue['Qobserved'])/catalogue['Qpredicted']
catalogue['fesc'] = fesc_classic
catalogue['HA/FUV_corr'] = catalogue['HA/FUV'] / (1-fesc_classic)
catalogue['eq_width_corr'] = catalogue['eq_width'] / (1-fesc_classic)

print(f'fesc={np.nanmean(fesc_classic[fesc_classic>0]):.2f}+-{np.nanstd(fesc_classic[fesc_classic>0]):.2f} (from {np.sum(fesc_classic>0)} objects)')
print(f"{np.sum(fesc_classic<0)} of {len(catalogue)} ({np.sum(fesc_classic<0)/len(catalogue)*100:.1f}%) regions have negative fesc")

In [ ]:
# for objects with negative fesc we redo the analysis with age-age_err
Qpredict_new = []
for row in tqdm(catalogue):
    if row['fesc']<0:
        idx = np.argmin(np.abs(time-(row['age']-3*row['age_err'])*u.Myr))
        row['Qpredicted'] = ( HI_rate[idx] * row['mass'] / 1e6 )

fesc_classic = (catalogue['Qpredicted']-catalogue['Qobserved'])/catalogue['Qpredicted']
catalogue['fesc'] = fesc_classic
catalogue['HA/FUV_corr'] = catalogue['HA/FUV'] / (1-fesc_classic)
catalogue['eq_width_corr'] = catalogue['eq_width'] / (1-fesc_classic)

print(f'fesc={np.nanmean(fesc_classic[fesc_classic>0]):.2f}+-{np.nanstd(fesc_classic[fesc_classic>0]):.2f} (from {np.sum(fesc_classic>0)} objects)')
print(f"{np.sum(fesc_classic<0)} of {len(catalogue)} ({np.sum(fesc_classic<0)/len(catalogue)*100:.1f}%) regions have negative fesc")

In [ ]:
criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=10) 
#criteria &= (catalogue['overlap_neb']>0.1) 
criteria &= (catalogue['overlap_asc']==1) 
#criteria &= fesc>0
tmp = catalogue[criteria].copy()

print(f'fesc={np.nanmean(tmp[tmp["fesc"]>0]["fesc"]):.2f} (from {np.sum(criteria)} objects)')
print(f"{np.sum(tmp['fesc']<0)} of {len(tmp)} ({np.sum(tmp['fesc']<0)/len(tmp)*100:.1f}%) regions have negative fesc")

### Compare different stellar models/population synthesis

In [ ]:
bpass_z014 = read_bpass(metallicity='z014')
bpass_z008 = read_bpass(metallicity='z008')

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(nrows=3,figsize=(two_column,two_column),sharex=True)

ax1.plot(bpass['age']/1e6,bpass['Halpha'],label='BPASSv014')
#ax1.plot(bpass_z008['age']/1e6,bpass_z008['Halpha'],label='BPASSv008')
ax1.plot(cluster.ewidth['Time']/1e6,cluster.ewidth['Luminosity_H_A'],label='GENEVAv40')
ax1.legend()
ax1.set(xlim=[0,10],ylabel=r'$\mathrm{H}\alpha \,/\, \mathrm{erg}\ \mathrm{s}^{-1}$',
        yscale='log',ylim=[1e39,2e41])

ax2.plot(bpass['age']/1e6,bpass['FUV'],label='BPASSv014')
#ax2.plot(bpass_z008['age']/1e6,bpass_z008['FUV'],label='BPASSv008')
ax2.plot(cluster.ewidth['Time']/1e6,cluster.FUV['FUV'],label='GENEVAv40')
#ax2.legend()
ax2.set(xlim=[0,10],ylabel=r'$\mathrm{FUV}\,/\, \mathrm{erg}\ \mathrm{s}^{-1}\ \mathrm{\AA}^{-1}$',
       yscale='log',ylim=[8e37,3e39])

ax3.plot(bpass['age']/1e6,bpass['Halpha']/bpass['FUV'],label='BPASSv014')
#ax3.plot(bpass_z008['age']/1e6,bpass_z008['Halpha']/bpass_z008['FUV'],label='BPASSv008')
ax3.plot(cluster.ewidth['Time']/1e6,cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'],label='GENEVAv40')
#ax3.legend()
ax3.set(xlim=[0,10],xlabel='age / Myr',ylabel=r'$\mathrm{H}\alpha/\mathrm{FUV}$')

plt.subplots_adjust(hspace=0.)
plt.show()

## PCA analysis

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.decomposition import PCA

In [ ]:
# define the sample
criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=10) 
criteria &= (catalogue['overlap_neb']>0.1) 
criteria &= (catalogue['overlap_asc']>0.9) 
tmp = catalogue[criteria].copy()
tmp = tmp[['age','mass','EBV_balmer','density','temperature','met_scal','logq_D91','fesc']]

x = tmp.to_pandas()
# remove all columns with NaN
tmp = tmp[np.all(~np.isnan(x),axis=1).values]
x = x[np.all(~np.isnan(x),axis=1)]
x = StandardScaler().fit_transform(x)
#x = x[...,1:]

In [ ]:
normalised_data = pd.DataFrame(x,columns=[f'feature{i}' for i in range(x.shape[1])])

In [ ]:
pca_hiiregion = PCA(n_components=2)
principalComponents_hiiregion = pca_hiiregion.fit_transform(x)
principal_hiiregion_Df = pd.DataFrame(data = principalComponents_hiiregion
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
fig,ax=plt.subplots()

idx = tmp['age']<3
ax.scatter(principal_hiiregion_Df.loc[idx,'principal component 1'],
           principal_hiiregion_Df.loc[idx,'principal component 2'],color='tab:blue')

idx = tmp['age']>3
ax.scatter(principal_hiiregion_Df.loc[idx,'principal component 1'],
           principal_hiiregion_Df.loc[idx,'principal component 2'],color='tab:red')

ax.set(xlabel='PCA-1',ylabel='PCA-2')
plt.show()

## Pretty Pictures

### Plot regions (MUSE & HST over WFI)

In [ ]:
from astropy.nddata import Cutout2D
from cluster.regions import find_sky_region

sample = set([x.stem.split('_')[0].upper() for x in (data_ext/'HST'/'white_light').iterdir()])
sample = np.unique(sample_table['name'])
ncols = 4
nrows = int(np.ceil(len(sample)/ncols))

width = thesis_width
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows),facecolor='black')
axes_iter = iter(axes.flatten())

ax=next(axes_iter)
# plot the PHANGS logo in the top left axis
image = mpl.image.imread(str(basedir/'references'/'Logo_white.png'))
img = np.sum(image,axis=2)
img = np.pad(img,[(120,36),(25,25)],mode='constant')
ax.imshow(img,cmap=plt.cm.gray)
ax.axis('off')

hst_sample = []
for name in sorted(sample):
    
    print(name)
    
    filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{name.lower()}_f275w_v1_exp-drc-sci.fits'
    
    if filename.is_file():
        with fits.open(filename) as hdul:
            hst_whitelight = NDData(hdul[0].data,mask=hdul[0].data==0,meta=hdul[0].header,wcs=WCS(hdul[0].header))
            hst_whitelight.data[hst_whitelight.data==0] = np.nan
        hst_sample.append(name)

    
    filename = data_ext / 'MUSE' / 'DR2.1' / 'MUSEDAP' / f'{name}_MAPS.fits'
    with fits.open(filename) as hdul:
        Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                        uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                        mask=np.isnan(hdul['HA6562_FLUX'].data),
                        meta=hdul['HA6562_FLUX'].header,
                        wcs=WCS(hdul['HA6562_FLUX'].header))
        
    filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
    with fits.open(filename) as hdul:

        WFI = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=WCS(hdul[0].header))
        

    reg_muse_pix, reg_muse_sky = find_sky_region(Halpha.mask.astype(int),wcs=Halpha.wcs)
    if name in hst_sample:
        reg_hst_pix, reg_hst_sky = find_sky_region(hst_whitelight.mask.astype(int),wcs=hst_whitelight.wcs)
    
    WFI_cutout = Cutout2D(WFI.data,sample_table.loc[name]['SkyCoord'],size=8*u.arcmin,wcs=WFI.wcs)
    
    # project from muse to hst coordinates
    reg_muse_wfi = reg_muse_sky.to_pixel(WFI_cutout.wcs)
    if name in hst_sample:
        reg_hst_wfi  = reg_hst_sky.to_pixel(WFI_cutout.wcs)

    ax = next(axes_iter)

    # plot image
    norm = simple_norm(WFI_cutout.data,clip=False,percent=99)
    ax.imshow(WFI_cutout.data,norm=norm,cmap=plt.cm.gray,origin='lower')

    reg_muse_wfi.plot(ax=ax,ec='tab:red',label='MUSE',lw=0.3)
    if name in hst_sample:
        reg_hst_wfi.plot(ax=ax,ec='tab:orange',label='HST',lw=0.3)
    t = ax.text(0.05,0.91,name, transform=ax.transAxes,color='white',fontsize=7)
    #t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    ax.set_xticks([])
    ax.set_yticks([])

for i in range(nrows*ncols-len(sample)-1):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=-0.01,hspace=0.05)
plt.savefig(basedir/'reports'/'PHANGS_atlas.jpg',facecolor='white',dpi=400)

plt.show()

In [ ]:
from astropy.nddata import Cutout2D
from cluster.regions import find_sky_region


ncols = 5
nrows = int(np.ceil(len(muse_sample)/ncols))

width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for name in sorted(muse_sample):
    
    print(name)
    
    catalogue_file = basedir/'..'/'PNLF'/'data'/'catalogues'/f'{name}_nebulae.txt'
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
    catalogue=catalogue[catalogue['type']=='PN']
    
    filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
    with fits.open(filename) as hdul:

        WFI = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=WCS(hdul[0].header))
        
    
    WFI_cutout = Cutout2D(WFI.data,sample_table.loc[name]['SkyCoord'],size=5*u.arcmin,wcs=WFI.wcs)

    ax = next(axes_iter)

    # plot image
    norm = simple_norm(WFI_cutout.data,clip=False,percent=99)
    ax.imshow(WFI_cutout.data,norm=norm,cmap=plt.cm.gray,origin='lower')
    
    x,y = catalogue['SkyCoord'].to_pixel(WFI_cutout.wcs)
    ax.scatter(x,y,marker='o',ec='tab:red',fc='none',s=1,lw=0.2)
    
    t = ax.text(0.05,0.91,name, transform=ax.transAxes,color='black',fontsize=8)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    ax.set_xticks([])
    ax.set_yticks([])

for i in range(nrows*ncols-len(muse_sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=-0.01,hspace=0.05)
plt.savefig(basedir/'reports'/'all_objects_PN.pdf',dpi=600)
plt.show()

In [ ]:
from astropy.nddata import Cutout2D
from cluster.regions import find_sky_region


ncols = 5
nrows = int(np.ceil(len(muse_sample)/ncols))

width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for name in sorted(muse_sample):
    
    print(name)
    
    catalogue = filter_table(nebulae,gal_name=name)
    #catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
    
    filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
    with fits.open(filename) as hdul:
        WFI = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=WCS(hdul[0].header))
        
    
    WFI_cutout = Cutout2D(WFI.data,sample_table.loc[name]['SkyCoord'],size=5*u.arcmin,wcs=WFI.wcs)

    ax = next(axes_iter)

    # plot image
    norm = simple_norm(WFI_cutout.data,clip=False,percent=99)
    ax.imshow(WFI_cutout.data,norm=norm,cmap=plt.cm.gray,origin='lower')
    
    x,y = catalogue['SkyCoord'].to_pixel(WFI_cutout.wcs)
    ax.scatter(x,y,marker='o',ec='tab:blue',fc='none',s=1,lw=0.2)
    
    t = ax.text(0.05,0.91,name, transform=ax.transAxes,color='black',fontsize=8)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    ax.set_xticks([])
    ax.set_yticks([])

for i in range(nrows*ncols-len(muse_sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=-0.01,hspace=0.05)
plt.savefig(basedir/'reports'/'all_objects_nebulae.pdf',dpi=600)
plt.show()

### Map of the sky

In [ ]:
from astropy.coordinates import Distance

sample = ascii.read(basedir/'..'/'pnlf'/'data'/'interim'/'sample.txt')
sample['SkyCoord'] = SkyCoord(sample['R.A.'],sample['Dec.'])
sample['d/Mpc'] = Distance(distmod=sample['(m-M)'])

ra = sample['SkyCoord'].ra
ra = ra.wrap_at(180*u.degree)
dec = sample['SkyCoord'].dec

In [ ]:
#mpl.use('pdf')
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(ra.radian,dec.radian,marker='.')
#ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(False)
ax.set_xticklabels([])
ax.set_yticklabels([])

d_max = np.max(sample['d/Mpc'])
width0 = 0.3 # in radian
for x,y,name,distance in zip(ra,dec,sample['name'],sample['d/Mpc']):
            
    filename = next((data_ext/'MUSE'/'DR2.1'/'copt'/'MUSEDAP').glob(f'{name}*.fits'))
    copt_res = float(filename.stem.split('-')[1].split('asec')[0])
    with fits.open(filename) as hdul:
        Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                        uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                        mask=np.isnan(hdul['HA6562_FLUX'].data),
                        meta=hdul['HA6562_FLUX'].header,
                        wcs=WCS(hdul['HA6562_FLUX'].header))
        
    img_width,img_height = Halpha.data.shape
    width  = width0 * distance / d_max
    height = width/img_width*img_height
    bounds = [x.radian-width/2,y.radian-height/2,width,height]

    ax_img = ax.inset_axes(bounds,transform=ax.transData)
    norm = simple_norm(Halpha.data,clip=False,percent=99)
    ax_img.imshow(Halpha.data,cmap=plt.cm.gray_r,norm=norm,origin='lower')
    ax_img.axis('off')
    ax_img.set_title(name)
        
    #ax.annotate(s,(x.radian,y.radian),xycoords='data',size='x-small')

fig.savefig("sky_map.pdf",dpi=600)
    

## Others

### create a LaTeX table for the journal

In [ ]:
catalogue['RA_asc'],catalogue['DEC_asc'] = zip(*[x.split(' ') for x in catalogue['SkyCoord_asc'].to_string(style='hmsdms',precision=2)])
catalogue['RA_neb'],catalogue['DEC_neb'] = zip(*[x.split(' ') for x in catalogue['SkyCoord_neb'].to_string(style='hmsdms',precision=2)])
catalogue['logmass'] = np.log10(catalogue['mass'])
catalogue['logmass_err'] = catalogue['mass_err'] / catalogue['mass']
catalogue['dots'] = '$\hdots$'

In [ ]:
# one large table
columns = ['gal_name','region_ID','RA_neb','DEC_neb','[SIII]/[SII]','[SIII]/[SII]_err','Delta_met_scal','met_scal_err','HA/FUV','HA/FUV_err','eq_width','EBV_balmer','EBV_balmer_err','overlap_neb','assoc_ID','RA_asc','DEC_asc','age','age_err','mass','mass_err','EBV_stars','EBV_stars_err','overlap_asc']
# create LaTeX table for a single galaxy (for paper)
tmp = catalogue[columns]


for col in columns:
    if not col.startswith('RA') and not col.startswith('DEC') and col!='gal_name':
        tmp[col].info.format = '%.2f'

ascii.write(tmp[-10:],sys.stdout,Writer=ascii.Latex,overwrite=True,exclude_names=['x','y','fwhm'])


In [ ]:
# or two separate tables
# for the associations
columns = ['gal_name','assoc_ID','RA_asc','DEC_asc','age','age_err','logmass','logmass_err','EBV_stars','EBV_stars_err','overlap_asc','dots']
# create LaTeX table for a single galaxy (for paper)
tmp = catalogue[columns]


for col in columns:
    if not col.startswith('RA') and not col.startswith('DEC') and col!='gal_name' and col!='dots':
        tmp[col].info.format = '%.2f'
tmp['assoc_ID'].info.format = '%.0f'
        
ascii.write(tmp[:10],sys.stdout,Writer=ascii.Latex,overwrite=True,exclude_names=['x','y','fwhm'])


# and now for the HII regions
columns = ['dots','region_ID','RA_neb','DEC_neb','[SIII]/[SII]','[SIII]/[SII]_err','Delta_met_scal','met_scal_err','HA/FUV','HA/FUV_err','eq_width','EBV_balmer','EBV_balmer_err','overlap_neb']
# create LaTeX table for a single galaxy (for paper)
tmp = catalogue[columns]

for col in columns:
    if not col.startswith('RA') and not col.startswith('DEC') and col!='gal_name' and col!='dots':
        tmp[col].info.format = '%.2f'
tmp['region_ID'].info.format = '%.0f'

ascii.write(tmp[:10],sys.stdout,Writer=ascii.Latex,overwrite=True,exclude_names=['x','y','fwhm'])




## Playground

### Density & Temperature

In [ ]:
RSII = nebulae['SII6716_FLUX_CORR'] / nebulae['SII6730_FLUX_CORR']
RSII_ERR = RSII * np.sqrt( (nebulae['SII6716_FLUX_CORR_ERR']/nebulae['SII6716_FLUX_CORR'])**2 + (nebulae['SII6730_FLUX_CORR_ERR']/nebulae['SII6730_FLUX_CORR'])**2 )
dif = (1.46-RSII)/(RSII_ERR*1.43)
print(np.sum(dif>3))

#nebulae['density'][dif<3] = np.nan

fig,(ax1,ax2)=plt.subplots(figsize=(two_column,two_column),nrows=2)

ax1.hist(RSII,bins=np.linspace(1.1,1.8,40))
ax1.axvline(1.46,color='black')
ax1.set(yscale='log',xlim=[1.1,1.8],ylim=[10,None],xlabel=r'$R_\mathrm{[SII]}$')


ax2.hist(dif,bins=np.linspace(0,20,40))
ax2.axvline(3,color='black')
ax2.set(yscale='log',xlim=[0,20],xlabel=r'$(1.46-R_\mathrm{[SII]}) / \delta R_\mathrm{[SII]}$')

plt.show()

In [ ]:
Rlim = lambda Te: 1.49-3.94e-6*Te
Rlim(12e3)


define the sample

In [ ]:
tmp = nebulae.copy()
tmp['RSII'] = tmp['SII6716_FLUX_CORR'] / tmp['SII6730_FLUX_CORR']
tmp['RSII_ERR'] = tmp['RSII'] * np.sqrt( (tmp['SII6716_FLUX_CORR_ERR']/tmp['SII6716_FLUX_CORR'])**2 + (tmp['SII6730_FLUX_CORR_ERR']/tmp['SII6730_FLUX_CORR'])**2 )

# make some S/N cuts
S2N = 5
#mask =  (tmp['NII5754_FLUX_CORR_REFIT']>S2N*tmp['NII5754_FLUX_CORR_REFIT_ERR']) & (tmp['NII6583_FLUX_CORR']>S2N*tmp['NII6583_FLUX_CORR_ERR'])
#mask &= (tmp['SII6730_FLUX_CORR']>S2N*tmp['SII6730_FLUX_CORR_ERR']) & (tmp['SII6716_FLUX_CORR']>S2N*tmp['SII6716_FLUX_CORR_ERR'])
mask = (1.46-tmp['RSII'])/(tmp['RSII_ERR']*1.43)>3

tmp =tmp[mask]
print(f'{len(tmp)} objects in sample')

In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2/1.618))

ax1.scatter(nebulae['EW_HA'],nebulae['SII6716_FLUX_CORR']/nebulae['SII6730_FLUX_CORR'])
corner_binned_stat(nebulae['EW_HA'],nebulae['SII6716_FLUX_CORR']/nebulae['SII6730_FLUX_CORR'],ax=ax1)
ax1.set(xlabel='EW(Ha)',ylabel='RSII',xlim=[0,450])
ax2.scatter(tmp['EW_HA'],densities)
corner_binned_stat(tmp['EW_HA'],densities,ax=ax2)
ax2.set(xlabel='EW(Ha)',ylabel='density',xlim=[0,450])

#ax.set(yscale='log',xlabel='[SII]6716/6730')
plt.show()

In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2/1.618))

ax1.scatter(tmp['EW_HA'],tmp['SII6716_FLUX_CORR']/tmp['SII6730_FLUX_CORR'])
corner_binned_stat(tmp['EW_HA'],tmp['SII6716_FLUX_CORR']/tmp['SII6730_FLUX_CORR'],ax=ax1)
ax1.set(xlabel='EW(Ha)',ylabel='RSII',xlim=[0,450])
ax2.scatter(tmp['EW_HA'],densities)
corner_binned_stat(tmp['EW_HA'],densities,ax=ax2)
ax2.set(xlabel='EW(Ha)',ylabel='density',xlim=[0,450])

#ax.set(yscale='log',xlabel='[SII]6716/6730')
plt.show()

In [ ]:
import pyneb as pn

den = 100
tem = 8e3

N2 = pn.Atom('N', 2)
temperatures = N2.getTemDen(int_ratio=tmp['NII5754_FLUX_CORR_REFIT']/tmp['NII6583_FLUX_CORR'], den=den, wave1=5755, wave2=6584)

S2 = pn.Atom('S', 2)
densities = S2.getTemDen(int_ratio=tmp['SII6716_FLUX_CORR']/tmp['SII6730_FLUX_CORR'], tem=tem, wave1=6716, wave2=6730)

diags = pn.Diagnostics()
Te,ne = diags.getCrossTemDen('[NII] 5755/6548', '[SII] 6731/6716', 
                     tmp['NII5754_FLUX_CORR_REFIT']/tmp['NII6583_FLUX_CORR']/0.34,
                     tmp['SII6730_FLUX_CORR']/tmp['SII6716_FLUX_CORR'])


write to nebula catalogue

In [ ]:
nebulae['density'] = np.nan
nebulae['density'][mask] = ne

nebulae['temperature'] = np.nan
nebulae['temperature'][mask] = Te

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

ax.hist(temperatures,bins=np.linspace(5.5e3,1.1e4,24),histtype='step',label=r'$\mathrm{density}=100\,\mathrm{cm}^{-1}$')
ax.hist(Te,bins=np.linspace(5.5e3,1.1e4,24),histtype='step',label='crossTemDen')

#ax.axvline(8e3,color='gray',ls='--')
ax.axvline(np.nanmean(Te),color='gray',label='mean')
ax.set(xlabel='Temperature / K',ylabel='N')
#ax.set_title(f'$T_{{mean}} = {np.nanmean(Te):.1f}$ K')
ax.legend()
plt.savefig(basedir/'reports'/'temperature_hist.pdf',dpi=400)
plt.show()

print(f'Tmean={np.nanmean(Te):.1f}')

In [ ]:
import pyneb as pn

tem = 8e3

ratio = np.linspace(0.5,1.5,100)

S2 = pn.Atom('S', 2)

fig,(ax1,ax2)=plt.subplots(figsize=(thesis_width,thesis_width/1.5),
                           nrows=2,sharex=True,gridspec_kw={'height_ratios':[1.618,1]})

ax1.hist(nebulae['SII6716_FLUX_CORR']/nebulae['SII6730_FLUX_CORR'],
         bins=np.linspace(0.5,2.5,41),histtype='step',color=tab10[2])
ax1.hist(nebulae['SII6716_FLUX_CORR']/nebulae['SII6730_FLUX_CORR'],
         bins=np.linspace(0.5,2.5,41),histtype='bar',color=tab10[2],alpha=0.2)

ax1.axvline(1.45,color=tab10[0])
ax1.set(yscale='log',ylabel=r'$N$',ylim=[2,None])

for tem in [6000,12000]:
    densities = S2.getTemDen(int_ratio=ratio, tem=tem, wave1=6716, wave2=6731)
    ax2.plot(ratio,densities,label=f'{tem} K',color=tab10[1],zorder=0)
ax2.axvline(1.45,color=tab10[0])
ax2.text(0.12,0.85,'12,000 K',rotation=-15,transform=ax2.transAxes,ha='left',va='top',color=tab10[1])
ax2.text(0.15,0.54,'6,000 K',rotation=-15,transform=ax2.transAxes,ha='left',va='top',color=tab10[1])
    
ax2.arrow(0.475,0.5,0.2,0,transform=ax2.transAxes,head_width=0.08,head_length=0.04,color=tab10[0])
ax2.text(0.5,0.55,'low density limit',transform=ax2.transAxes,ha='left',va='bottom',fontsize=10,color=tab10[0])

ax2.set(yscale='log',xlabel=r'$F_{[\mathrm{S}\,\tiny{\textsc{ii}}] \lambda 6716}\,/\,F_{[\mathrm{S}\,\tiny{\textsc{ii}}] \lambda 6731}$',ylabel='density / cm$^{-1}$',
        xlim=[0.5,2.5],ylim=[20,2e4])
ax2.set_yticks([1e2,1e3,1e4])

plt.subplots_adjust(hspace=0.02)
plt.savefig(basedir/'reports'/'density.pdf',dpi=300)
plt.show()


In [ ]:
densities1 = S2.getTemDen(int_ratio=ratio, tem=6000, wave1=6716, wave2=6731)
densities2 = S2.getTemDen(int_ratio=ratio, tem=12000, wave1=6716, wave2=6731)

In [ ]:
def getDen(tmp,temp=1e4,sample_size = 1000):
    '''compute Density with uncertainties
    

    '''
    S2 = pn.Atom('S', 2)

    SII6730 = np.random.normal(loc=tmp['SII6730_FLUX_CORR'],scale=tmp['SII6730_FLUX_CORR_ERR'],size=(sample_size,len(tmp)))
    SII6716 = np.random.normal(loc=tmp['SII6716_FLUX_CORR'],scale=tmp['SII6716_FLUX_CORR_ERR'],size=(sample_size,len(tmp)))

    # calculcate for an array of values and use the std as the uncertainty
    densities   = S2.getTemDen(int_ratio=SII6716/SII6730, tem=temp, wave1=6717, wave2=6731)
    density_err = np.nanstd(densities,axis=0)
    
    density   = S2.getTemDen(int_ratio=tmp['SII6716_FLUX_CORR']/tmp['SII6730_FLUX_CORR'], 
                             tem=temp, wave1=6717, wave2=6731)

    
    return density,density_err

density1,density_err1 = getDen(tmp[:10],temp=8e3,sample_size=1000)
density2,density_err2 = getDen(tmp[:10],temp=8e3,sample_size=100)

In [ ]:
S2.getTemDen?

In [ ]:
ratios = tmp['SII6716_FLUX_CORR']/tmp['SII6730_FLUX_CORR']
density_8k = S2.getTemDen(int_ratio=ratios, tem=8e3, wave1=6717, wave2=6731)
density_10k = S2.getTemDen(int_ratio=ratios, tem=1e4, wave1=6717, wave2=6731)
density_12k = S2.getTemDen(int_ratio=ratios, tem=1.2e4, wave1=6717, wave2=6731)

In [ ]:
plt.scatter(tmp['SII6716_FLUX_CORR']/tmp['SII6716_FLUX_CORR_ERR'],density/density_err)
plt.axvline(100)
plt.axhline(3)

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))

lim = (0,100)
ax.errorbar(density,density,yerr=density_err,xerr=density_err,fmt='o',color='gray',zorder=0)
ax.scatter(density_8k,density_10k,label='8k vs 10k')
ax.scatter(density_8k,density_12k,label='8k vs 12k')
ax.scatter(density_10k,density_12k,label='10k vs 12k')
ax.legend()
ax.plot(lim,lim,color='black')
ax.set(xlim=lim,ylim=lim,xlabel='density',ylabel='density')

plt.show()

### GMC sep


In [ ]:
from tqdm import tqdm
from astropy.coordinates import match_coordinates_sky

def get_value(matrix, index, default_value=np.nan):
    '''
    The `to_pixel` method returns the x,y coordinates. However in the 
    image they correspond to img[y,x]
    '''
    result = np.zeros(len(index))+default_value
    mask = (index[:,1] < matrix.shape[0]) & (index[:,0] < matrix.shape[1])
    mask &= (index[:,1] >= 0) & (index[:,0] >=0)

    valid = index[mask]
    result[mask] = matrix[valid[:,1], valid[:,0]]
    return result


for gal_name in np.unique(nebulae['gal_name']):

    # first the CO
    co_filename = data_ext/'ALMA'/'v4p0'/f'{gal_name.lower()}_12m+7m+tp_co21_broad_tpeak.fits'
    if not co_filename.is_file():
        print(f'no ALMA CO for {gal_name}')
    else:
        with fits.open(co_filename) as hdul:
            CO = NDData(data=hdul[0].data,
                        meta=hdul[0].header,
                        wcs=WCS(hdul[0].header))


    # and then the GMC catalogue
    gmc_directory  = data_ext/'Products'/'GMC'/'matched'
    gmc_resolution = '150pc'
    gmc_filename = [x for x in (gmc_directory/gmc_resolution).iterdir() if gal_name.lower() in x.stem]
    if len(gmc_filename)==0:
        print(f'no GMC catalogue for {gal_name}')
    else:
        gmc_filename = gmc_filename[0]
        with fits.open(gmc_filename) as hdul:
            GMC = Table(hdul[1].data)
        GMC['SkyCoord'] = SkyCoord(GMC['XCTR_DEG']*u.deg,GMC['YCTR_DEG']*u.deg)
        
        N_HII = np.sum(nebulae['gal_name']==gal_name)
        points = np.random.uniform(low=(0,0),high=CO.data.shape,size=(2*N_HII,2))

        value = get_value(CO.data,points.astype(int))
        x,y = points[~np.isnan(value)][:N_HII].T
        coords = SkyCoord.from_pixel(x,y,wcs=CO.wcs)
        
        idx,sep,_=match_coordinates_sky(coords,GMC['SkyCoord'])
        
        sep_arcsec = np.mean(sep).to(u.arcsec)
        sep_pc  = sep_arcsec.to(u.radian).value * sample_table.loc[gal_name]['distance']*1e6
        
        print(f'{gal_name}: {sep_arcsec:.2f}, {sep_pc:.2f}')

### Measure DIG

In [ ]:
def measure_dig(data,mask,label,position,factor=1,max_iter=10,size=32,plot=False):
    '''measure the diffuse ionized gas around an HII-region'''
    
    cutout_mask = Cutout2D(mask.data,position,size=(size,size),mode='partial',fill_value=np.nan)
    cutout_data = Cutout2D(data.data,position,size=(size,size),mode='partial',fill_value=np.nan)
    
    area_mask  = np.sum(cutout_mask.data==label)
    input_mask = cutout_mask.data==label
    
    n_iter = 0
    while True:
        n_iter+=1
        boundaries = find_boundaries(input_mask,mode='outer')
        input_mask |=boundaries
        area_boundary = np.sum(input_mask & np.isnan(cutout_mask.data)) 
        if area_boundary > factor*area_mask or n_iter>max_iter: break
            
    if plot:
        fig,ax=plt.subplots(figsize=(5,5))
        ax.imshow(cutout_mask.data,origin='lower')
        mask = np.zeros((*cutout_mask.shape,4))
        mask[input_mask & np.isnan(cutout_mask.data),:] = (1,0,0,0.5)
        ax.imshow(mask,origin='lower')
        plt.show()
        
    #if np.sum(boundaries & np.isnan(cutout_mask.data))==0:
    #    print(f'no boundaries for {label}')
    dig = cutout_data.data[input_mask & np.isnan(cutout_mask.data)]

    return np.median(dig),np.mean(dig),np.sum(dig)



### Old and young populations

In [ ]:
from regions import read_ds9

tmp = catalogue[catalogue['gal_name']=='NGC1365']

# filter image with uncertainties
filename = data_ext / 'HST' / 'filterImages' / f'NGC1365_uvis_f275w_exp_drc_sci.fits'
with fits.open(filename) as hdul:
    F275 = NDData(hdul[0].data,
                  mask=hdul[0].data==0,
                  meta=hdul[0].header,
                  wcs=WCS(hdul[0].header))

associations,associations_mask = read_associations(folder=data_ext/'HST',target='NGC1365',scalepc=32)
reg_young, reg_old = read_ds9(basedir/'data'/'tmp'/'young_old.reg')

# the catalogue
young_sample = tmp[reg_young.contains(tmp['SkyCoord_asc'],wcs=associations_mask.wcs)]
young_sample = young_sample[young_sample['age']<20]
old_sample = tmp[reg_old.contains(tmp['SkyCoord_asc'],wcs=associations_mask.wcs)]
old_sample = old_sample[old_sample['age']>20]

In [ ]:
# the maps
young = np.isin(associations_mask.data,associations[associations['age']<10]['assoc_ID'])
old   = np.isin(associations_mask.data,associations[associations['age']>10]['assoc_ID'])

young = young.astype(float)
young[young==0] = np.nan

old = old.astype(float)
old[old==0] = np.nan

In [ ]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=F275.wcs)

norm = simple_norm(F275.data,clip=False,percent=99)
ax.imshow(F275.data,norm=norm,origin='lower',cmap=plt.cm.Greys,alpha=0.5)
ax.imshow(old,vmin=0,vmax=1,cmap=plt.cm.Reds,alpha=0.8)
ax.imshow(young,vmin=0,vmax=1,cmap=plt.cm.Blues,alpha=0.8)

reg_young_pix = reg_young.to_pixel(F275.wcs)
reg_old_pix = reg_old.to_pixel(F275.wcs)

ax.add_artist(reg_young_pix.as_artist())
ax.add_artist(reg_old_pix.as_artist())


ax.set(xlim=[2000,5000],ylim=[3000,5000])
plt.show()

In [ ]:
xlim = [0,10]


fig,ax=plt.subplots(figsize=(single_column,single_column))

sc = ax.scatter(young_sample['EBV_stars'],young_sample['EBV_balmer'],color='g',label='upper arm')
sc = ax.scatter(old_sample['EBV_stars'],old_sample['EBV_balmer'],color='m',label='lower arm')
ax.legend()
ax.plot([0,1],[0,2],color='black')
ax.plot([0,2],[0,2],color='black')
#fig.colorbar(sc,label='age / Myr',cax=cax)
ax.set(xlim=[0,0.7],ylim=[0,0.7],xlabel='E(B-V) stars',ylabel='E(B-V) Balmer')

plt.show()

### Deproject radii

In [ ]:
from pnlf.auxiliary import project

def r25(name,table):
    '''calculate deprojected r25
    
    '''

    # get the pixel position of the centre
    with fits.open(data_ext/'MUSE_DR2.1'/'MUSEDAP'/f'{name}_MAPS.fits') as hdul:
        wcs = WCS(hdul['FLUX'].header)
    centre = sample_table.loc[name]['SkyCoord']
    x_cen,y_cen = centre.to_pixel(wcs)
    
    pa  = sample_table.loc[name]['posang']
    inc = sample_table.loc[name]['Inclination']
    r25 = sample_table.loc[name]['r25']*u.arcmin
    
    # deproject
    x_depr,y_depr = project(table['x_neb']-x_cen,table['y_neb']-y_cen,pa,inc)
    skycoord_depr = SkyCoord.from_pixel(x_depr+x_cen,y_depr+y_cen,wcs)
    
    # separation to centre
    sep = skycoord_depr.separation(centre)
    
    return (sep/r25).decompose()

name = 'NGC0628'

# the catalogue with the positions
tmp = nebulae[nebulae['gal_name']==name]

r25(name,tmp)

### Look at units of FUV and Halpha

In [ ]:
from starburst import Cluster, find_model, make_folder, list_available_models

cluster = Cluster(stellar_model='GENEVAv40',metallicity=0.014)
cluster.measure_FUV()

In [ ]:
from speclite.filters import FilterResponse, load_filters, plot_filters

response_curve = ascii.read(basedir/'data'/'external'/'astrosat_response_curve.txt',
                                     names=['wavelength','EA','Filter'])

F148W_mask = response_curve['Filter']=='F148W'
F148W_lam = response_curve['wavelength'][F148W_mask]*u.angstrom
F148W_res = response_curve['EA'][F148W_mask] / max(response_curve['EA'][F148W_mask])
F148W = FilterResponse(F148W_lam,F148W_res,meta=dict(group_name='Astrosat',band_name='F148W'))

F154W_mask = response_curve['Filter']=='F154W'
F154W_lam  = response_curve['wavelength'][F154W_mask]*u.angstrom
F154W_res  = response_curve['EA'][F154W_mask] / max(response_curve['EA'][F154W_mask])
F154W = FilterResponse(F154W_lam,F154W_res,meta=dict(group_name='Astrosat',band_name='F154W'))

astrosat_filter = load_filters('Astrosat-F148W', 'Astrosat-F154W')
plot_filters(astrosat_filter)

In [ ]:
age = 1e6 * u.yr

# find closest availalbe age
ages=cluster.spectrum['Time']
age =ages[np.argmin(np.abs(ages-age))]

wavelength = cluster.spectrum[cluster.spectrum['Time']==age]['Wavelength']
spectrum   = cluster.spectrum[cluster.spectrum['Time']==age]['Total']

ages=cluster.uvline['Time']
age =ages[np.argmin(np.abs(ages-age))]
wavelength_uv = cluster.uvline[cluster.uvline['Time']==age]['Wavelength']
spectrum_uv   = cluster.uvline[cluster.uvline['Time']==age]['lum']

fig,ax=plt.subplots()
ax.plot(wavelength,spectrum,color='tab:red')
ax.plot(wavelength_uv,spectrum_uv,color='tab:blue')
ax.set(xlim=[1000,2000],yscale='log',ylim=[1e38,1e40],
       xlabel='wavelength / AA',ylabel='erg / s / AA')

plt.show()

In [ ]:
# compare different methods to calculate FUV from starburst99
age = np.unique(cluster.uvline['Time'])[4]

wavelength = cluster.uvline['Wavelength'][cluster.uvline['Time']==age]
spectrum   = cluster.uvline['lum'][cluster.uvline['Time']==age]
print(f'uvline,   filter: {np.mean(np.interp(wavelength,F148W_lam,F148W_res)*spectrum):.2g}')

mask = (wavelength>1250*u.angstrom) & (wavelength<1800*u.angstrom)
print(f'uvline,   range:  {np.mean(spectrum[mask]):.2g}')

wavelength = cluster.spectrum['Wavelength'][cluster.spectrum['Time']==age]
spectrum   = cluster.spectrum['Total'][cluster.spectrum['Time']==age]
print(f'spectrum, filter: {np.mean(np.interp(wavelength,F148W_lam,F148W_res)*spectrum):.2g}')

mask = (wavelength>1250*u.angstrom) & (wavelength<1800*u.angstrom)
print(f'spectrum, range:  {np.mean(spectrum[mask]):.2g}')


In [ ]:
from starburst import Cluster, find_model, make_folder, list_available_models

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

for stellar_model in [41,42,43,44,45]:
    cluster = Cluster(stellar_model=stellar_model)
    cluster.measure_FUV()
    _,_,metallicity,_ = find_model(stellar_model)
    
    ax.plot(cluster.ewidth['Time']/1e6,
            cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'],
            label=f'Z={metallicity}')

ax.legend()
ax.set(xlim=[0,10],xlabel='age / Myr',ylabel=r'H$\alpha$ / FUV')
#ax.set_title('Geneva Tracks with Rotation')

plt.savefig(basedir/'reports'/'HaFUV_starburst99_padova.pdf',dpi=400)
plt.savefig(basedir/'reports'/'HaFUV_starburst99_padova.png',dpi=400)

plt.show()

In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,0.75*two_column/2))

ax1.plot(cluster.ewidth['Time']/1e6,cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'])
ax1.set(xlim=[0,10],xlabel='age / Myr',ylabel=r'H$\alpha$ / FUV')

ax2.plot(cluster.ewidth['Time']/1e6,cluster.ewidth['eq_width_H_A'])
ax2.set(xlim=[0,10],xlabel='age / Myr',ylabel=r'EW(H$\alpha$)')
plt.tight_layout()

#plt.savefig(basedir/'reports'/'HaFUV.pdf')
plt.show()

In [ ]:
age = 1e6 * u.yr

# find closest availalbe age
ages=cluster.spectrum['Time']
age =ages[np.argmin(np.abs(ages-age))]

wavelength = cluster.spectrum[cluster.spectrum['Time']==age]['Wavelength']
spectrum   = cluster.spectrum[cluster.spectrum['Time']==age]['Total']

fig,ax=plt.subplots()
ax.plot(wavelength,spectrum)
ax.set(xlim=[500,8000],yscale='log',ylim=[1e35,1e40])

ax2 = ax.twinx()
ax2.plot(response_curve['lam'],response_curve['r'],color='grey')

plt.show()

### 2D density histogram

In [ ]:
from scipy.stats import binned_statistic_2d
from scipy.interpolate import interpn

x = np.array([0.5,0.3,0.5,3.5])
y = np.array([0.5,0.6,2.5,1.5])
z = np.array([7.2,7.8,6.5,7.9])

xlim,ylim=[0,4],[0,4]

def hist_scatter(x,y,z,xlim,ylim):
    
    hist , x_e, y_e    = np.histogram2d(x,y, bins=4,range =[xlim,ylim], density=True)
    mean , x_e, y_e, _ = binned_statistic_2d(x,y,z,bins=4,range = [xlim,ylim])
    z = interpn((0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ),mean,np.vstack([x,y]).T,method="nearest",bounds_error=False)

    fig,ax=plt.subplots()
    cmap = plt.cm.get_cmap('gray_r',4)
    im = ax.imshow(mean.T,origin='lower',cmap=cmap,extent=[*xlim,*ylim],vmin=0,vmax=2)
    sc=ax.scatter(x,y,c=z,cmap=plt.cm.viridis,vmin=6,vmax=8)
    fig.colorbar(sc)
    ax.set(xlim=xlim,ylim=ylim)
    plt.show()
    
hist_scatter(x,y,z,xlim,ylim)

### Measure NUV

In [ ]:
from astropy.nddata import InverseVariance

gal_name = 'NGC2835'

filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
error_file = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_err-drc-wht.fits'

if not filename.is_file():
    print(f'no NUV data for {gal_name}')
else:
    with fits.open(filename) as hdul:
        F275 = NDData(hdul[0].data,
                        mask=hdul[0].data==0,
                        meta=hdul[0].header,
                        wcs=WCS(hdul[0].header))
        with fits.open(error_file) as hdul:
            F275.uncertainty = InverseVariance(hdul[0].data)

In [ ]:
from cluster.io import read_associations

HSTband = 'nuv'

NUV_fluxes = {}
NUV_fluxes_err = {}
for scalepc in [8,16,32,64]:
    print(f'working on {scalepc}pc')
    associations, associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',
                                                        target=gal_name.lower(),scalepc=scalepc,
                                                        HSTband='nuv',version='v1p2',data='all')

    # measure flux in mask and covert to physical units
    std_err_map = np.sqrt(1/F275.uncertainty.array)

    NUV = [np.sum(F275.data[associations_mask.data==assoc_ID]) for assoc_ID in associations['assoc_ID']]
    NUV_err = [np.sqrt(np.sum(std_err_map[associations_mask.data==assoc_ID]**2)) for assoc_ID in associations['assoc_ID']]

    NUV_mJy = 1e3*np.array(NUV)* F275.meta['PHOTFNU']*u.mJy
    NUV_mJy_err = 1e3*np.array(NUV_err)* F275.meta['PHOTFNU']*u.mJy

    NUV_flam = np.array(NUV)* F275.meta['PHOTFLAM']
    NUV_flam_err = np.array(NUV_err)* F275.meta['PHOTFLAM']
    
    NUV_fluxes[scalepc] = NUV_mJy
    NUV_fluxes_err[scalepc] = NUV_mJy_err

In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))

ax1.scatter(associations['NUV_dolflux_mjy'],NUV_mJy,s=1)
ax1.plot([5e-5,1],[5e-5,1],color='black')
ax1.set(xlim=[5e-5,1e0],ylim=[5e-5,1e0],xscale='log',yscale='log',
        xlabel='NUV / mJy from catalogue',ylabel='NUV / mJy from image')
ax1.set_title(r'using \texttt{PHOTFNU}')


# we are missing a factor 100 in the untis here
ax2.scatter(associations['NUV_FLUX'],NUV_flam,s=1)
ax2.plot([5e-19,5e-13],[5e-19,5e-13],color='black')
ax2.set(xlim=[5e-19,5e-15],ylim=[5e-19,5e-15],xscale='log',yscale='log',
        xlabel=r'NUV / erg s$^{-1}$ cm$^{-2}$ \AA$^{-1}$ from catalogue',ylabel=r'NUV / erg s$^{-1}$ cm$^{-2}$ \AA$^{-1}$ from image')
ax2.set_title(r'using \texttt{PHOTFLAM}')

#fig.suptitle(f'{gal_name}, NUV (F275W) for {scalepc}pc associations')
plt.tight_layout()

plt.savefig(basedir/'reports'/f'remeasure_NUV_{gal_name}_{scalepc}pc.pdf')

plt.show()

In [ ]:
(F275.meta['PHOTFNU']*u.Jy).to(u.erg/u.s/u.cm**2/u.Angstrom,equivalencies=u.spectral_density(2704*u.AA))

In [ ]:
F275.meta['PHOTFLAM']*u.erg/u.s/u.cm**2/u.Angstrom

In [ ]:
fig,axes=plt.subplots(ncols=4,figsize=(1.5*two_column,two_column/4*1.5))

for scalepc,ax in zip([8,16,32,64],axes):
    
    associations = read_associations(folder=data_ext/'Products'/'stellar_associations',
                                     target=gal_name.lower(),scalepc=scalepc,
                                     HSTband='nuv',version='v1p2',data='catalogue')
    
    dif = np.mean((associations['NUV_dolflux_mjy']-1.67*NUV_fluxes[scalepc].value)/associations['NUV_dolflux_mjy'])
    ax.scatter(associations['NUV_dolflux_mjy'],1.67*NUV_fluxes[scalepc],s=1)
    ax.plot([5e-5,1],[5e-5,1],color='black')
    ax.set(xlim=[5e-5,1e0],ylim=[5e-5,1e0],xscale='log',yscale='log',
            xlabel='NUV / mJy from catalogue',ylabel='NUV / mJy from image')
    ax.set_title(f'{scalepc}pc ({100*dif:.2f}\% smaller)')


#fig.suptitle(f'{gal_name}, NUV (F275W) for {scalepc}pc associations')
plt.tight_layout()

plt.savefig(basedir/'reports'/f'remeasure_NUV_all_resolutions_{gal_name}.pdf')

plt.show()

now measure FUV inside the nebula mask

In [ ]:
from dust_extinction.parameter_averages import O94, CCM89
from astropy.nddata import NDData, StdDevUncertainty, InverseVariance

HSTbands_wave = {'NUV':2704*u.AA,'U':3355*u.AA,'B':4325*u.AA,'V':5308*u.AA,'I':8024*u.AA}
freq_to_wave = lambda band: u.mJy.to(u.erg/u.s/u.cm**2/u.Angstrom,equivalencies=u.spectral_density(HSTbands_wave[band]))

extinction_model = O94(Rv=3.1)

def extinction(EBV,EBV_err,wavelength,plot=False):
    '''Calculate the extinction for a given EBV and wavelength with errors'''
    
    EBV = np.atleast_1d(EBV)
    sample_size = 100000

    ext = extinction_model.extinguish(wavelength,Ebv=EBV)
    
    EBV_rand = np.random.normal(loc=EBV,scale=EBV_err,size=(sample_size,len(EBV)))
    ext_arr  = extinction_model.extinguish(wavelength,Ebv=EBV_rand)
        
    ext_err  = np.std(ext_arr,axis=0)
    ext_mean = np.mean(ext_arr,axis=0)
 
    return ext,ext_err


# nebulae catalogue from Francesco (mostly HII-regions)
with fits.open(data_ext / 'Products' / 'Nebulae catalogue' / 'Nebulae_catalogue_v2.fits') as hdul:
    nebulae = Table(hdul[1].data)
nebulae['NUV_FLUX'] = np.nan
nebulae['NUV_FLUX_ERR'] = np.nan
nebulae['NUV_FLUX_CORR'] = np.nan
nebulae['NUV_FLUX_CORR_ERR'] = np.nan

In [ ]:
gal_name = 'NGC1365'

print(f'start with {gal_name}')
p = {x:sample_table.loc[gal_name][x] for x in sample_table.columns}

filename = data_ext / 'MUSE'/ 'DR2.1' / 'MUSEDAP' / f'{gal_name}_MAPS.fits'
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))

filename = data_ext / 'Products' / 'Nebulae catalogue' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan

print(f'read in nebulae catalogue')

# NUV image
filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
error_file = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_err-drc-wht.fits'

with fits.open(filename) as hdul:
    F275 = NDData(hdul[0].data,
                    mask=hdul[0].data==0,
                    meta=hdul[0].header,
                    wcs=WCS(hdul[0].header))
    with fits.open(error_file) as hdul:
        F275.uncertainty = InverseVariance(hdul[0].data)
print(f'read in HST data')

In [ ]:
from astrotools.regions import Regions
from reproject import reproject_interp

muse_regions = Regions(mask=nebulae_mask.data,projection=nebulae_mask.meta,bkg=-1)
hst_regions = muse_regions.reproject(F275.meta)
print('regions reprojected')

muse_reproj, footprint = reproject_interp((nebulae_mask.mask,nebulae_mask.wcs),F275.meta)
mean,median,std=sigma_clipped_stats(F275.data[footprint.astype(bool)])
print('measuring sigma_clipped_stats')

tmp = nebulae[nebulae['gal_name']==gal_name]

std_err_map = np.sqrt(1/F275.uncertainty.array)

flux = np.array([np.sum(F275.data[hst_regions.mask==ID]) for ID in tmp['region_ID']])
err  = np.array([np.sqrt(np.sum(std_err_map[hst_regions.mask==ID]**2)) for ID in tmp['region_ID']])

# convert counts to physical units
flux = np.array(flux) * F275.meta['PHOTFLAM']
err  = np.array(err) * F275.meta['PHOTFLAM']
print('measuring flux')

# E(B-V) is estimated from nebulae. E(B-V)_star = 0.5 E(B-V)_nebulae. NUV comes directly from stars
extinction_mw  = extinction_model.extinguish(2704*u.angstrom,Ebv=0.5*p['E(B-V)'])
ext_int,ext_int_err = extinction(0.5*tmp['EBV'],tmp['EBV_ERR'],wavelength=2704*u.angstrom)

nebulae['NUV_FLUX'][nebulae['gal_name']==gal_name] = 1e20*flux / extinction_mw
nebulae['NUV_FLUX_ERR'][nebulae['gal_name']==gal_name] = 1e20*err / extinction_mw

nebulae['NUV_FLUX_CORR'][nebulae['gal_name']==gal_name] = 1e20*flux / extinction_mw / ext_int 
nebulae['NUV_FLUX_CORR_ERR'][nebulae['gal_name']==gal_name] =  nebulae['NUV_FLUX_CORR'][nebulae['gal_name']==gal_name] *np.sqrt((err/flux)**2 + (ext_int_err/ext_int)**2)  

print('extinction correction and write to catalogue\n')

### Errors of FUV

In [ ]:
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_fuv.fits') as hdul:
    fuv = Table(hdul[1].data)
    
with fits.open(basedir/'data'/'interim'/'old'/f'Nebulae_Catalogue_v2p1_fuv.fits') as hdul:
    fuv_old = Table(hdul[1].data)

In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))

tmp = fuv_old[fuv_old['gal_name']=='NGC3627']
ax1.hist(tmp['FUV_FLUX'],bins=np.logspace(2,5,12))
ax1.set(xscale='log')

tmp = fuv[fuv['gal_name']=='NGC3351']
ax2.hist(tmp['FUV_FLUX'],bins=np.logspace(2,5,12))
ax2.set(xscale='log')

plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column))

lim = [1e2,1e6]

fuv_flux_old = fuv_old['FUV_FLUX_CORR'][fuv['gal_name']!='NGC3351']
fuv_flux_new = fuv['FUV_FLUX_CORR'][fuv['gal_name']!='NGC3351']
ax.scatter(fuv_old['FUV_FLUX_CORR'][fuv['gal_name']=='NGC3351'],fuv['FUV_FLUX'][fuv['gal_name']=='NGC3351'],alpha=0.6,label='NGC3351',s=0.8)
ax.scatter(fuv_flux_old,fuv_flux_new,alpha=0.6,label='not NGC3351',s=0.8)
print(f'difference {100*np.nanmean((fuv_flux_old-fuv_flux_new)/fuv_flux_old):.2f}%')
ax.plot(lim,lim,color='black')
ax.legend()

ax.set(xscale='log',yscale='log',xlim=lim,ylim=lim,
       xlabel='FUV old',ylabel='FUV new')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

bins = np.logspace(-1,2)
ax.hist(fuv['FUV_FLUX_CORR']/fuv['FUV_FLUX_CORR_ERR'],bins=bins)
ax.set(xscale='log',xlim=[0.1,1e2],xlabel=r'FUV/ $\delta$FUV')
mean_ston = np.nanmean(fuv['FUV_FLUX_CORR']/fuv['FUV_FLUX_CORR_ERR'])
ax.set_title(f'mean S/N={mean_ston:.2f}')
plt.savefig(basedir/'reports'/'benchmarks'/'FUV_StoN.pdf',dpi=300)
plt.show()

In [ ]:
fuv['FUV_FLUX_ERR_NEW'] = np.nan
fuv['IntTime'] = np.nan
fuv['CTSTOFLUX'] = np.nan
for gal_name in np.unique(fuv['gal_name']):
    astro_file = data_ext /'Astrosat' / f'{gal_name}_FUV_F148W_flux_reproj.fits'
    if not astro_file.is_file():
        astro_file = data_ext /'Astrosat' / f'{gal_name}_FUV_F154W_flux_reproj.fits'
        if not astro_file.is_file():
            print(f'no astrosat file for {gal_name}')
            continue
            
    with fits.open(astro_file) as hdul:
        for row in hdul[0].header['COMMENT']:
            if row.startswith('CTSTOFLUX'):
                _,CTSTOFLUX = row.split(':')
                CTSTOFLUX = float(CTSTOFLUX)
            if row.startswith('IntTime'):
                _,IntTime = row.split(':')
                IntTime = float(IntTime)
        header=hdul[0].header
    break

In [ ]:
 np.nanmean(100*(fuv['FUV_FLUX_ERR']-fuv['FUV_FLUX_ERR_NEW'])/fuv['FUV_FLUX_ERR'])

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column))

lim = [1e2,2e3]

for gal_name in np.unique(fuv['gal_name']):
    tmp = fuv[fuv['gal_name']==gal_name]
    ax.scatter(tmp['FUV_FLUX_ERR'],tmp['FUV_FLUX_ERR_NEW'],c=tmp['IntTime']/60,vmin=0,vmax=100)

ax.plot(lim,lim,color='black')

ax.set(xscale='log',yscale='log',xlim=lim,ylim=lim)
plt.show()


In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column))

lim = [5e2,5e8]

ax.scatter(nebulae['HA6562_FLUX_CORR'],nebulae['HA_conv_FLUX_CORR'])

ax.plot(lim,lim,color='black')

ax.set(xscale='log',yscale='log',xlim=lim,ylim=lim,
       xlabel=r'H$\alpha$ catalogue',ylabel=r'H$\alpha$ from convolved image')
plt.savefig(basedir/'reports'/'benchmarks'/'Halpha_convolved.pdf',dpi=300)
plt.show()


In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column))

lim = [5e-3,5e5]

ax.scatter(nebulae['HA6562_FLUX_ERR'],nebulae['HA_conv_FLUX_ERR'])

ax.plot(lim,lim,color='black')

ax.set(xscale='log',yscale='log',xlim=lim,ylim=lim,
       xlabel=r'H$\alpha$ Error catalogue',ylabel=r'H$\alpha$ Error from convolved image')
#plt.savefig(basedir/'reports'/'benchmarks'/'Halpha_convolved.pdf',dpi=300)
plt.show()


In [ ]:
fig,ax=plt.subplots()

sc = ax.scatter(fuv['FUV_FLUX'],fuv['FUV_FLUX']/fuv['FUV_FLUX_ERR'],c=fuv['IntTime']/60)
ax.set(xscale='log',xlim=[7e1,2e6],yscale='log',ylim=[1,100],
       xlabel='FUV flux',ylabel='S/N FUV')
fig.colorbar(sc,label='Integration time / min')
plt.show()

In [ ]:
from astrotools.regions import Regions

# nebulae catalogue from Francesco (mostly HII-regions)
with fits.open(data_ext / 'Products' / 'Nebulae catalogue'/ 'Nebulae_catalogue_v2.fits') as hdul:
    nebulae = Table(hdul[1].data)
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg,frame='icrs')

nebulae['FUV_FLUX'] = np.nan
nebulae['FUV_FLUX_ERR'] = np.nan
nebulae['FUV_FLUX_CORR'] = np.nan
nebulae['FUV_FLUX_CORR_ERR'] = np.nan

nebulae['HA_conv_FLUX'] = np.nan
nebulae['HA_conv_FLUX_ERR'] = np.nan
nebulae['HA_conv_FLUX_CORR'] = np.nan
nebulae['HA_conv_FLUX_CORR_ERR'] = np.nan


gal_name = 'NGC1566'
        
print(f'start with {gal_name}')

print(f'read in nebulae catalogue')
filename = next((data_ext/'MUSE'/'DR2.1'/'copt').glob(f'{gal_name}*.fits'))
copt_res = float(filename.stem.split('-')[1].split('asec')[0])
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))

filename = data_ext / 'Products' / 'Nebulae catalogue' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan

print(f'read in astrosat data')
astro_file = data_ext /'Astrosat' / f'{gal_name}_FUV_F148W_flux_reproj.fits'
if not astro_file.is_file():
    astro_file = data_ext /'Astrosat' / f'{gal_name}_FUV_F154W_flux_reproj.fits'
    if not astro_file.is_file():
        print(f'no astrosat file for {gal_name}')

with fits.open(astro_file) as hdul:
    d = hdul[0].data
    astrosat = NDData(hdul[0].data,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    for row in hdul[0].header['COMMENT']:
        if row.startswith('CTSTOFLUX'):
            _,CTSTOFLUX = row.split(':')
            CTSTOFLUX = float(CTSTOFLUX)
        if row.startswith('IntTime'):
            _,IntTime = row.split(':')
            IntTime = float(IntTime)


print('reproject regions')
muse_regions = Regions(mask=nebulae_mask.data,projection=nebulae_mask.meta,bkg=-1)
astrosat_regions = muse_regions.reproject(astrosat.meta)

tmp = nebulae[nebulae['gal_name']==gal_name]

In [ ]:
from skimage.measure import find_contours

fig,(ax1,ax2) = plt.subplots(ncols=2,figsize=(two_column,two_column/2))

position = tmp[1]['SkyCoord']
size = 5*u.arcsec
Cutout_Halpha = Cutout2D(Halpha.data,position,size=size,wcs=Halpha.wcs)
Cutout_FUV = Cutout2D(astrosat.data,position,size=size,wcs=astrosat.wcs)
Cutout_mask = Cutout2D(astrosat_regions.mask,position,size=size,wcs=astrosat.wcs)
Cutout_mask_neb = Cutout2D(muse_regions.mask,position,size=size,wcs=Halpha.wcs)

norm = simple_norm(Cutout_Halpha.data,clip=False,percent=99)
ax1.imshow(Cutout_Halpha.data,norm=norm,origin='lower',cmap=plt.cm.Greens)
ax1.axis('off')

contours = []
region_ID = np.unique(Cutout_mask_neb.data[~np.isnan(Cutout_mask_neb.data)])
for i in region_ID:
    blank_mask = np.zeros_like(Cutout_mask_neb.data)
    blank_mask[Cutout_mask_neb.data==i] = 1
    contours += find_contours(blank_mask, 0.5)

for coords in contours:
    ax1.plot(coords[:,1],coords[:,0],color='tab:red',lw=1,label='association')

norm = simple_norm(Cutout_FUV.data,clip=False,percent=99)
ax2.imshow(Cutout_FUV.data,norm=norm,origin='lower',cmap=plt.cm.Blues)
ax2.axis('off')

ax2.imshow(Cutout_mask.data,origin='lower',cmap=plt.cm.gray,alpha=0.6)

contours = []
region_ID = np.unique(Cutout_mask.data[~np.isnan(Cutout_mask.data)])
for i in region_ID:
    blank_mask = np.zeros_like(Cutout_mask.data)
    blank_mask[Cutout_mask.data==i] = 1
    contours += find_contours(blank_mask, 0.5)

for coords in contours:
    ax2.plot(coords[:,1],coords[:,0],color='tab:red',lw=1,label='association')

plt.show()

In [ ]:
2.5*np.log10(np.sqrt(2*np.pi))+2.5*np.log10(std) + 2.5/np.log(10)*(x-mu)**2/(2*std**2)-13.74

In [ ]:
x = np.linspace(0.5e-17,1.5e-17)
plt.plot(-2.5*np.log10(x)-13.74,norm.pdf(x,1e-17,1e-18),color='blue')
plt.xlim((28,29.5))

plt.show()

### Extinction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def extinction(EBV,EBV_err,wavelength,plot=False):
    '''Calculate the extinction for a given EBV and wavelength with errors'''
    
    EBV = np.atleast_1d(EBV)
    EBV_err = np.atleast_1d(EBV_err)
    sample_size = 100000
    
    ext = pn.RedCorr(R_V=3.1,E_BV=EBV,law='CCM89 oD94').getCorr(wavelength)
    
    EBV_rand = np.random.normal(loc=EBV,scale=EBV_err,size=(sample_size,len(EBV)))
    ext_arr  = pn.RedCorr(R_V=3.1,E_BV=EBV_rand,law='CCM89 oD94').getCorr(wavelength)
    
    ext_err  = np.std(ext_arr,axis=0)
    ext_mean = np.mean(ext_arr,axis=0)
    
    if plot:
        fig,(ax1,ax2) =plt.subplots(nrows=1,ncols=2,figsize=(6,6/2))
        ax1.hist(EBV_rand[:,0],bins=100)
        ax1.axvline(EBV[0],color='black')
        ax1.set(xlabel='E(B-V)')
        ax2.hist(ext_arr[:,0],bins=100)
        ax2.axvline(ext[0],color='black')
        ax2.set(xlabel='extinction')
        plt.show()
 
    return ext,ext_err

extinction(EBV=[0.089,0.3],EBV_err=[0.01,0.01],wavelength=5007,plot=True)

### Overlapping Circles

https://math.stackexchange.com/questions/3543367/area-of-overlap-of-two-circles


In [ ]:
def overlap(r1,r2,x):
    h=1/(2*x)*np.sqrt(2*x**2*r1**2+2*x**2*r2**2+2*r1**2*r2**2-x**4-r1**4-r2**4)

    return r1**2*np.arcsin(h/r1)+r2**2*np.arcsin(h/r2)-x*h


def draw_overlap(n1,n2,n_overlap):
    '''
    
    
    '''
    
    if n1<n2:
        n1,n2=n2,n1
    if n_overlap>n2/2:
        raise ValueError('overlap can not be larger than individual samples')
    
    r1 = 1
    r2 = np.sqrt(n2/n1)

    separation = np.linspace(r1+r2,r1)
    area = overlap(r1,r2,separation)
    x = np.interp(np.sqrt(n_overlap/n1),area,separation)
    
    fig,ax = plt.subplots(figsize=(single_column,single_column/1.618))
    patch = plt.Circle((0,0 ),r1,alpha=0.5,color='tab:red')
    ax.add_artist(patch)
    patch = plt.Circle((x,0),r2,alpha=0.5,color='tab:blue') 
    ax.add_artist(patch)

    ax.set_aspect(1) 
    ax.axis('off')
    ax.set(xlim=[-1.2,x+r2+0.2],ylim=[-1.2,1.2])
    plt.show()
    
draw_overlap(23000,17000,4000)

In [ ]:
tmp = assoc_tmp[assoc_tmp['gal_name']=='NGC2835']

print(f'1to1: {np.sum(tmp["1to1"])}')
print(f'contained: {np.sum(tmp["overlap"]=="contained")}')
print(f'partial: {np.sum(tmp["overlap"]=="partial")}')
print(f'isolated: {np.sum(tmp["overlap"]=="isolated")}')

In [ ]:
from scipy.stats import spearmanr
from astrotools.plot.utils import bin_stat
from astrotools.plot.corner import corner_binned_percentile
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig,ax1=plt.subplots(nrows=1,figsize=(single_column,single_column/1.618))

# ----------------------- ax1 ------------------------------------------
tmp = catalogue[catalogue['gal_name']=='NGC2835']

q1 = 68
q2 = 98

xlim =[8e2,1e5]
ylim = [1e4,5e7]
ylim = [1e36,8e39]
vmin,vmax = 0,15
nbins = 6
bins = np.logspace(2.7,5.2,nbins)

cmap = mpl.cm.get_cmap('plasma',6)
norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)
rho = []
age_bins = [0,5,10,15]
for i in range(len(age_bins)-1):

    sub = tmp[(tmp['age']>=age_bins[i]) & (tmp['age']<age_bins[i+1])]
    x,y = sub['mass'],sub['HA6562_LUM_CORR']
    x,mean,std = bin_stat(x,y,bins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_bins[i]+1)))

x,y = tmp['mass'],tmp['HA6562_LUM_CORR']
x,y = x[~np.isnan(y) & np.isfinite(y)],y[~np.isnan(y) & np.isfinite(y)]
corner_binned_percentile(x,y,ax1,nbins=bins,n=68,color='gray',alpha=0.6)
corner_binned_percentile(x,y,ax1,nbins=bins,n=98,color='gray',alpha=0.3)

rho = spearmanr(x,y,nan_policy='omit')[0]
label = r'$\rho'+f'={np.nanmean(rho):.2f}$'
t = ax1.text(0.05,0.93,label,transform=ax1.transAxes,ha='left',va='top')
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

ax1.set(xscale='log',yscale='log',xlim=xlim,ylim=ylim,
        xlabel=r'mass / M$_\odot$',ylabel=r'$F(\mathrm{H}\alpha)$ / erg s$^{-1}$')

# we need a scatter plot instance for the color bar
sc = ax1.scatter(19*[1],19*[1],c=19*[1],cmap=cmap,vmin=vmin,vmax=vmax)
#divider = make_axes_locatable(ax1)
#cax = divider.append_axes('top', size="10%", pad=0.3)
#cbar = fig.colorbar(sc,label='age / Myr',cax=cax,orientation='horizontal')
#cbar.ax.xaxis.set_ticks_position('top')

cbar = fig.colorbar(sc,label='age / Myr')
plt.show()

old vs new nebulae catalogue

In [ ]:
# the original catalogue from Francesco
with fits.open(data_ext / 'Products' / 'Nebulae_catalogs' / 'Nebulae_catalogue_v2' /'Nebulae_catalogue_v2.fits') as hdul:
    nebulae_old = Table(hdul[1].data)
with fits.open(data_ext / 'Products' / 'Nebulae_catalogs' / 'Nebulae_catalogue_v3' /'Nebulae_Catalogue_v3.fits') as hdul:
    nebulae_new = Table(hdul[1].data)
    
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_eq.fits') as hdul:
    eq_width = Table(hdul[1].data)
tmp = join(nebulae_new,eq_width,keys=['gal_name','region_ID'])
    

In [ ]:
fig,ax=plt.subplots()

lim = [0,100]
ax.scatter(tmp['eq_width'],tmp['EW_HA'])
ax.plot(lim,lim,color='black')
ax.set(xlim=lim,ylim=lim)
plt.show()

In [ ]:
fig,ax=plt.subplots()

lim = [0,500]
ax.scatter(nebulae_old['HA6562_FLUX']/nebulae_old['HA6562_FLUX_ERR'],nebulae_new['HA6562_FLUX']/nebulae_new['HA6562_FLUX_ERR'])
ax.plot(lim,lim,color='black')
ax.set(xlim=lim,ylim=lim)
plt.show()

In [ ]:
t = tmp #[tmp['HA6562_FLUX']>tmp['HA6562_FLUX_ERR']]
f"{np.mean(t['eq_width']-t['EW_HA']):.2f}+-{np.std(t['eq_width']-t['EW_HA']):.2f}"

In [ ]:
def area(r0,r1,x0,x1):
    
    fig,ax=plt.subplots()
    
    
    

### Others

In [ ]:
eq_widths = Table.read(basedir/'data'/'interim'/'Nebulae_Catalogue_v2p1_EW.fits')
eq_widths['EW_HA'] = eq_widths['Halpha']/eq_widths['continuum_Halpha']
eq_widths['EW_HA_corr'] = eq_widths['Halpha']/(eq_widths['continuum_Halpha']-eq_widths['continuum_Halpha_bkg'])

with fits.open(basedir / 'data' / 'interim' / 'Nebulae_Catalogue_v3.fits') as hdul:
    nebulae = Table(hdul[1].data)

In [ ]:
fig,(ax1,ax2)=plt.subplots(figsize=(two_column,two_column/2),ncols=2)

lim = [0,300]

ax1.scatter(nebulae['EW_HA'],eq_widths['EW_HA'])
ax1.plot(lim,lim,color='black')
ax1.set(xlim=lim,ylim=lim,xlabel='EW Kathryn',ylabel='EW Fabian')

tmp = eq_widths[((eq_widths['continuum_Halpha']-eq_widths['continuum_Halpha_bkg'])>eq_widths['continuum_Halpha_error'])]
print(len(tmp))
ax2.scatter(tmp['EW_HA'],tmp['EW_HA_corr'])
ax2.plot(lim,lim,color='black')
ax2.set(xlim=lim,ylim=[0,3000],xlabel='EW original',ylabel='EW corrected')

plt.tight_layout()
plt.savefig(basedir/'reports'/'ew_corrected.png',dpi=400)
plt.show()

In [ ]:
fig,ax=plt.subplots()

bins = np.logspace(2,5,16)
ax.hist(fuv['FUV_FLUX'],bins=bins,alpha=0.5)
ax.hist(fuv['FUV_BKG_FLUX'],bins=bins,alpha=0.5)
ax.set(xscale='log')

plt.show()

In [ ]:
print(f"EW: {np.mean(catalogue['continuum_Halpha_bkg']/catalogue['continuum_Halpha'])*100:.1f}")
print(f"FUV: {np.mean(catalogue['FUV_BKG_FLUX']/catalogue['FUV_FLUX'])*100:.1f}")

In [ ]:
print(f"EW: {np.mean(nebulae['continuum_Halpha_bkg']/nebulae['continuum_Halpha'])*100:.1f} %")
print(f"FUV: {np.mean(nebulae['FUV_BKG_FLUX']/nebulae['FUV_FLUX'])*100:.1f} %")

In [ ]:
def extinction(EBV,EBV_err,wavelength,plot=False):
    '''Calculate the extinction for a given EBV and wavelength with errors
    
    Parameters
    ----------

    EBV : array

    EBV_err : array

    wavelength : float
    
    '''
    
    EBV = np.atleast_1d(EBV)
    EBV_err = np.atleast_1d(EBV_err)
    sample_size = 50000
    
    ext = pn.RedCorr(R_V=3.1,E_BV=EBV,law='CCM89 oD94').getCorr(wavelength)
    
    EBV_rand = np.random.normal(loc=EBV,scale=EBV_err,size=(sample_size,len(EBV)))
    ext_arr  = pn.RedCorr(R_V=3.1,E_BV=EBV_rand,law='CCM89 oD94').getCorr(wavelength)
    
    ext_err  = np.std(ext_arr,axis=0)
    ext_mean = np.mean(ext_arr,axis=0)
    
    return ext,ext_err

In [ ]:
pn.RedCorr(R_V=3.1,E_BV=0.044,law='CCM89 oD94').getCorr(1481)

In [ ]:
import pyneb as pn
from tqdm import tqdm

with fits.open(basedir / 'data' / 'interim' / 'Nebulae_Catalogue_v3.fits') as hdul:
    nebulae = Table(hdul[1].data)
fuv = Table.read(basedir/'data'/'interim'/'Nebulae_Catalogue_v2p1_FUV_reproj_old.fits')
fuv = join(fuv,nebulae[['gal_name','region_ID','EBV','EBV_ERR']],keys=['gal_name','region_ID'])

# Milky Way E(B-V) from  Schlafly & Finkbeiner (2011)
EBV_MW = {'IC5332': 0.015,'NGC0628': 0.062,'NGC1087': 0.03,'NGC1300': 0.026,
          'NGC1365': 0.018,'NGC1385': 0.018,'NGC1433': 0.008,'NGC1512': 0.009,
          'NGC1566': 0.008,'NGC1672': 0.021,'NGC2835': 0.089,'NGC3351': 0.024,
          'NGC3627': 0.037,'NGC4254': 0.035,'NGC4303': 0.02,'NGC4321': 0.023,
          'NGC4535': 0.017,'NGC5068': 0.091,'NGC7496': 0.008}

#fuv['FUV_FLUX_CORR2'] = np.nan
#fuv['FUV_FLUX_CORR2_ERR'] = np.nan

for gal_name in tqdm(np.unique(fuv['gal_name'])):
    
    tmp = fuv[fuv['gal_name']==gal_name]

    EBV_correction = 1
    rc_MW = pn.RedCorr(R_V=3.1,E_BV=EBV_correction*EBV_MW[gal_name],law='CCM89 oD94')
    
    ext_int,ext_int_err = extinction(EBV_correction*tmp['EBV'],tmp['EBV_ERR'],wavelength=1481)    

    fuv['FUV_FLUX_CORR'][fuv['gal_name']==gal_name] = tmp['FUV_FLUX'] * ext_int 
    fuv['FUV_FLUX_CORR_ERR'][fuv['gal_name']==gal_name] =  fuv['FUV_FLUX_CORR'][fuv['gal_name']==gal_name] *np.sqrt((fuv['FUV_FLUX_ERR'][fuv['gal_name']==gal_name]/fuv['FUV_FLUX'][fuv['gal_name']==gal_name])**2 + (ext_int_err/ext_int)**2)  

    

In [ ]:
from datetime import date
    
fuv[['EBV','EBV_ERR']]

doc = f'''this catalogue contains the FUV fluxes for the objects in the nebula 
catalogue, measured from the Astrosat data (using the F148W filter for
all galaxies except for NGC1433 and NGC1512, for which the F154W filter
was used). All fluxes are in [f]=1e-20 erg s-1 cm-2 AA-1 and corrected 
for Milky Way foreground extinction (with the extinction curve from 
O'Donnell (1994) and E(B-V) from Schlafly & Finkbeiner (2011)). The 
columns ending with _CORR are also corrected for internal extinction, 
based on the E(B-V) from the nebula catalogue. 
Based on the nebula catalogue v2p0. 
This catalogue was created with the following script:
https://github.com/fschmnn/cluster/blob/master/scripts/measure_FUV.py
last update: {date.today().strftime("%b %d, %Y")}
'''

primary_hdu = fits.PrimaryHDU()
for i,comment in enumerate(doc.split('\n')):
    if i==0:
        primary_hdu.header['COMMENT'] = comment
    else:
        primary_hdu.header[''] = comment
table_hdu   = fits.BinTableHDU(fuv)
hdul = fits.HDUList([primary_hdu, table_hdu])
hdul.writeto('Nebulae_Catalogue_v2p1_FUV_reproj.fits',overwrite=True)

In [ ]:
import pyneb as pn

Av1 = pn.RedCorr(R_V=3.1,E_BV=nebulae['EBV_balmer'],law='CCM89 oD94').getCorr(1481)
Av044 = pn.RedCorr(R_V=3.1,E_BV=0.44*nebulae['EBV_balmer'],law='CCM89 oD94').getCorr(1481)

In [ ]:
pn.RedCorr(R_V=3.1,E_BV=0.44*2,law='CCM89 oD94').getCorr(1481)

In [ ]:
from astrotools.plot import compare

ax=compare(Av1,Av044,lim=[0.1,1e5])
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(single_column,single_column/1.618))

ax.plot(nebulae['EBV_balmer'],Av1,label=r'$X=1$')
ax.plot(nebulae['EBV_balmer'],Av044,label=r'$X=0.44$')
ax.legend()
#ax.scatter(nebulae['EBV_balmer'],Av044)
ax.set(yscale='log',xlim=[0,1],ylim=[0.8,8e3],xlabel=r'$E(B-V)$',ylabel='RedCorr')
plt.show()

In [ ]:
fuv = Table.read(basedir/'data'/'interim'/'Nebulae_Catalogue_v2p1_FUV_bkg.fits')


fig,ax=plt.subplots()
ax.hist(100*fuv['FUV_BKG_FLUX']/fuv['FUV_FLUX'],bins=np.linspace(0,100,20))

plt.show()